# Introduction to LangChain and RAG

Welcome to the first part of our workshop! In this session, we'll explore how to build AI-powered applications using **LangChain**, a popular framework for developing applications with Large Language Models (LLMs). We'll start with a simple chatbot and then enhance it with Retrieval Augmented Generation (RAG).

## Setting Up Our Environment

First, we need to set up our environment. We'll use OpenAI's models, so we need an API key. You can define your `OPENAI_API_KEY` in the `.env` file.

The code retrieve the key and sets some global configurations:
- `LLM_MODEL`: The specific model we'll use
- `LLM_TEMPERATURE`: Controls randomness in responses (0 means very deterministic)

In [297]:
import os

In [298]:
if not os.environ.get("OPENAI_API_KEY"):
    raise ValueError("Please set OPENAI_API_KEY environment variable")

LLM_MODEL = "gpt-4o-mini"
LLM_TEMPERATURE = 0

## Building a Simple ChatBot

Let's start with creating a basic chatbot using **LangChain**. We'll use:
- `ChatOpenAI`: The interface to OpenAI's chat models
- `SystemMessage`: Defines the bot's behavior and role
- `HumanMessage`: Represents user input

Our chatbot will act as a Financial Analyst. We'll create it by:
1. Instantiating the model
2. Defining a system prompt that sets the bot's role
3. Sending a user query and getting a response with `.invoke()`

This demonstrates the basic pattern of LLM interactions: prompt → response.

In [299]:
from IPython.display import Markdown
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

In [300]:
# Create a ChatOpenAI instance with the LLM model and temperature
base_model = ChatOpenAI(model=LLM_MODEL, temperature=LLM_TEMPERATURE)

In [301]:
BASE_PROMPT = """
You are a Financial Analyst with access to a Bloomberg Financial News database.

Start by identifying industry relevant to the sector you want to invest into. Think of upstream and downstream industries, eg if you want to invest in Generative AI think of upstream GPU manufacturers and downstream industries most impacted by GenAI for instance call centers.

Identify the key players and their challengers for that sector based on upstream and downstream industries analysis.

Create a table with an investment recommendation for each stock identified (buy, hold, sell) and the reason why rooted in the news extracted from the database. Quote the source and date of the news you refer to support the investment strategy for each stock. Present this in a table format.

Query the database to extract stock information.

Summarize the investment strategy in an executive summary."""

In [302]:
# Request from the client
request = "I want to invest in Generative AI?"

# Message list for the base model
messages = [
    SystemMessage(BASE_PROMPT),
    HumanMessage(request),
]

# Invoke the model with the messages
response = base_model.invoke(messages)

In [303]:
Markdown(response.content)

To invest in Generative AI, we need to analyze both upstream and downstream industries. 

### Upstream Industries:
1. **GPU Manufacturers**: These companies produce the hardware necessary for training and running generative AI models.
   - Key Players: NVIDIA, AMD, Intel
2. **Cloud Service Providers**: They offer the infrastructure for deploying AI applications.
   - Key Players: Amazon Web Services (AWS), Microsoft Azure, Google Cloud

### Downstream Industries:
1. **Software Companies**: These companies develop applications that utilize generative AI.
   - Key Players: OpenAI, Adobe, Salesforce
2. **Industries Impacted by Generative AI**: 
   - Call Centers (e.g., companies using AI for customer service)
   - Content Creation (e.g., marketing agencies, media companies)

### Key Players and Challengers:
- **NVIDIA**: Dominates the GPU market but faces competition from AMD and Intel.
- **OpenAI**: Leading in generative AI software but faces competition from Google DeepMind and Anthropic.
- **AWS**: Leading in cloud services but competes with Microsoft Azure and Google Cloud.

### Investment Recommendation Table:

| Stock          | Recommendation | Reason                                                                 | Source                          | Date       |
|----------------|----------------|------------------------------------------------------------------------|---------------------------------|------------|
| NVIDIA         | Buy            | Strong demand for GPUs in AI applications; recent partnerships with major tech firms. | Bloomberg News                  | Oct 15, 2023 |
| AMD            | Hold           | Competitive products but facing supply chain challenges.               | Bloomberg News                  | Oct 10, 2023 |
| Intel          | Sell           | Struggling to keep up with competitors in the AI hardware space.       | Bloomberg News                  | Oct 12, 2023 |
| OpenAI         | Buy            | Leading position in generative AI with strong growth potential.        | Bloomberg News                  | Oct 14, 2023 |
| Adobe          | Buy            | Integrating generative AI into creative tools, driving user engagement. | Bloomberg News                  | Oct 11, 2023 |
| AWS            | Buy            | Continues to lead in cloud services with strong AI offerings.          | Bloomberg News                  | Oct 13, 2023 |
| Microsoft Azure| Buy            | Significant investments in AI and strong growth in cloud services.     | Bloomberg News                  | Oct 16, 2023 |

### Executive Summary:
Investing in Generative AI presents a promising opportunity, particularly through key players in both upstream and downstream industries. NVIDIA and OpenAI are recommended as strong buys due to their leadership and innovation in the sector. Adobe and AWS also show robust growth potential, making them attractive investments. AMD is rated as a hold, while Intel is advised to be sold due to its struggles in the competitive landscape. This investment strategy is supported by recent news highlighting the growth and demand for generative AI technologies and the companies leading this charge.

## Understanding Retrieval Augmented Generation (RAG)

Now comes the exciting part! RAG is a technique that enhances LLM responses by giving them access to external knowledge. Instead of relying solely on the model's training data, we can provide relevant information from our own database.

### Vector Database Setup

To implement RAG, we need:
1. A collection of documents (in our case, a currated set of 1'000 articles from Bloomberg financial news)
2. A way to search these documents efficiently (vector database)
3. A function to retrieve relevant information based on queries

Here we use:
- `Chroma`: A vector database for storing and retrieving documents
- `OpenAIEmbeddings`: Converts text into vector representations

Let's first set up the global configuration for our retriever.

In [304]:
EMBEDDING_MODEL = "text-embedding-3-small"
RETRIEVAL_K = 10

We'll then define helper functions to load our documents and store them in a vector store.

In [305]:
import pickle

from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

In [306]:
def load_documents(pickle_filepath: str) -> list[Document]:
    """Load documents from a pickle file."""
    with open(pickle_filepath, "rb") as file:
        return pickle.load(file)


def initialize_vector_store(document_chunks: list[Document]) -> Chroma:
    """Reset the Chroma collection and initialize a vector store using document chunks."""
    Chroma().reset_collection()
    embedding_model = OpenAIEmbeddings(model=EMBEDDING_MODEL)
    return Chroma.from_documents(documents=document_chunks, embedding=embedding_model)

Let's load our documents and inspect the first one.

In [307]:
data_dir = "/content/data/"
data_file = "bloomberg_financial_news_1k.pkl"

# Load the Bloomberg documents from the pickle file
documents = load_documents(os.path.join(data_dir, data_file))

In [308]:
doc_str = f"{documents[0].metadata['Headline']}\n\n{documents[0].page_content}"
Markdown(doc_str)

Kuwait Wealth Fund Doubles Investments in U.K. to $24 Billion

Kuwait Investment Authority , the country’s sovereign wealth fund, doubled investments in the U.K. over the last 10 years to more than $24 billion. Investment in Britain was “across all asset classes, sectors and industries,” Managing Director Bader Al-Saad said in a speech in London on June 28. Kuwait Investment Office, the KIA’s London-based investment arm, now manages more than $120 billion globally compared with $27 billion 10 years ago. “Alongside the real estate subsidiary, which manages our real estate assets, we have set up an infrastructure fund which we are operating from London,” Al-Saad said. The investment office “will be looking to manage more funds, in more asset classes, and in more regions of the world in the near future,” he said. The KIA, which has about $342 billion of assets according to the Sovereign Wealth Fund Institute in California , holds stakes in Daimler AG and BP Plc. (BP/) The fund manages Kuwait’s General Reserve Fund and Future Generations Fund, and invests in local, Arab and international markets.

### Initializing the Vector Store and the Retriever

The vector store and retriever are key components of our RAG system. Here's what happens in this section:

- Initialize a new Chroma vector store with these documents
- Create a retriever that will fetch the `RETRIEVAL_K` most relevant documents according to their embedding

In [309]:
# Initialize the vector store with the documents
vector_store = initialize_vector_store(documents)

# Create a retriever instance from the vector store
retriever = vector_store.as_retriever(search_kwargs={"k": RETRIEVAL_K})

The retriever acts like a smart search engine - when given a question or topic, it returns the most relevant documents from our database. It does so by finding the documents similar embeddings to the query. In LangChain, this is also done with `.invoke()`. Let's try an example.

In [328]:
retrieval_query = "Generative AI"

# Invoke the retriever with the retrieval query
retrieved_documents = retriever.invoke(retrieval_query)

In [329]:
display(retrieved_documents)

[Document(id='3205f445-1423-4331-ab67-898a9f493edc', metadata={'Headline': 'Nissan Opens Silicon Valley Center for Self-Driving Car Research', 'Link': 'http://www.bloomberg.com/news/2013-02-18/nissan-opens-silicon-valley-center-for-self-driving-car-research.html'}, page_content='Nissan Motor Co. opened a research center in California’s Silicon Valley that the Japanese carmaker plans to make its hub for research on self-driving vehicles and Internet-connected auto technology. The facility will be staffed by more than 60 engineers and technicians within three years, Carla Bailo , Nissan’s senior vice president for North American research and development, said in a phone interview. Work on so-called autonomous vehicle systems will move from an R&D center in Japan to Sunnyvale, she said, declining to provide investment details for the project. “We’re going to focus on this technology really in the heart of where it lies today,” Bailo said in the Feb. 15 interview. “We are going to be shift

### Creating the RAG System

We can now augment our basic chatbot by providing it access to the retriever using **LangChain** tools, which allow the model to:
- Query the document database if needed
- Provide an answer based on the retrieved documents

#### Creating a tool with LangChain

We can create a tool using the `@tool` decorator from **LangChain** and provide it to the model using `.bind_tools()`. The model will receive all the relevant information about the tool thanks to the decorator. This way it knows how it works and can decide when to use it.

In [330]:
from langchain_core.tools import tool

In [331]:
@tool
def retrieval(retrieval_query: str) -> list[Document]:
    """Retrieve documents based on a query."""
    return retriever.invoke(retrieval_query)


# Create a list of tools and a dictionnary of tool functions by name
tools = [retrieval]
tools_by_name = {tool.name: tool for tool in tools}

In [332]:
RAG_PROMPT = """
You are a Financial Analyst with access to a Bloomberg Financial News database.

Start by identifying industry relevant to the sector you want to invest into. Think of upstream and downstream industries, eg if you want to invest in Generative AI think of upstream GPU manufacturers and downstream industries most impacted by GenAI for instance call centers.

Identify the key players and their challengers for that sector based on upstream and downstream industries analysis.

Create a table with an investment recommendation for each stock identified (buy, hold, sell) and the reason why rooted in the news extracted from the database. Quote the source and date of the news you refer to support the investment strategy for each stock. Present this in a table format.

Query the database to extract stock information.

Summarize the investment strategy in an executive summary.

"""

# Create the RAG model by binding the base model with the retrieval tool
rag_model = base_model.bind_tools(tools)

In [333]:
request = "I want to invest in Generative AI"

# Message list for the RAG model
messages = [
    SystemMessage(RAG_PROMPT),
    HumanMessage(request),
]

# Invoke the RAG model with the messages
rag_response = rag_model.invoke(messages)

Let's check the answer. As we can see its content is empty, but a tool call has been made.

In [334]:
Markdown(f"Content: {rag_response.content}\n\nTool Calls: {rag_response.tool_calls}")

Content: To invest in Generative AI, we need to consider both upstream and downstream industries. 

### Upstream Industries:
1. **GPU Manufacturers**: Companies that produce graphics processing units (GPUs) are critical for the training and operation of generative AI models.
   - Key Players: NVIDIA, AMD, Intel

2. **Cloud Computing Providers**: Companies that provide cloud infrastructure for AI workloads.
   - Key Players: Amazon Web Services (AWS), Microsoft Azure, Google Cloud

### Downstream Industries:
1. **Software Development**: Companies that develop applications utilizing generative AI.
   - Key Players: OpenAI, Anthropic, Stability AI

2. **Customer Service**: Industries that implement generative AI for chatbots and virtual assistants.
   - Key Players: Zendesk, Salesforce, ServiceNow

3. **Content Creation**: Companies that use generative AI for content generation in marketing, media, etc.
   - Key Players: Adobe, Canva

### Key Players and Their Challengers:
- **NVIDIA**: Dominates the GPU market but faces competition from AMD and Intel.
- **Amazon (AWS)**: Leading in cloud services but competes with Microsoft Azure and Google Cloud.
- **OpenAI**: A leader in generative AI but faces competition from Anthropic and Stability AI.

Next, I will gather news articles related to these companies to form an investment recommendation table.

Tool Calls: [{'name': 'retrieval', 'args': {'retrieval_query': 'NVIDIA news'}, 'id': 'call_DyCALBQmpCjiq5e21ws3UdXz', 'type': 'tool_call'}, {'name': 'retrieval', 'args': {'retrieval_query': 'AMD news'}, 'id': 'call_N6UMvm1ihtE5QQZTvLXnoI2w', 'type': 'tool_call'}, {'name': 'retrieval', 'args': {'retrieval_query': 'Intel news'}, 'id': 'call_KgLBQaMsq2G3nhnRBpBpKRCu', 'type': 'tool_call'}, {'name': 'retrieval', 'args': {'retrieval_query': 'Amazon AWS news'}, 'id': 'call_zEJYPUasVyqW1s40osofK8uE', 'type': 'tool_call'}, {'name': 'retrieval', 'args': {'retrieval_query': 'Microsoft Azure news'}, 'id': 'call_FFMsAzQZPWdkhttgMXfHd9hJ', 'type': 'tool_call'}, {'name': 'retrieval', 'args': {'retrieval_query': 'Google Cloud news'}, 'id': 'call_sJIysB9K2FfwNrP4ouImTfgJ', 'type': 'tool_call'}, {'name': 'retrieval', 'args': {'retrieval_query': 'OpenAI news'}, 'id': 'call_mvlNlWYBIGwHjeaYFlvZljZn', 'type': 'tool_call'}, {'name': 'retrieval', 'args': {'retrieval_query': 'Anthropic news'}, 'id': 'call_MVwYIz4Y79M7m1ynlSR5P6ZG', 'type': 'tool_call'}, {'name': 'retrieval', 'args': {'retrieval_query': 'Stability AI news'}, 'id': 'call_CyhqLPkCxnYVt1qz18vZ70cJ', 'type': 'tool_call'}, {'name': 'retrieval', 'args': {'retrieval_query': 'Zendesk news'}, 'id': 'call_W3iR5rB2y2YDRXeoyul8Iioy', 'type': 'tool_call'}, {'name': 'retrieval', 'args': {'retrieval_query': 'Salesforce news'}, 'id': 'call_5y91ZEEgnskjo0xMyvgbDWb8', 'type': 'tool_call'}, {'name': 'retrieval', 'args': {'retrieval_query': 'ServiceNow news'}, 'id': 'call_PATsf6SsQqXfbJYrdmybpZWb', 'type': 'tool_call'}, {'name': 'retrieval', 'args': {'retrieval_query': 'Adobe news'}, 'id': 'call_sy8uZ2LBz8Er6ooaOudhDpLk', 'type': 'tool_call'}, {'name': 'retrieval', 'args': {'retrieval_query': 'Canva news'}, 'id': 'call_JVkztGK5RWDPe1aZZkdfbIZH', 'type': 'tool_call'}]

Let's use the retrieval tool to retrieve documents following the model's query.

In [336]:
# Check if the RAG model response contains tool calls
if rag_response.tool_calls:
    documents_str_list = []  # Store documents from each tool call
    for tool_call in rag_response.tool_calls:  # Iterate through tool calls

        # Get the tool from the tool call
        tool = tools_by_name[tool_call["name"]]

        # Invoke the tool with the tool call arguments
        documents = tool.invoke(tool_call["args"])

        # Combine the retrieved documents into a single string and append
        documents_str_list.append(
            "\n\n".join(
                [
                    f"{doc.metadata['Headline']}\n\n{doc.page_content}\n"
                    for doc in documents
                ]
            )
        )

    # Concatenate results from all tool calls (if multiple)
    documents_str = "\n\n".join(documents_str_list)

In [337]:
Markdown(documents_str)

Nissan Opens Silicon Valley Center for Self-Driving Car Research

Nissan Motor Co. opened a research center in California’s Silicon Valley that the Japanese carmaker plans to make its hub for research on self-driving vehicles and Internet-connected auto technology. The facility will be staffed by more than 60 engineers and technicians within three years, Carla Bailo , Nissan’s senior vice president for North American research and development, said in a phone interview. Work on so-called autonomous vehicle systems will move from an R&D center in Japan to Sunnyvale, she said, declining to provide investment details for the project. “We’re going to focus on this technology really in the heart of where it lies today,” Bailo said in the Feb. 15 interview. “We are going to be shifting work that’s being done in our Nissan Advanced Technical Center in Atsugi, and moving that work over to the heart of the industry.” Automakers and companies including Google Inc. plan new electronics and programming for vehicles with a goal of reducing collisions, fuel use and improving traffic flow. Carlos Ghosn , Nissan’s chief executive officer, told reporters last month in Detroit that regulatory, legal and technological hurdles mean self-driving cars won’t be feasible until about 2020. Google has tested a self-driving Prius in California for years, and the U.S. Defense Department has sponsored autonomous- vehicle research for more than a decade. Nissan’s Sunnyvale center will work with automotive labs at Stanford University, the University of California’s Berkeley and Davis campuses, and Silicon Valley companies, Bailo said. The Yokohama-based carmaker plans to apply technology developed at the new R&D center within a decade, she said. Maarten Sierhuis , a former NASA scientist specializing in artificial intelligence research, will lead the Sunnyvale facility, Nissan said today. Toyota Motor Corp. opened its own Silicon Valley outpost in Mountain View in 2012 to work with technology companies there on in-car data and entertainment systems. The company said in January that it’s developing autonomous safety systems to create a virtual “co-pilot” in vehicles to help drivers avoid accidents rather than self-driving cars and trucks.


Fusion-io Surges on Sales Forecast and Acquisition of NexGen

Fusion-io Inc. (FIO) , a maker of data- storage computers, climbed the most in eight months after forecasting sales that topped analysts’ estimates and announcing the acquisition of NexGen Storage Inc. Revenue in the fourth quarter, which ends in June, will be about $110 million, the Salt Lake City-based company said yesterday in a statement. That’s higher than the average analyst estimate of $107.5 million, according to data compiled by Bloomberg. Fusion-io, whose biggest customers include Apple Inc. (AAPL) and Facebook Inc., is aiming to sell to small and medium-sized businesses with more affordable flash-memory-based services. The company said it purchased NexGen for about $119 million in cash and stock, and is already promoting NexGen’s hybrid storage product on its website. Fusion-io rose 20 percent to $20.03 at 9:46 a.m. in New York, and earlier touched $20.20 for the biggest intraday gain since August. The stock had dropped 27 percent this year through yesterday, compared with an 11 percent gain for the Standard & Poor’s 500 Index. While revenue in the fiscal third quarter fell 7 percent to $87.7 million, it topped the $80.4 million average analyst estimate. Fusion-io said its net loss widened to $20 million, or 21 cents a share, from $4.68 million, or 5 cents, a year earlier. Fusion-io, which employs Apple co-founder Steve Wozniak as chief scientist, sells corporate storage computers based on flash memory , which is faster and more expensive than widely used magnetic disk drives. NexGen, based in Louisville, Colorado , introduced its product in November 2011, a week after saying it raised $10 million from Next World Capital, Grotech Ventures and Access Venture Partners. “This would take years to build out as a stand-alone company,” NexGen Chief Executive Officer John Spiers wrote yesterday in a blog post. “Overnight, those concerns are eliminated.”


GT Advanced to Idle Plant, Expects Solar Recovery in 2014

GT Advanced Technologies Inc. (GTAT) , a U.S. supplier of manufacturing equipment for solar and energy- efficient lighting products, plans to sell a factory in St. Louis. The facility is developing high-efficiency solar technology that may not be needed until demand for photovoltaic panels picks up next year. The company will fire 35 people at the polysilicon plant, according to a statement today. Nashua, New Hampshire-based GT Advanced also said it will scale back its acquisition efforts this year. A global glut of solar production capacity drove down panel prices 61 percent in the past two years and slowed sales of GT Advanced’s systems. The company is transferring development of a high-efficiency polysilicon technology, HiCz, to a Merrimack, New Hampshire , plant from St. Louis. The systems will be ready this year and Chief Executive Officer Tom Gutierrez expects orders in 2014 when he anticipates solar demand to recover. “Given current market conditions in the PV industry, orders for the new generation systems may not occur until 2014 when the industry begins to recover,” the company said in the statement. Idling the plant will save about $15 million annually. The company will incur charges, which weren’t quantified, for the fourth quarter and in the current year for the restructuring. 2013 Forecast GT Advanced expects 2013 earnings of 25 cents to 45 cents a share, it said Dec. 18. That forecast includes the anticipated savings from the St. Louis plant, according to today’s statement. GT Advanced also expects to take a charge of $80 million to $90 million for the fourth quarter to write down the majority of its inventory of its existing line of polysilicon production systems, known as the DSS family. David Gray , chief strategy and new business officer, will leave the company Jan. 25 and his position is being eliminated as Gutierrez assumes responsibility for GT Advanced’s reduced acquisition efforts The new structure will “streamline our operations and align our business with current market and business conditions,” Gutierrez said in the statement. GT Advanced fell 0.9 percent to $3.41 at the close in New York.


Daimler to Add Renault-Nissan Fuso Vans to Global Cooperation

Daimler AG (DAI) and Renault SA (RNO) are expanding their global cooperation to include commercial vans made in Japan as the manufacturers continue to identify new projects together. Renault’s alliance partner Nissan Motor Corp. (7201) will supply Daimler with the NV350 Urvan to be sold under its Mitsubishi Fuso brand in selected export markets, the companies said today at the International Auto Show in Frankfurt. “We will continue to seek out new projects together that make sense for all partners,” Daimler Chief Executive Officer Dieter Zetsche said at a news conference. The manufacturers have gradually added projects to their three-year-old cooperation, which was originally focused on Stuttgart, Germany-based Daimler’s Smart city cars and Mercedes-Benz delivery vans. Last year, the companies agreed to jointly develop a four-cylinder gasoline engine and to license Mercedes transmissions for Nissan’s Infiniti luxury brand to be made in Mexico. “There’s no limit to the cooperation, there’s no taboo,” Carlos Ghosn , chief executive officer at both Renault and Nissan, said at the event. Tennessee Production The ramp up of a joint engine production in Decherd, Tennessee is on track for production start in mid-2014, Ghosn said. The engines will be used in the mid-sized Mercedes-Benz C-Class and new Nissan Infiniti cars. Mercedes and Infiniti are not competing for the same customers, said Ghosn. “Cross-shopping between the two brands is limited.” The two partners will also look into a possible cooperation in the large van segment even as talks haven’t started yet, Zetsche said. A Daimler contract to rebadge the Mercedes Sprinter for Volkswagen AG (VOW) will cease at the end of 2016, Daimler said last week. The decision to end production for Volkswagen was independent of any possible cooperation with Renault-Nissan, Zetsche said. Daimler and Renault, which is based in the Paris suburb of Boulogne-Billancourt, cooperate on Daimler’s smaller Citan delivery van, which the German company adapted from Renault’s Kangoo. The manufacturers are also considering teaming up on future compact car models. The review continues and no decision on a possible joint production in Mexico has been made, Ghosn said.


Detnor Gains After Raising Jette Oil Output Guidance: Oslo Mover

Det Norske Oljeselskap ASA (DETNOR) , an energy producer with stakes in Norway’s biggest find in 40 years, pared gains in Oslo trading after increasing production guidance for its Jette field in the North Sea by less than had been reported. The Trondheim-based company gained 1.9 percent to 92.5 kroner as of 1:07 p.m. local time, having earlier surged as much as 3.6 percent to 94 kroner, the highest intraday level in seven months. Detnor, which started output from Jette on May 21, expects gross initial production from the field’s two wells of about 12,000 to 15,000 barrels of oil a day, up from earlier guidance of about 10,000 barrels, it said in a statement. Shares jumped earlier after Offshore.no, an oil-industry website, said Jette is producing at about 21,000 barrels a day, citing an unidentified person. Output at that rate would have been double expectations, Alex Gheorghe, an analyst at RS Platou Markets AS, said in an e-mail. “The news is positive since the project was written off last year, and management has previously disclosed it’s unlikely it will recover costs,” Gheorghe wrote. “We expect the stock to outperform by about 1 to 2 kroner a share.” Detnor is transforming itself from an exploration company into an oil producer as pumping begins at fields including Jette and Ivar Aasen, scheduled to start in the fourth quarter of 2016. The company took a 1.9 billion kroner ($329 million) writedown on Jette in the third quarter after technical problems with a production well. “Jette is a small field and there are uncertainties with respect to future production rates,” Detnor said today. The company has a 70 percent stake in Jette, while Petoro AS, which manages Norway ’s direct stakes in its oil and gas assets, has the remaining 30 percent. Shares in Detnor have more than tripled in the last two years after finds including Johan Sverdrup, which could hold as much as 3.6 billion barrels of oil, making it the biggest discovery off Norway since 1974. Detnor has stakes in two of the three licenses containing Sverdrup.


Intel CEO Krzanich Plans Faster Shift to Mobile Chips

Intel Corp. (INTC) ’s Brian Krzanich, who took over today as chief executive officer, said the world’s largest computer chipmaker has the assets it needs to accelerate a push into mobile devices. “We see that we’ve been a bit slow to move into that space,” Krzanich said at the annual shareholder meeting at the company’s headquarters in Santa Clara , California. “The base of assets that we have will allow us to grow in that area much faster moving forwards.” Krzanich, Intel’s sixth chief executive, takes the helm as the company struggles to cut into Qualcomm Inc. (QCOM) ’s lead in the $85.4 billion mobile-chip market. Industry-leading manufacturing and new products will enable the company to start making inroads in phones and tablets, he said. Krzanich said he and newly promoted company President Renee James will try to make Intel more outward-looking in order to predict future trends in computing, Krzanich said. That’s more important as consumers, rather than corporations, take the leading role in determining which devices and technologies are successful, Chief Financial Officer Stacy Smith said in an interview following today’s meeting. “The lifecycle of those devices and the lifecycle of the leadership of the market is very short,” he said. “The consumer is driving these very fast technology shifts.” Krzanich, a former factory manager, succeeds Paul Otellini , who retired. Intel has improved its execution in creating products for new markets and Krzanich, with his background in operations, is very focused on making sure that there are no slip ups, said Smith. Intel shares rose less than 1 percent, to $24.23 at 2:05 p.m. in New York. While the stock has gained 18 percent this year, it dropped more than 6 percent during Otellini’s tenure, which began in May 2005.


KDDI to Sell $100 Million Gree Stake to Book Investment Profit

KDDI Corp. , Japan’s second-biggest wireless carrier, plans to sell a $100 million stake in mobile- game operator Gree Inc. to book a profit from its six-year investment. The carrier has asked Daiwa Securities Group Inc. to sell 8 million Gree shares between Feb. 13 and May 12, it said in a regulatory filing today. KDDI, the second-largest shareholder in Gree , will retain 8 million shares, or about 3 percent. “We can profit from the sale,” Keiichi Sakurai, a spokesman for Tokyo-based KDDI, said by phone today. “The sale will also let Gree do business more freely.” Gree has more than doubled in Tokyo trading since listing in 2008 on rising mobile phone usage and the popularity of games including “Monster Quest” and “Crime City”. This year, the company has fallen 14 percent after forecasting a decline in annual profit because of delays to new games and costs from expanding overseas. The game developer rose 2.7 percent to 1,154 yen in Tokyo trading today before the KDDI announcement. KDDI first invested in the company in 2006, according to Gree’s website. “There won’t be any change in our relationship,” Shinichi Iriyama , a Gree spokesman, said by phone. “We will continue building a good relationship with KDDI.”


Nissan CEO Wants at Least 20% Leaf Gain With Cheaper Car

Carlos Ghosn , who leads Nissan Motor Co. (7201) and Renault SA (RNO) , said he expects global deliveries of all- electric Nissan Leafs to expand at least 20 percent this year aided by the addition of a lower-priced version. The company today said the new S trim version of its rechargeable hatchback will have a $28,800 base price when it goes on sale this year. Prices for its current SV and SL grade cars for 2013 will be $31,820 and $34,840, respectively, the company said at the North American International Auto Show in Detroit. Nissan, Japan’s second-largest carmaker, sold 9,819 Leafs in the U.S. last year, less than half of Ghosn’s 20,000 goal. Ghosn cited limited charge infrastructure as one reason for the result and declined to set a specific sales target for the lithium-ion powered hatchback “You can expect them to be up,” Ghosn told reporters today in Detroit. “It would be fair to say” that Leaf should post at least a 20 increase, he said. Base prices for the 2012 SV and SL Leafs are $35,200 and $37,250, respectively, before a $7,500 federal tax credit, according to Nissan’s website. He said the company’s bullish early targets for the car reflected “passion” for the product rather than “arrogance.” Production of the car and its battery packs is under way in Smyrna, Tennessee, the company said this month. Nissan is coming off record U.S. sales in 2012, aided by deliveries of Altima sedans and Rogue crossovers. The Yokohama, Japan-based company boosted deliveries 9.5 percent to 1.14 million Nissan and Infiniti brand vehicles. Nissan’s U.S. headquarters are in Franklin, Tennessee.


Nicolas Cage Hired by China’s BAIC Motor to Promote Senova Cars

Actor Nicolas Cage has signed on as brand ambassador for Chinese automaker BAIC Motor Corp., joining Brad Pitt and Barcelona soccer star Lionel Messi in promoting automobiles in the world’s largest vehicle market. Cage will appear in a television commercial to be aired in May, according to Beijing-based BAIC Motor. The state-owned automaker said it chose the 49-year-old Academy Award winner as its brand spokesman because he’s well-known to Chinese audiences, especially target buyers in their 40s. “Cage will be our ambassador for the Senova D series that will hit the market soon,” Alex Qu, a spokesman for the company, said by phone today. Qu declined to provide more details on the events that Cage will attend in China. BAIC, whose parent traces its roots to making sidecar motorcycles used in the Korean War , is stepping up publicity as it prepares to roll out its Senova brand in the second quarter. The automaker is also targeting sales to senior government officials, with Beijing municipal Party Secretary Guo Jinlong and Mayor Wang Anshun expressing interesting in replacing their Audi sedans, BAIC’s overseas business head Dong Haiyang said. Cage, currently the voice of a caveman in the animated film “The Croods,” won an Oscar for his performance in “Leaving Las Vegas ” and has also starred in movies such as “Drive Angry.” General Motors Co. (GM) signed up Pitt in January to promote its China-made Cadillac XTS, while Chery Automobile Co. hired Barcelona soccer player Messi in 2010 to pitch their Riich brand cars. Zhejiang Geely Holding Group Co.’s Volvo Cars entered an endorsement deal with Houston Rockets player Jeremy Lin last year. BAIC Motor’s parent has said it is seeking an initial share sale for the car unit in Hong Kong and mainland China. To contact Bloomberg News staff for this story: Tian Ying in Beijing at  ytian@bloomberg.net 


Magnum Rises After Eagle Ford Asset Sale: Dallas Mover

Magnum Hunter Resources Corp. (MHR) , the Houston-based oil and natural gas producer with operations in West Virginia , North Dakota and Saskatchewan, advanced the most in more than four months after it agreed to sell Texas assets for double the amount invested in them. Magnum Hunter rose 5.6 percent to $4.02 at 11:10 a.m. in New York after earlier gaining 12 percent, the most intraday since Nov. 19. The shares have fallen 38 percent in the last year. Magnum Hunter sold mineral leases on 19,000 net acres in the Eagle Ford Shale for $401 million in cash, including production equivalent to about 3,000 barrels of oil a day to Penn Virgina Corp., according to statements today by the companies. The deal is expected to close in mid May. Penn Virginia , based in Radnor, Pennsylvania, fell 0.3 percent today. The company also announced it will sell $400 million of senior notes due 2020 to finance the purchase, which will increase its Eagle Ford holdings to about 54,000 net acres. The sale price was double the capital invested in the leases during the past three years, Magnum Hunter Chairman and Chief Executive Officer Gary C. Evans said in the company’s statement today. The company is keeping 7,000 net acres in the Eagle Ford deposit.


ARM Sees Shift to Cheaper Smartphones Boosting Chip Royalty

ARM Holdings Plc (ARM) , the U.K. chip designer whose products power Apple Inc. (AAPL) ’s iPhones, predicts a shift in demand toward lower-priced smartphones will help boost royalty revenue. “There is a shift advantage for ARM as mid-range and entry smartphones drive smartphone growth. We’ve got that dynamic going into next year,” Chief Financial Officer Tim Score said at a conference in Barcelona yesterday. “Those phones generate much more royalty for ARM because they use more silicon.” ARM has had five consecutive years of revenue growth as its designs for chips that control graphics and processing were used in increasingly popular smartphones and tablets. Royalty payments took the spotlight last quarter as they missed analysts’ estimates. ARM shares have fallen almost 9 percent since the earnings report on Oct. 22. “As ARM technology becomes more sophisticated, the royalty percentage goes up,” Score said. “On average, it’s going to very gradually go up.” Global smartphone shipments surged 40 percent last quarter, slowing from 53 percent growth for the previous three-month period, according to researcher IDC. The average selling prices of the devices fell by 13 percent to $317 from a year earlier, because of demand for more affordable handsets. In spite of concerns about slowing growth, especially in China, Score said ARM can continue expanding its customer base and outperforming the semiconductor market in the next five years, driven by cheaper handsets, enterprise networking and micro-controllers. ARM shares fell less than 1 percent to 953.50 pence at 9:16 a.m. in London. New Chief Chief Executive Officer Simon Segars, who succeeded Warren East in July, is confronting a shift in the industry’s growth to low-end smartphones and new entrant Intel Corp. (INTC) , which has begun designing its own chips for mobile phones. ARM, based in Cambridge, England, is also seeking to boost its market share in servers as companies from Facebook Inc. (FB) to Google Inc. (GOOG) look for lower-power machines to drive their data centers. It’s “possible to identify such a market and build a chip specifically for that market, whether it is to handle only storage or only video loading, for example. We can make the chip much more efficient,” Score said at the annual Technology, Media and Telecoms Conference organized by Morgan Stanley. “Everyone knows Intel has a very strong ecosystem around servers, we’re not kidding ourselves,” Score said. “For that reason we see it as further out.”


Fusion-io Surges on Sales Forecast and Acquisition of NexGen

Fusion-io Inc. (FIO) , a maker of data- storage computers, climbed the most in eight months after forecasting sales that topped analysts’ estimates and announcing the acquisition of NexGen Storage Inc. Revenue in the fourth quarter, which ends in June, will be about $110 million, the Salt Lake City-based company said yesterday in a statement. That’s higher than the average analyst estimate of $107.5 million, according to data compiled by Bloomberg. Fusion-io, whose biggest customers include Apple Inc. (AAPL) and Facebook Inc., is aiming to sell to small and medium-sized businesses with more affordable flash-memory-based services. The company said it purchased NexGen for about $119 million in cash and stock, and is already promoting NexGen’s hybrid storage product on its website. Fusion-io rose 20 percent to $20.03 at 9:46 a.m. in New York, and earlier touched $20.20 for the biggest intraday gain since August. The stock had dropped 27 percent this year through yesterday, compared with an 11 percent gain for the Standard & Poor’s 500 Index. While revenue in the fiscal third quarter fell 7 percent to $87.7 million, it topped the $80.4 million average analyst estimate. Fusion-io said its net loss widened to $20 million, or 21 cents a share, from $4.68 million, or 5 cents, a year earlier. Fusion-io, which employs Apple co-founder Steve Wozniak as chief scientist, sells corporate storage computers based on flash memory , which is faster and more expensive than widely used magnetic disk drives. NexGen, based in Louisville, Colorado , introduced its product in November 2011, a week after saying it raised $10 million from Next World Capital, Grotech Ventures and Access Venture Partners. “This would take years to build out as a stand-alone company,” NexGen Chief Executive Officer John Spiers wrote yesterday in a blog post. “Overnight, those concerns are eliminated.”


GT Advanced to Idle Plant, Expects Solar Recovery in 2014

GT Advanced Technologies Inc. (GTAT) , a U.S. supplier of manufacturing equipment for solar and energy- efficient lighting products, plans to sell a factory in St. Louis. The facility is developing high-efficiency solar technology that may not be needed until demand for photovoltaic panels picks up next year. The company will fire 35 people at the polysilicon plant, according to a statement today. Nashua, New Hampshire-based GT Advanced also said it will scale back its acquisition efforts this year. A global glut of solar production capacity drove down panel prices 61 percent in the past two years and slowed sales of GT Advanced’s systems. The company is transferring development of a high-efficiency polysilicon technology, HiCz, to a Merrimack, New Hampshire , plant from St. Louis. The systems will be ready this year and Chief Executive Officer Tom Gutierrez expects orders in 2014 when he anticipates solar demand to recover. “Given current market conditions in the PV industry, orders for the new generation systems may not occur until 2014 when the industry begins to recover,” the company said in the statement. Idling the plant will save about $15 million annually. The company will incur charges, which weren’t quantified, for the fourth quarter and in the current year for the restructuring. 2013 Forecast GT Advanced expects 2013 earnings of 25 cents to 45 cents a share, it said Dec. 18. That forecast includes the anticipated savings from the St. Louis plant, according to today’s statement. GT Advanced also expects to take a charge of $80 million to $90 million for the fourth quarter to write down the majority of its inventory of its existing line of polysilicon production systems, known as the DSS family. David Gray , chief strategy and new business officer, will leave the company Jan. 25 and his position is being eliminated as Gutierrez assumes responsibility for GT Advanced’s reduced acquisition efforts The new structure will “streamline our operations and align our business with current market and business conditions,” Gutierrez said in the statement. GT Advanced fell 0.9 percent to $3.41 at the close in New York.


Agthia Jumps Most on Record as Quarterly Profit Climbs 38%

Agthia Group PJSC (AGTHIA) jumped the most on record after the Abu Dhabi-based company reported a 38 percent increase in first-quarter profit. The shares soared 11 percent, the most since they were listed in July 2005, to 2.9 dirhams at the close in the emirate. About 1.8 million shares were traded, more than seven times the three-month daily average, according to data compiled by Bloomberg. The benchmark ADX General Index rose 1 percent. Net income rose to 37 million dirhams ($10.1 million), the company that creates, invests, trades and operates companies that are involved in the food and beverage business said in a statement yesterday. “We remain optimistic and expect another successful growth year,” the company said, commenting on the outlook for 2013. Today’s jump pushed Agthia’s gains this year to 33 percent, compared with a 25 percent rise for the benchmark index. The stock’s 14-day relative strength index rose to 73 today. A reading above 70 indicates to some analysts that a security or index is poised to decline. Three analysts recommend investors buy the stock and four advise holding the shares, data compiled by Bloomberg show.


Intel CEO Krzanich Plans Faster Shift to Mobile Chips

Intel Corp. (INTC) ’s Brian Krzanich, who took over today as chief executive officer, said the world’s largest computer chipmaker has the assets it needs to accelerate a push into mobile devices. “We see that we’ve been a bit slow to move into that space,” Krzanich said at the annual shareholder meeting at the company’s headquarters in Santa Clara , California. “The base of assets that we have will allow us to grow in that area much faster moving forwards.” Krzanich, Intel’s sixth chief executive, takes the helm as the company struggles to cut into Qualcomm Inc. (QCOM) ’s lead in the $85.4 billion mobile-chip market. Industry-leading manufacturing and new products will enable the company to start making inroads in phones and tablets, he said. Krzanich said he and newly promoted company President Renee James will try to make Intel more outward-looking in order to predict future trends in computing, Krzanich said. That’s more important as consumers, rather than corporations, take the leading role in determining which devices and technologies are successful, Chief Financial Officer Stacy Smith said in an interview following today’s meeting. “The lifecycle of those devices and the lifecycle of the leadership of the market is very short,” he said. “The consumer is driving these very fast technology shifts.” Krzanich, a former factory manager, succeeds Paul Otellini , who retired. Intel has improved its execution in creating products for new markets and Krzanich, with his background in operations, is very focused on making sure that there are no slip ups, said Smith. Intel shares rose less than 1 percent, to $24.23 at 2:05 p.m. in New York. While the stock has gained 18 percent this year, it dropped more than 6 percent during Otellini’s tenure, which began in May 2005.


Western Digital Hits 15-Year Peak on Talk of Buyout

Western Digital Corp., the maker of hard drives and home-entertainment devices, reached its highest in more than 15 years on speculation the company may go private following the potential buyout of Dell Inc. (DELL)  Western Digital, based in Irvine, California, advanced 5.7 percent to $47.36 at the close in New York, the biggest gain since Dec. 5 and the highest closing price since September 1997. Dell, the Round Rock, Texas-based personal computer maker, is in talks to go private, Bloomberg News reported last week, citing people with knowledge of the situation. Funding from Microsoft Corp. (MSFT) may help clinch the buyout by Silver Lake Management LLC, people said today. If that deal moves forward, Western Digital would also be a takeout candidate, said Mark Miller , an analyst at Noble Financial Capital Markets. “Western Digital could follow,” Miller, a former research manager at Western Digital, said in a telephone interview. Steve Shattuck, a spokesman for Western Digital, declined to comment. As a private company, Western Digital wouldn’t have to respond to public shareholder demands and could increase its cash flow, Miller said. “Its cash flows could be $4 billion to $5 billion a year in a few years,” Miller said. The company generated $3.65 billion in cash from operations in the past four quarters, according to data compiled by Bloomberg. Investor sentiment on Western Digital has also turned positive after Seagate Technology Plc (STX) recently raised its sales forecast, Miller said. Seagate, based in Dublin, Ireland, rose 7.1 percent to $37.89, its highest close since December 2002. “Things are not as bad as people thought,” he said.


Apple Is Said to Debut New IPads at Oct. 22 Event

Apple Inc. (AAPL) will update its lineup of iPads at an Oct. 22 event, a person with knowledge of the plans said, as the company works to remain ahead of rivals in the increasingly crowded tablet market. The Cupertino, California-based company will unveil a new version of the original 9.7-inch iPad as well as the smaller iPad mini, said the person, who asked not to be identified because the plans haven’t been made official. Apple also will show its new Mac personal-computer operating system, called Mavericks, and Mac Pro PC, the person said. The event, scheduled for a month after Apple debuted two new iPhone models, will feature changes to the bigger iPad that include a thinner design, people familiar with the plans said in August. A new iPad mini will sport the high-definition screen now on the iPhone and bigger iPads, the people said. The event was reported earlier by AllThingsD. Trudy Muller , a spokeswoman for Apple, declined to comment. In the absence of a new iPad, the tablet market shrank in the second quarter, suggesting pent-up demand for the new models. Global tablet unit sales fell 9.7 percent to 45.1 million, with iPad shipments falling to 14.6 million units from 19.5 million in the first quarter, according to researcher IDC. While Apple remains the market leader for tablets, which meld the functions of touch-screen smartphones and laptops, its share has been threatened as competitors such as Amazon.com Inc., Samsung Electronics Co. (005930) and Microsoft Corp. introduce other models. Apple shares fell 1.4 percent to $480.94 at yesterday’s close in New York. The stock has fallen 9.6 percent this year.


Magnum Rises After Eagle Ford Asset Sale: Dallas Mover

Magnum Hunter Resources Corp. (MHR) , the Houston-based oil and natural gas producer with operations in West Virginia , North Dakota and Saskatchewan, advanced the most in more than four months after it agreed to sell Texas assets for double the amount invested in them. Magnum Hunter rose 5.6 percent to $4.02 at 11:10 a.m. in New York after earlier gaining 12 percent, the most intraday since Nov. 19. The shares have fallen 38 percent in the last year. Magnum Hunter sold mineral leases on 19,000 net acres in the Eagle Ford Shale for $401 million in cash, including production equivalent to about 3,000 barrels of oil a day to Penn Virgina Corp., according to statements today by the companies. The deal is expected to close in mid May. Penn Virginia , based in Radnor, Pennsylvania, fell 0.3 percent today. The company also announced it will sell $400 million of senior notes due 2020 to finance the purchase, which will increase its Eagle Ford holdings to about 54,000 net acres. The sale price was double the capital invested in the leases during the past three years, Magnum Hunter Chairman and Chief Executive Officer Gary C. Evans said in the company’s statement today. The company is keeping 7,000 net acres in the Eagle Ford deposit.


Newmont Takes $1.77 Billion Writedown After Gold Falls

(Corrects writedown in headline and first paragraph.) Newmont Mining Corp. (NEM) , the second-largest gold producer, reported a surprise loss after taking a $1.77 billion writedown on the value of stockpiles and two Australian mines following a slump in bullion prices. The second-quarter net loss was $2.02 billion, or $4.06 a share, compared with net income of $279 million, or 56 cents, a year earlier, the Greenwood Village , Colorado-based miner said today in a statement. The loss, excluding impairments and other one-time items, was 10 cents a share, compared with the average of 17 estimates compiled by Bloomberg for a profit of 42 cents. Gold mining companies have announced at least $15 billion of writedowns in the past two months after the precious metal’s steepest quarterly drop in more than nine decades. Newcrest Mining Ltd. (NCM) said June 7 it may take a charge of as much as A$6 billion ($5.6 billion) and Goldcorp Inc. (G) , the biggest gold producer by market value, said yesterday it took a writedown on the value of assets of $1.96 billion. The second-quarter results were released after the close of regular trading in New York , where Newmont rose 1.1 percent to $29.93. Newmont said it was taking impairment charges on its Boddington and Tanami mines in Australia , and writing down the value of stockpiles and ore on leach pads. The producer said it reduced spending by $362 million this year, from the first half of 2012, to lower costs. “We are also on track to reduce our corporate work force by more than one third,” Chief Executive Officer Gary Goldberg said in the statement. Job Cuts About 80 jobs, including some vacant positions, from a workforce of 1,870 will be cut by the end of September at the Boddington mine in Western Australia , the company said today in a separate statement. Newmont said second-quarter sales fell to $1.99 billion from $2.23 billion. The average of seven estimates was for $2.09 billion. Gold futures in New York rose 0.7 percent to settle at $1,329.50 an ounce yesterday. The metal has dropped 31 percent from a record $1,923.70 in September 2011. (Newmont scheduled a conference call for 10 a.m. New York time today. U.S. callers should dial 1-888-566-1822, international callers 1-312-470-7116. Pass code: Newmont.)


BMC Software Drops After Profit Forecast Misses: Houston Mover

BMC Software Inc. (BMC) , a maker of programs that help companies update servers and personal computers, fell after forecasting fiscal 2013 profit that missed analysts’ estimates, a sign it’s struggling to clinch big contracts. The shares declined 6.2 percent to $41.71 at the close in New York , the biggest decline since October 2011. The Houston- based company said yesterday that profit excluding some items will be $3.35 to $3.45 a share for fiscal 2013, which ends in March. That’s less than BMC’s prior forecast of $3.49 to $3.59 and trails analysts’ average $3.56 projection, according to data compiled by Bloomberg. Activist shareholder Elliott Associates LP pressed BMC to consider a sale last year, unsettling customers and making it harder for the company to clinch deals, BMC Chief Executive Officer Bob Beauchamp told analysts on a conference call yesterday. The company had two large contract renewals pushed to the March quarter from the December period, crimping results. “BMC underperformed across all metrics for the quarter,” and deals in the mainframe and server-software businesses didn’t get closed, Abhey Lamba, an analyst at Mizuho Securities USA Inc., wrote in a research report. “Despite having completed its accelerated share repurchase program, the company lowered its EPS target for the year, which is clearly disappointing.” ‘Growth Areas’ The company said it needs to be “more consistent and disciplined” in the way it secures big contracts, according to the statement. Management is “scrutinizing the entire company to improve our operational discipline,” Beauchamp said. BMC added board members and initiated a $1 billion share buyback to assuage Elliott’s complaints that it wasn’t doing enough to maximize shareholder value. Competitors are “aggressively” capitalizing on the pressure by Elliott, prompting many customers to seek personal visits from BMC management before completing purchases, Beauchamp said. Beauchamp said during a conference call with analysts that he’s undertaking an operational review aimed at directing spending toward better performing products to increase earnings, cash flow and operating margin amid low revenue growth. The company is trying to improve profitability even as revenue this year is projected to grow in the low single digits. Web-delivered software, cloud-computing management products and tools for managing mobile devices are selling well and will get a greater share of investment compared with underperforming products, he said. “We’re going to find a way to move funds to these growth areas,” Beauchamp said. “It’s just moving faster.” BMC kicked off the review after the end of the quarter on Dec. 31, according to Mark Stouse, a spokesman.


Intel CEO Krzanich Plans Faster Shift to Mobile Chips

Intel Corp. (INTC) ’s Brian Krzanich, who took over today as chief executive officer, said the world’s largest computer chipmaker has the assets it needs to accelerate a push into mobile devices. “We see that we’ve been a bit slow to move into that space,” Krzanich said at the annual shareholder meeting at the company’s headquarters in Santa Clara , California. “The base of assets that we have will allow us to grow in that area much faster moving forwards.” Krzanich, Intel’s sixth chief executive, takes the helm as the company struggles to cut into Qualcomm Inc. (QCOM) ’s lead in the $85.4 billion mobile-chip market. Industry-leading manufacturing and new products will enable the company to start making inroads in phones and tablets, he said. Krzanich said he and newly promoted company President Renee James will try to make Intel more outward-looking in order to predict future trends in computing, Krzanich said. That’s more important as consumers, rather than corporations, take the leading role in determining which devices and technologies are successful, Chief Financial Officer Stacy Smith said in an interview following today’s meeting. “The lifecycle of those devices and the lifecycle of the leadership of the market is very short,” he said. “The consumer is driving these very fast technology shifts.” Krzanich, a former factory manager, succeeds Paul Otellini , who retired. Intel has improved its execution in creating products for new markets and Krzanich, with his background in operations, is very focused on making sure that there are no slip ups, said Smith. Intel shares rose less than 1 percent, to $24.23 at 2:05 p.m. in New York. While the stock has gained 18 percent this year, it dropped more than 6 percent during Otellini’s tenure, which began in May 2005.


ARM Sees Shift to Cheaper Smartphones Boosting Chip Royalty

ARM Holdings Plc (ARM) , the U.K. chip designer whose products power Apple Inc. (AAPL) ’s iPhones, predicts a shift in demand toward lower-priced smartphones will help boost royalty revenue. “There is a shift advantage for ARM as mid-range and entry smartphones drive smartphone growth. We’ve got that dynamic going into next year,” Chief Financial Officer Tim Score said at a conference in Barcelona yesterday. “Those phones generate much more royalty for ARM because they use more silicon.” ARM has had five consecutive years of revenue growth as its designs for chips that control graphics and processing were used in increasingly popular smartphones and tablets. Royalty payments took the spotlight last quarter as they missed analysts’ estimates. ARM shares have fallen almost 9 percent since the earnings report on Oct. 22. “As ARM technology becomes more sophisticated, the royalty percentage goes up,” Score said. “On average, it’s going to very gradually go up.” Global smartphone shipments surged 40 percent last quarter, slowing from 53 percent growth for the previous three-month period, according to researcher IDC. The average selling prices of the devices fell by 13 percent to $317 from a year earlier, because of demand for more affordable handsets. In spite of concerns about slowing growth, especially in China, Score said ARM can continue expanding its customer base and outperforming the semiconductor market in the next five years, driven by cheaper handsets, enterprise networking and micro-controllers. ARM shares fell less than 1 percent to 953.50 pence at 9:16 a.m. in London. New Chief Chief Executive Officer Simon Segars, who succeeded Warren East in July, is confronting a shift in the industry’s growth to low-end smartphones and new entrant Intel Corp. (INTC) , which has begun designing its own chips for mobile phones. ARM, based in Cambridge, England, is also seeking to boost its market share in servers as companies from Facebook Inc. (FB) to Google Inc. (GOOG) look for lower-power machines to drive their data centers. It’s “possible to identify such a market and build a chip specifically for that market, whether it is to handle only storage or only video loading, for example. We can make the chip much more efficient,” Score said at the annual Technology, Media and Telecoms Conference organized by Morgan Stanley. “Everyone knows Intel has a very strong ecosystem around servers, we’re not kidding ourselves,” Score said. “For that reason we see it as further out.”


Fusion-io Surges on Sales Forecast and Acquisition of NexGen

Fusion-io Inc. (FIO) , a maker of data- storage computers, climbed the most in eight months after forecasting sales that topped analysts’ estimates and announcing the acquisition of NexGen Storage Inc. Revenue in the fourth quarter, which ends in June, will be about $110 million, the Salt Lake City-based company said yesterday in a statement. That’s higher than the average analyst estimate of $107.5 million, according to data compiled by Bloomberg. Fusion-io, whose biggest customers include Apple Inc. (AAPL) and Facebook Inc., is aiming to sell to small and medium-sized businesses with more affordable flash-memory-based services. The company said it purchased NexGen for about $119 million in cash and stock, and is already promoting NexGen’s hybrid storage product on its website. Fusion-io rose 20 percent to $20.03 at 9:46 a.m. in New York, and earlier touched $20.20 for the biggest intraday gain since August. The stock had dropped 27 percent this year through yesterday, compared with an 11 percent gain for the Standard & Poor’s 500 Index. While revenue in the fiscal third quarter fell 7 percent to $87.7 million, it topped the $80.4 million average analyst estimate. Fusion-io said its net loss widened to $20 million, or 21 cents a share, from $4.68 million, or 5 cents, a year earlier. Fusion-io, which employs Apple co-founder Steve Wozniak as chief scientist, sells corporate storage computers based on flash memory , which is faster and more expensive than widely used magnetic disk drives. NexGen, based in Louisville, Colorado , introduced its product in November 2011, a week after saying it raised $10 million from Next World Capital, Grotech Ventures and Access Venture Partners. “This would take years to build out as a stand-alone company,” NexGen Chief Executive Officer John Spiers wrote yesterday in a blog post. “Overnight, those concerns are eliminated.”


Western Digital Hits 15-Year Peak on Talk of Buyout

Western Digital Corp., the maker of hard drives and home-entertainment devices, reached its highest in more than 15 years on speculation the company may go private following the potential buyout of Dell Inc. (DELL)  Western Digital, based in Irvine, California, advanced 5.7 percent to $47.36 at the close in New York, the biggest gain since Dec. 5 and the highest closing price since September 1997. Dell, the Round Rock, Texas-based personal computer maker, is in talks to go private, Bloomberg News reported last week, citing people with knowledge of the situation. Funding from Microsoft Corp. (MSFT) may help clinch the buyout by Silver Lake Management LLC, people said today. If that deal moves forward, Western Digital would also be a takeout candidate, said Mark Miller , an analyst at Noble Financial Capital Markets. “Western Digital could follow,” Miller, a former research manager at Western Digital, said in a telephone interview. Steve Shattuck, a spokesman for Western Digital, declined to comment. As a private company, Western Digital wouldn’t have to respond to public shareholder demands and could increase its cash flow, Miller said. “Its cash flows could be $4 billion to $5 billion a year in a few years,” Miller said. The company generated $3.65 billion in cash from operations in the past four quarters, according to data compiled by Bloomberg. Investor sentiment on Western Digital has also turned positive after Seagate Technology Plc (STX) recently raised its sales forecast, Miller said. Seagate, based in Dublin, Ireland, rose 7.1 percent to $37.89, its highest close since December 2002. “Things are not as bad as people thought,” he said.


NSA Center for Spy Data Suffers Electrical Failures

A $1.2 billion data center being built in Utah for the National Security Agency to house U.S. intelligence secrets has been plagued by electrical failures, according to an agency official. The electrical failures at the facility located in the suburbs of Salt Lake City have been mitigated and the center is completing acceptance testing, the official, who asked not to be identified, said in an e-mail. The problems were reported earlier by the Wall Street Journal. The center -- one of the Defense Department’s biggest construction projects in the U.S. -- is intended to be “a state-of-the-art facility designed to support the intelligence community’s efforts to further strengthen and protect the nation’s cyber security,” according to a January 2011 NSA statement. “During the testing and commissioning of the Utah Data Center, problems were discovered with certain parts of the electrical system,” according to a statement by the Army Corps of Engineers passed on by the NSA last night. “Issues such as these can arise in any project, and are the reason the Corps tests and reviews every aspect of any project prior to releasing it to the customer.” The NSA’s spying programs include storing the phone records of millions of Americans as well as the e-mail and Internet activity of suspected foreign terrorists who may communicate with U.S. citizens, according to documents exposed in June by former government contractor Edward Snowden. 10 Meltdowns The causes of the center’s problems, which include 10 electrical meltdowns in the past 13 months, have destroyed hundreds of thousands of dollars worth of machinery and delayed the its opening by a year, according to the Wall Street Journal. The Utah facility sits on about 247 acres and includes 1.2-milion-square-feet of enclosed space and will host the power, space, cooling and communications needed for specialized computing, said the NSA official, who requested anonymity because of not being authorized to publicly discuss the project’s problems. “In an era when our nation and its allies are increasingly dependent on the integrity of information and systems supported, transmitted, or stored in cyberspace, it is essential that that space is as resilient and secure as possible,” John Inglis, NSA deputy director, said in the January 2011 statement on the need for the facility.


Samsung Boosting Venture-Capital Funding to $1.1 Billion

Samsung Electronics Co. (005930) , the world’s largest maker of mobile phones, said it was boosting venture- capital funding by $100 million to help create new businesses for its unit making memory chips and electronics parts. The company unveiled the Catalyst Fund for early stage startups, as well as a strategy and innovation center in Menlo Park , California , at an event near the site on Sand Hill Road. The company and affiliates also run the $1 billion Samsung Ventures America Fund. “We must reach out to global hot spots and global talent,” Young Sohn, president and chief strategy officer of device solutions for Samsung, said at the event. “This is the epicenter of disruptive forces.” Like other companies such as Intel Corp. (INTC) , Google Inc. and Microsoft Corp., Samsung is trying to use its cash to encourage the development of technology that will benefit its business in the future. Traditional venture-capital investors aren’t investing in basic science and technology startups, creating a gap that Samsung intends to help fill, Sohn said. Samsung’s venture arm, now headquartered in Seoul, has more than $1 billion in assets under management and did 20 deals worth $160 million last year, Sohn said. Investments by the new funds will be in the areas of Internet-connected devices, mobile health, computer-human interface, cloud-data centers and mobile privacy, he said. Shares of the Suwon, South Korea-based company fell 0.1 percent to 1,435,000 won in South Korean trading today.


VirnetX Falls as Texas Jury Struggles in Cisco Patent Case

VirnetX Holding Corp. (VHC) fell as much as 27 percent after a federal jury in Tyler, Texas, said it was struggling to reach a unanimous verdict in its patent- infringement case against Cisco Systems Inc. (CSCO)  After the jury told the judge it was deadlocked, U.S. District Judge Leonard Davis told jurors to continue deliberating. VirnetX was down $5.83, or 16 percent, to $29.84 in New York trading at 3:07 p.m. Earlier, it tumbled as low as $26.02 for its biggest drop since August. VirnetX contends Cisco infringes four patents for virtual private networks, through which a website owner can securely interact with a customer or an employee can work at home and have protected access to a company’s electronic files. It’s seeking $258 million from San Jose, California-based Cisco. Cisco, the world’s biggest maker of computer-networking equipment, denies infringing the patents and contends they are invalid. Cisco rose less than 1 cent to $21.58. VirnetX won a $368.2 million verdict against Apple Inc. (AAPL) in November over the same technology, including two of the same patents, before a different Tyler jury. In 2010, Zephyr Cove, Nevada-based VirnetX reached a $200 million settlement with Microsoft Corp. over the same inventions. The case is VirnetX Inc. v. Cisco Systems Inc., 10cv417, U.S. District Court for the Eastern District of Texas (Tyler).


Oxford Nanopore’s Gene Sequencer Facing Technical Delays

Oxford Nanopore Technologies Ltd., the company in a race with Illumina Inc. (ILMN) and Life Technologies Corp. (LIFE) to develop smaller, cheaper and quicker gene-sequencing devices, is facing technical setbacks. Oxford Nanopore, based in Oxford, England , has had to redesign components of a custom sensor because it wasn’t performing satisfactorily, which has lengthened development and testing periods, according to a post yesterday on the blog Pathogens: Genes and Genomes. The blog is found on the University of Birmingham’s website and written by Nick Loman, a doctor who works as a bioinformatics specialist at the English university. “That put us back about five months, but it was the right thing to do,” Loman quoted Oxford Nanopore Chief Technology Officer Clive Brown as saying at the Advances in Genome Biology and Technology conference last month. The closely held company said more than a year ago that it expected to begin selling its products by the end of 2012. Yesterday, it didn’t deny Loman’s account of his conversation with Brown when contacted by Bloomberg News. “We should have made it clear that was off the record,” Chief Executive Officer Gordon Sanghera said of Loman’s report. “We aren’t giving any further comment on it.” The company has also had to address chemistry in its devices degrading and has been striving to reduce the error rate to 1 percent from 4 percent, according to Loman’s account of his conversation with Brown, which took place during the Feb. 20-23 AGBT meeting in Marco Island, Florida. Technical Risks IP Group Plc (IPO) owns about 20 percent of Oxford Nanopore. Competitor Illumina owns about 15 percent, and the company’s other shareholders include hedge-fund manager Lansdowne Partners LP and Invesco Perpetual, the U.K. group of mutual funds. Numis Securities’ London-based analyst Charles Weston reduced his estimate of Oxford Nanopore’s value by 25 percent to $1.5 billion from $2 billion in a Feb. 20 note to clients, citing technical-development risks. Loman wrote that Brown told him the device wouldn’t be “a year, two years late” and that the company would release data from its early-access program for its products this year. Oxford Nanopore was spun out of Oxford University in 2005 and its technology was initially based on research by Hagan Bayley, a professor of chemical biology at the university. Its instruments use a novel technique known as strand sequencing, in which an entire string of DNA is guided by an enzyme and passes intact through a tiny hole in a cell membrane one-billionth of a meter wide.


Nissan Opens Silicon Valley Center for Self-Driving Car Research

Nissan Motor Co. opened a research center in California’s Silicon Valley that the Japanese carmaker plans to make its hub for research on self-driving vehicles and Internet-connected auto technology. The facility will be staffed by more than 60 engineers and technicians within three years, Carla Bailo , Nissan’s senior vice president for North American research and development, said in a phone interview. Work on so-called autonomous vehicle systems will move from an R&D center in Japan to Sunnyvale, she said, declining to provide investment details for the project. “We’re going to focus on this technology really in the heart of where it lies today,” Bailo said in the Feb. 15 interview. “We are going to be shifting work that’s being done in our Nissan Advanced Technical Center in Atsugi, and moving that work over to the heart of the industry.” Automakers and companies including Google Inc. plan new electronics and programming for vehicles with a goal of reducing collisions, fuel use and improving traffic flow. Carlos Ghosn , Nissan’s chief executive officer, told reporters last month in Detroit that regulatory, legal and technological hurdles mean self-driving cars won’t be feasible until about 2020. Google has tested a self-driving Prius in California for years, and the U.S. Defense Department has sponsored autonomous- vehicle research for more than a decade. Nissan’s Sunnyvale center will work with automotive labs at Stanford University, the University of California’s Berkeley and Davis campuses, and Silicon Valley companies, Bailo said. The Yokohama-based carmaker plans to apply technology developed at the new R&D center within a decade, she said. Maarten Sierhuis , a former NASA scientist specializing in artificial intelligence research, will lead the Sunnyvale facility, Nissan said today. Toyota Motor Corp. opened its own Silicon Valley outpost in Mountain View in 2012 to work with technology companies there on in-car data and entertainment systems. The company said in January that it’s developing autonomous safety systems to create a virtual “co-pilot” in vehicles to help drivers avoid accidents rather than self-driving cars and trucks.


GT Advanced to Idle Plant, Expects Solar Recovery in 2014

GT Advanced Technologies Inc. (GTAT) , a U.S. supplier of manufacturing equipment for solar and energy- efficient lighting products, plans to sell a factory in St. Louis. The facility is developing high-efficiency solar technology that may not be needed until demand for photovoltaic panels picks up next year. The company will fire 35 people at the polysilicon plant, according to a statement today. Nashua, New Hampshire-based GT Advanced also said it will scale back its acquisition efforts this year. A global glut of solar production capacity drove down panel prices 61 percent in the past two years and slowed sales of GT Advanced’s systems. The company is transferring development of a high-efficiency polysilicon technology, HiCz, to a Merrimack, New Hampshire , plant from St. Louis. The systems will be ready this year and Chief Executive Officer Tom Gutierrez expects orders in 2014 when he anticipates solar demand to recover. “Given current market conditions in the PV industry, orders for the new generation systems may not occur until 2014 when the industry begins to recover,” the company said in the statement. Idling the plant will save about $15 million annually. The company will incur charges, which weren’t quantified, for the fourth quarter and in the current year for the restructuring. 2013 Forecast GT Advanced expects 2013 earnings of 25 cents to 45 cents a share, it said Dec. 18. That forecast includes the anticipated savings from the St. Louis plant, according to today’s statement. GT Advanced also expects to take a charge of $80 million to $90 million for the fourth quarter to write down the majority of its inventory of its existing line of polysilicon production systems, known as the DSS family. David Gray , chief strategy and new business officer, will leave the company Jan. 25 and his position is being eliminated as Gutierrez assumes responsibility for GT Advanced’s reduced acquisition efforts The new structure will “streamline our operations and align our business with current market and business conditions,” Gutierrez said in the statement. GT Advanced fell 0.9 percent to $3.41 at the close in New York.


Weil's View on Finance, Afternoon Edition

Good afternoon, dear readers. Here is some of what I've been reading this afternoon while watching the markets go down. Hamptons excess back to being excessive The wealthy enclaves of Long Island, New York, called the Hamptons have fully rebounded -- and then some. Jim Rutenberg of the New York Times got an interview with Joe Farrell, the area's largest real-estate developer, and a tour of his $43 million, 17,000-square-foot home. The going rate to rent it for two weeks: $500,000. Europe's banks preparing for new regulator The European Central Bank is about to become the lead regulator for the euro area's biggest lenders. One of its first assignments will be an asset-quality review focusing on banks' loans. Patrick Jenkins of the Financial Times writes: "Properly done, the exercise could quickly establish the ECB as a tough regulator and help restore investor faith in Europe’s lenders. A succession of botched stress tests over recent years and laggardly recapitalization of troubled lenders have deterred investors from backing the sector." The real test will be if the ECB has the guts to try telling Deutsche Bank what to do. Angela Merkel blames her predecessor (who is from a different political party, of course) for Europe's debt crisis It's election season in Germany, and the country's leader is in full campaign mode. "For example, Greece shouldn't have been allowed into the euro" in 2001, Merkel told supporters at a rally in Rendsberg. “Chancellor [Gerhard] Schroeder accepted Greece in and weakened the Stability Pact, and both decisions were fundamentally wrong, and one of the starting points for our current troubles.” Jack Lew's letter to Congress on the debt ceiling Here we go again. The U.S. will hit the debt ceiling in mid-October, the Treasury secretary said. "Operating the government with no borrowing authority, and with only the cash on hand on a given day, would place the United States in an unacceptable position," Lew said. That's a given. But by now we're so accustomed to this ritual that the markets are assuming there will be a last-second deal, as usual. Haiku of the week (in case you didn't know there were business journalists out there who write things like this) This comes from Seattle Times columnist Jon Talton, who just returned from a two-week vacation: Two weeks off the grid I was differently informed Now, girding myself (Jonathan Weil is a Bloomberg View columnist. Follow him on Twitter.)


EMGS Confident of Boosting Backlog After Signing Brazil Deal

Electromagnetic GeoServices ASA (EMGS) , a Norwegian oil and gas surveyor, said it’s confident of increasing its order backlog after signing an agreement that expands its coverage of the waters off Brazil. “Towards the end of the year we will have at least one longer-term contract communicated,” Chief Financial Officer Svein Knudsen said in an interview in Oslo today. “We’re promising improved backlog and we’re sticking to that promise.” EMGS, based in Trondheim, plans to almost double its order backlog to at least $100 million by the end of the first half, Knudsen said in a presentation today. Shares in the company have fallen more than 30 percent since the start of the year amid concerns that the company’s backlog is shrinking. That’s even after EMGS signed a three-year agreement with Royal Dutch Shell Plc for the provision of 3D electromagnetic services globally. EMGS, which uses electromagnetic technology to search for oil and gas under the seabed, signed a deal with Spectrum ASA (SPU) to access its multi-client 2D survey library in order to plan a multi-client 3D survey in the Foz de Amazonas basin off Brazil, it said in a statement today. The survey will cover blocks that are included in the 11th licensing round, it said. EMGS expects to agree on similar collaborations elsewhere in the world with potential partners including Petroleum Geo- Services ASA (PGS) and TGS Nopec Geophysical ASA, Norway ’s two-largest seismic companies, Knudsen said. EMGS uses a method of mapping the seabed that competes with the seismic technology employed by companies such as Schlumberger Ltd., TGS and PGS. Shares in EMGS gained as much as 3.3 percent and closed unchanged at 9 kroner in Oslo.


Tesco Plans to Invest $750 Million in Digital Push, Chief Says

Tesco Plc, the U.K.’s largest grocer, will invest $750 million this year to accelerate the roll-out of digital services and plans to vie with Amazon.com Inc. and Netflix Inc. (NFLX) for U.K. customers, Chief Executive Officer Philip Clarke said. “To compete in the new area of retailing I believe that Tesco has to be more than a retail company,” Clarke said at a conference in Singapore today. “We have to become a technology company as well. That’s why, in 2013, we will invest three quarters of a billion U.S. dollars in technology, up threefold in three years.” Retailers across the world are racing to build online stores as customers shift to shopping via computers, mobile phones and tablets. Tesco has also created a digital platform called Blinkbox in response to customers increasingly downloading movies, books and music online, rather than buying DVDs or CDs in supermarkets or from town-center stores. “Last year in the U.K., sales of physical books, movies and music fell by 25 percent,” Clarke said. “Internet versions grew by 150 percent.” This month Tesco started a free Internet television service for the 16 million members of its Clubcard loyalty program. Clubcard customers can watch television shows and movies, with the content “specifically tailored to family and kids,” Tesco said. The channel, called Clubcard TV, will be funded by “targeted advertising,” Tesco said, and advertisers that have signed up for the introduction include Kellogg Co. (K) , Johnson & Johnson (JNJ) , Colgate-Palmolive Co. (CL) and Danone SA. (BN)  Tesco, which has stores in China, Thailand, Malaysia and Korea, will also roll out its online grocery offering across Asia, Clarke said. “Last year I called time on the old ‘space race’ -- I said that in the future, we wouldn’t simply grow by buying more real estate, but instead change the way we engage with our customers and embrace digital retailing,” he said.


Whether Public or Private, Information Technology Is Hard To Do Right

A timely reminder that the Department of Health and Human Services is not the only place having information technology woes: This at the NSA -- which is supposed to be so good at what it does that I am half expecting a cheery note from my personal NSA agent reminding me that I should have had whole wheat toast this morning instead of the cheap white bread. Yet the NSA is not even being undone by something difficult, like melding multiple databases into one coherent website. It's having trouble keeping electrical boxes from melting down, which is one of those problems I thought the utilities had solved decades ago. There’s a lesson in this for both sides of the political spectrum. We live our lives immersed in wondrous technology -- especially those of us who spend our workdays on the Internet. Over time, we’ve come to think that anyone can do this sort of thing. But IT is hard. Conservatives who argue that this shows the government can never do IT right should remember that lots of companies can -- and do -- get IT wrong. We think the private sector is so good at it because we see only the winners who got it right. The many projects that went horribly wrong have slipped out of view, and memory. But liberals who have been proclaiming that the health exchange glitches will be fixed eventually because after all, Amazon does this, should remember that the end of every glitchy project is not a product that actually works. Horrifyingly bad launches, into which category I’d say the exchanges now fall, often end when the product is jerked out of production. More than occasionally, the company that made the product goes away. Next time you go onto Amazon, or Netflix or Travelocity, remember these glitches and remind yourself what a hard job these companies are doing amazingly well. You live in an age of technological miracles. You should occasionally let yourself feel the awe that this ought to inspire.


Pearson’s Penguin Settles States’ E-Book Antitrust Claims

Pearson Plc’s (PSON) Penguin unit said it reached a $75 million settlement of antitrust claims by 33 states and consumers over electronic-book pricing. Penguin, which was sued last year for allegedly participating in a conspiracy with other publishers and Apple Inc. (AAPL) to fix the price of e-books, announced the settlement in a statement today. The company resolved related claims with the U.S. Justice Department in December. The states claimed damages on behalf of people who were forced to pay too much for e-books because of the alleged conspiracy. The Justice Department and the states claimed Apple and five publishers, including Penguin, fixed prices in an attempt to undermine Amazon.com Inc. (AMZN) ’s dominance of the market. The Penguin settlement must be approved by a federal judge in New York before it can take effect. Apple is scheduled to begin a trial June 3 on civil antitrust claims by the Justice Department and states. The case is U.S. v. Apple, 11-cv-02293, U.S. District Court, Southern District of New York ( Manhattan ).


Woodside May Ditch Gas Agreement in Israel, Deutsche Bank Says

Woodside Petroleum Ltd. (WPL) , Australia’s second-biggest oil and gas producer, may pull out of a deal to invest in Israel ’s largest natural gas field, freeing up as much as $1 billion to return to investors, Deutsche Bank AG said. “Woodside must be considering its position,” John Hirjee , a Melbourne-based analyst at Deutsche Bank, said today by phone as proposed limits to Israel’s natural gas export levels increased speculation Woodside will quit the Leviathan project. Perth-based Woodside agreed in December to pay as much as $2.3 billion for a stake in Leviathan to tap both the domestic gas and liquefied natural gas markets. Prime Minister Benjamin Netanyahu said in June Israel will cap exports of its natural gas resources at 40 percent, less than the 50 percent recommended last year by a government committee. “Reduced export levels from Leviathan compared to Woodside’s initial expectations, along with uncertainty around preferred export development concept by the joint venture,” may see the company walk away from the deal, Commonwealth Bank of Australia (CBA) analysts Luke Smith and Lachlan Cuskelly said in a July 26 note. Daniel Clery, a spokesman for Perth-based Woodside, declined to comment on Leviathan speculation. Initial Payment Woodside agreed to make an initial payment of $696 million to the project partners Noble Energy Inc. (NBL) , Delek Drilling LP, Avner Oil Exploration LLP and Ratio Oil Exploration (1992) LP. It also agreed to give the partners $200 million once laws permitting LNG exports come into force and $350 million when a final investment decision on an LNG project is made. “Closing of these agreements has been delayed primarily due to the uncertainty over exports,” Noble Chief Executive Officer Charles Davidson told analysts on a call last week. “Hopefully we are nearing closure on that issue.” While the Israeli policy limits exports at 40 percent, larger fields such as Leviathan would be able to export a higher amount, Davidson said on the call. Woodside decided in April to return about $520 million to investors in dividends after scrapping the Browse LNG proposal. Woodside also signaled a delay to the next stage of its Pluto LNG project in Australia , ending talks with companies to obtain gas supplies needed for expansion, the company said April 18. “One way to alleviate concern about lack of growth” options for Woodside would be to announce a share buyback should the Leviathan deal collapse, Deutsche Bank’s Hirjee said.


Fusion-io Surges on Sales Forecast and Acquisition of NexGen

Fusion-io Inc. (FIO) , a maker of data- storage computers, climbed the most in eight months after forecasting sales that topped analysts’ estimates and announcing the acquisition of NexGen Storage Inc. Revenue in the fourth quarter, which ends in June, will be about $110 million, the Salt Lake City-based company said yesterday in a statement. That’s higher than the average analyst estimate of $107.5 million, according to data compiled by Bloomberg. Fusion-io, whose biggest customers include Apple Inc. (AAPL) and Facebook Inc., is aiming to sell to small and medium-sized businesses with more affordable flash-memory-based services. The company said it purchased NexGen for about $119 million in cash and stock, and is already promoting NexGen’s hybrid storage product on its website. Fusion-io rose 20 percent to $20.03 at 9:46 a.m. in New York, and earlier touched $20.20 for the biggest intraday gain since August. The stock had dropped 27 percent this year through yesterday, compared with an 11 percent gain for the Standard & Poor’s 500 Index. While revenue in the fiscal third quarter fell 7 percent to $87.7 million, it topped the $80.4 million average analyst estimate. Fusion-io said its net loss widened to $20 million, or 21 cents a share, from $4.68 million, or 5 cents, a year earlier. Fusion-io, which employs Apple co-founder Steve Wozniak as chief scientist, sells corporate storage computers based on flash memory , which is faster and more expensive than widely used magnetic disk drives. NexGen, based in Louisville, Colorado , introduced its product in November 2011, a week after saying it raised $10 million from Next World Capital, Grotech Ventures and Access Venture Partners. “This would take years to build out as a stand-alone company,” NexGen Chief Executive Officer John Spiers wrote yesterday in a blog post. “Overnight, those concerns are eliminated.”


BMC Software Drops After Profit Forecast Misses: Houston Mover

BMC Software Inc. (BMC) , a maker of programs that help companies update servers and personal computers, fell after forecasting fiscal 2013 profit that missed analysts’ estimates, a sign it’s struggling to clinch big contracts. The shares declined 6.2 percent to $41.71 at the close in New York , the biggest decline since October 2011. The Houston- based company said yesterday that profit excluding some items will be $3.35 to $3.45 a share for fiscal 2013, which ends in March. That’s less than BMC’s prior forecast of $3.49 to $3.59 and trails analysts’ average $3.56 projection, according to data compiled by Bloomberg. Activist shareholder Elliott Associates LP pressed BMC to consider a sale last year, unsettling customers and making it harder for the company to clinch deals, BMC Chief Executive Officer Bob Beauchamp told analysts on a conference call yesterday. The company had two large contract renewals pushed to the March quarter from the December period, crimping results. “BMC underperformed across all metrics for the quarter,” and deals in the mainframe and server-software businesses didn’t get closed, Abhey Lamba, an analyst at Mizuho Securities USA Inc., wrote in a research report. “Despite having completed its accelerated share repurchase program, the company lowered its EPS target for the year, which is clearly disappointing.” ‘Growth Areas’ The company said it needs to be “more consistent and disciplined” in the way it secures big contracts, according to the statement. Management is “scrutinizing the entire company to improve our operational discipline,” Beauchamp said. BMC added board members and initiated a $1 billion share buyback to assuage Elliott’s complaints that it wasn’t doing enough to maximize shareholder value. Competitors are “aggressively” capitalizing on the pressure by Elliott, prompting many customers to seek personal visits from BMC management before completing purchases, Beauchamp said. Beauchamp said during a conference call with analysts that he’s undertaking an operational review aimed at directing spending toward better performing products to increase earnings, cash flow and operating margin amid low revenue growth. The company is trying to improve profitability even as revenue this year is projected to grow in the low single digits. Web-delivered software, cloud-computing management products and tools for managing mobile devices are selling well and will get a greater share of investment compared with underperforming products, he said. “We’re going to find a way to move funds to these growth areas,” Beauchamp said. “It’s just moving faster.” BMC kicked off the review after the end of the quarter on Dec. 31, according to Mark Stouse, a spokesman.


Aussie, N.Z. Dollars Rise Against Most Peers as Commodities Gain

The Australian and New Zealand dollars gained versus the majority of their most-traded counterparts as commodities rose for a second day. The South Pacific currencies were little changed against the U.S. dollar as stocks declined. Iron, Australia’s biggest export in dollar terms, climbed. The Aussie dollar was little changed at $1.0504 yesterday in New York trading. It rose 0.3 percent to A$1.2454 per euro and dropped 0.9 percent to 91.43 yen. New Zealand’s dollar, nicknamed the kiwi, was little changed at 83.66 U.S. cents. It rose 0.2 percent to NZ$1.5637 per euro and slid 0.9 percent to 72.83 yen. Standard & Poor’s GSCI Index of 24 raw materials rose as much as 0.6 percent before paring the gain to 0.2 percent. Iron- ore swaps advanced to the highest level in 14 months as China ’s accelerating economy boosted demand. January contracts gained 1.3 percent to $152 a dry metric ton, the highest since October 2011, according to GFI Group Inc. China is Australia’s biggest trade partner and New Zealand’s second-largest export market. The S&P 500 Index fell as much as 0.7 percent before trimming losses to 0.3 percent.


Ceragon Jumps Most in 1 Month on Order: Israel Overnight

Ceragon Networks Ltd. (CRNT) , a maker of systems for mobile-phone and Internet providers, rose the most in a month in New York after reporting orders of $3.5 million that tempered investors’ concern about a revenue drop. The shares climbed 2.3 percent to $3.11 yesterday, the most since June 7, to trade 0.3 percent above the stock listed in Tel Aviv. The Bloomberg Israel-US Equity Index gained a second day, led by Caesarstone Sdot Yam Ltd., a maker of quartz countertops. ClickSoftware (CKSW) Technologies Ltd., whose second-biggest investor is George Soros ’s Soros Fund Management LLC, extended a three-day plunge to 15 percent. Ceragon, which got 16 percent of sales from the Asia-Pacific region last year, said yesterday that a “large” mobile operator in the region placed the orders. Ceragon is still down 20 percent since April 8, when Chief Executive Officer Ira Palti cut revenue forecasts. The company has reported losses for nine consecutive quarters and is projected to post the biggest decline in annual sales in more than a decade, according to the average estimate of analysts surveyed by Bloomberg. The new order “shows some momentum,” Joseph Wolf , an analyst at Barclays Plc, who rates the stock the equivalent of hold, said in a telephone interview from Tel Aviv yesterday. “But it’s not an inflection point in the weak demand environment that’s an overhang on Ceragon’s growth rate. It’s a good company with a very tough end market.” The Bloomberg Israel-US gauge rose 0.7 percent to 92.15, the highest price since June 18. The benchmark TA-25 Index (TA-25) advanced for the first time in three days, adding 0.7 percent to 1,214.69 at 10:24 a.m. in Tel Aviv. Ceragon’s Tel Aviv shares this morning slipped 0.8 percent to 11.19 shekels, or $3.09, in volume of 4 percent of the three-month daily average. Caesarstone (CSTE) jumped 6 percent to $29.10, the highest price since May 30. The shares have rallied 82 percent in 2013. ClickSoftware, an Israeli software developer, declined 4.4 percent to $7.11, the lowest price since April 23. Earlier this week, the Petach Tikva, Israel-based company reported profit and revenue that trailed analysts’ projections.


Atos Reviews Options for Worldline Electronic-Payments Unit

Atos (ATO) , a rival of computer-services giant International Business Machines Corp., confirmed sales will grow this year and profitability will improve as it seeks acquisitions to strengthen its payments unit. Net income rose 14 percent to 116 million euros ($153 million) in the first half as sales fell 0.6 percent from a year earlier to 4.3 billion euros, the Bezons, France-based company said yesterday in a statement. Atos confirmed its yearly group forecast for “slight revenue growth” and an operating margin around 7.5 percent of sales, up from 6.6 percent in 2012 and 6.5 percent in the first half of this year. Shares fell 2.7 percent to 56.43 euros at 11:03 a.m. in Paris. Atos -- headed by Thierry Breton, a former finance minister who became chief executive officer in November 2008 -- has completed carving out its electronic payment activities into a unit dubbed Worldline. The move is geared at giving Atos more financial flexibility to expand the payments business, including through acquisitions. “Worldline is now ready to seize opportunities in an industry that is poised to consolidate,” Atos’s Senior Executive Vice President Gilles Grapinet said yesterday during a conference call. A share of Worldline could be sold to one or more partners should an “industrial opportunity” present itself to Worldline, he said. An initial public offering is also an option, he said. Grapinet declined to say whether Atos is in talks on acquisitions or Worldline’s financing. The company has been approached by bankers about the unit’s options, the executive had said in an interview in April. Atos shares have risen about 10 percent this year through yesterday.


Ballmer by the Numbers: How Microsoft Has Fared During the CEO's Reign

When investors discuss the legacy of Steve Ballmer, they often cite Microsoft's share drop from the heights of the bubbly 1990s. The company's stock price was soaring at a split-adjusted high of $59.56 just weeks before Ballmer took over as CEO in January 2000, and since that year, it has never closed above $40. Wall Street's tacit disapproval of Ballmer is an important factor in the executive's legacy as the company, which reports earnings later today, searches for his successor. But there are other numbers that help define the 13-year tenure of the software giant's second CEO. Ballmer is no Bill Gates, but he presided over a six-fold increase in server-software sales, now one of the company's largest businesses. Sales from the Office unit tripled. Ballmer instituted dividends that would have yielded $8.23 for investors who have held the stock the whole time the CEO was in charge. (Many of them didn't.) There was no Xbox for sale before Ballmer. After amassing huge losses early on, it's gone on to generate $7.1 billion in sales during the last fiscal year. Perhaps more important, people actually love the product. Even the Windows business, Microsoft's original cash cow, has more than doubled in sales since Ballmer became CEO. But that number doesn't tell the full story. In 2000 , PCs were really the only computing device in town for most people, and Windows had more than 90 percent of the market. With the advent of smartphones and tablets, and Microsoft's complete failure to amass share there, Goldman Sachs estimated last year that Windows runs on just 19 percent of the computing devices consumers use. That's the number that should cause the most agony for Ballmer's successor.


Nissan Opens Silicon Valley Center for Self-Driving Car Research

Nissan Motor Co. opened a research center in California’s Silicon Valley that the Japanese carmaker plans to make its hub for research on self-driving vehicles and Internet-connected auto technology. The facility will be staffed by more than 60 engineers and technicians within three years, Carla Bailo , Nissan’s senior vice president for North American research and development, said in a phone interview. Work on so-called autonomous vehicle systems will move from an R&D center in Japan to Sunnyvale, she said, declining to provide investment details for the project. “We’re going to focus on this technology really in the heart of where it lies today,” Bailo said in the Feb. 15 interview. “We are going to be shifting work that’s being done in our Nissan Advanced Technical Center in Atsugi, and moving that work over to the heart of the industry.” Automakers and companies including Google Inc. plan new electronics and programming for vehicles with a goal of reducing collisions, fuel use and improving traffic flow. Carlos Ghosn , Nissan’s chief executive officer, told reporters last month in Detroit that regulatory, legal and technological hurdles mean self-driving cars won’t be feasible until about 2020. Google has tested a self-driving Prius in California for years, and the U.S. Defense Department has sponsored autonomous- vehicle research for more than a decade. Nissan’s Sunnyvale center will work with automotive labs at Stanford University, the University of California’s Berkeley and Davis campuses, and Silicon Valley companies, Bailo said. The Yokohama-based carmaker plans to apply technology developed at the new R&D center within a decade, she said. Maarten Sierhuis , a former NASA scientist specializing in artificial intelligence research, will lead the Sunnyvale facility, Nissan said today. Toyota Motor Corp. opened its own Silicon Valley outpost in Mountain View in 2012 to work with technology companies there on in-car data and entertainment systems. The company said in January that it’s developing autonomous safety systems to create a virtual “co-pilot” in vehicles to help drivers avoid accidents rather than self-driving cars and trucks.


ARM Sees Shift to Cheaper Smartphones Boosting Chip Royalty

ARM Holdings Plc (ARM) , the U.K. chip designer whose products power Apple Inc. (AAPL) ’s iPhones, predicts a shift in demand toward lower-priced smartphones will help boost royalty revenue. “There is a shift advantage for ARM as mid-range and entry smartphones drive smartphone growth. We’ve got that dynamic going into next year,” Chief Financial Officer Tim Score said at a conference in Barcelona yesterday. “Those phones generate much more royalty for ARM because they use more silicon.” ARM has had five consecutive years of revenue growth as its designs for chips that control graphics and processing were used in increasingly popular smartphones and tablets. Royalty payments took the spotlight last quarter as they missed analysts’ estimates. ARM shares have fallen almost 9 percent since the earnings report on Oct. 22. “As ARM technology becomes more sophisticated, the royalty percentage goes up,” Score said. “On average, it’s going to very gradually go up.” Global smartphone shipments surged 40 percent last quarter, slowing from 53 percent growth for the previous three-month period, according to researcher IDC. The average selling prices of the devices fell by 13 percent to $317 from a year earlier, because of demand for more affordable handsets. In spite of concerns about slowing growth, especially in China, Score said ARM can continue expanding its customer base and outperforming the semiconductor market in the next five years, driven by cheaper handsets, enterprise networking and micro-controllers. ARM shares fell less than 1 percent to 953.50 pence at 9:16 a.m. in London. New Chief Chief Executive Officer Simon Segars, who succeeded Warren East in July, is confronting a shift in the industry’s growth to low-end smartphones and new entrant Intel Corp. (INTC) , which has begun designing its own chips for mobile phones. ARM, based in Cambridge, England, is also seeking to boost its market share in servers as companies from Facebook Inc. (FB) to Google Inc. (GOOG) look for lower-power machines to drive their data centers. It’s “possible to identify such a market and build a chip specifically for that market, whether it is to handle only storage or only video loading, for example. We can make the chip much more efficient,” Score said at the annual Technology, Media and Telecoms Conference organized by Morgan Stanley. “Everyone knows Intel has a very strong ecosystem around servers, we’re not kidding ourselves,” Score said. “For that reason we see it as further out.”


Fusion-io Surges on Sales Forecast and Acquisition of NexGen

Fusion-io Inc. (FIO) , a maker of data- storage computers, climbed the most in eight months after forecasting sales that topped analysts’ estimates and announcing the acquisition of NexGen Storage Inc. Revenue in the fourth quarter, which ends in June, will be about $110 million, the Salt Lake City-based company said yesterday in a statement. That’s higher than the average analyst estimate of $107.5 million, according to data compiled by Bloomberg. Fusion-io, whose biggest customers include Apple Inc. (AAPL) and Facebook Inc., is aiming to sell to small and medium-sized businesses with more affordable flash-memory-based services. The company said it purchased NexGen for about $119 million in cash and stock, and is already promoting NexGen’s hybrid storage product on its website. Fusion-io rose 20 percent to $20.03 at 9:46 a.m. in New York, and earlier touched $20.20 for the biggest intraday gain since August. The stock had dropped 27 percent this year through yesterday, compared with an 11 percent gain for the Standard & Poor’s 500 Index. While revenue in the fiscal third quarter fell 7 percent to $87.7 million, it topped the $80.4 million average analyst estimate. Fusion-io said its net loss widened to $20 million, or 21 cents a share, from $4.68 million, or 5 cents, a year earlier. Fusion-io, which employs Apple co-founder Steve Wozniak as chief scientist, sells corporate storage computers based on flash memory , which is faster and more expensive than widely used magnetic disk drives. NexGen, based in Louisville, Colorado , introduced its product in November 2011, a week after saying it raised $10 million from Next World Capital, Grotech Ventures and Access Venture Partners. “This would take years to build out as a stand-alone company,” NexGen Chief Executive Officer John Spiers wrote yesterday in a blog post. “Overnight, those concerns are eliminated.”


BMC Software Drops After Profit Forecast Misses: Houston Mover

BMC Software Inc. (BMC) , a maker of programs that help companies update servers and personal computers, fell after forecasting fiscal 2013 profit that missed analysts’ estimates, a sign it’s struggling to clinch big contracts. The shares declined 6.2 percent to $41.71 at the close in New York , the biggest decline since October 2011. The Houston- based company said yesterday that profit excluding some items will be $3.35 to $3.45 a share for fiscal 2013, which ends in March. That’s less than BMC’s prior forecast of $3.49 to $3.59 and trails analysts’ average $3.56 projection, according to data compiled by Bloomberg. Activist shareholder Elliott Associates LP pressed BMC to consider a sale last year, unsettling customers and making it harder for the company to clinch deals, BMC Chief Executive Officer Bob Beauchamp told analysts on a conference call yesterday. The company had two large contract renewals pushed to the March quarter from the December period, crimping results. “BMC underperformed across all metrics for the quarter,” and deals in the mainframe and server-software businesses didn’t get closed, Abhey Lamba, an analyst at Mizuho Securities USA Inc., wrote in a research report. “Despite having completed its accelerated share repurchase program, the company lowered its EPS target for the year, which is clearly disappointing.” ‘Growth Areas’ The company said it needs to be “more consistent and disciplined” in the way it secures big contracts, according to the statement. Management is “scrutinizing the entire company to improve our operational discipline,” Beauchamp said. BMC added board members and initiated a $1 billion share buyback to assuage Elliott’s complaints that it wasn’t doing enough to maximize shareholder value. Competitors are “aggressively” capitalizing on the pressure by Elliott, prompting many customers to seek personal visits from BMC management before completing purchases, Beauchamp said. Beauchamp said during a conference call with analysts that he’s undertaking an operational review aimed at directing spending toward better performing products to increase earnings, cash flow and operating margin amid low revenue growth. The company is trying to improve profitability even as revenue this year is projected to grow in the low single digits. Web-delivered software, cloud-computing management products and tools for managing mobile devices are selling well and will get a greater share of investment compared with underperforming products, he said. “We’re going to find a way to move funds to these growth areas,” Beauchamp said. “It’s just moving faster.” BMC kicked off the review after the end of the quarter on Dec. 31, according to Mark Stouse, a spokesman.


EMGS Confident of Boosting Backlog After Signing Brazil Deal

Electromagnetic GeoServices ASA (EMGS) , a Norwegian oil and gas surveyor, said it’s confident of increasing its order backlog after signing an agreement that expands its coverage of the waters off Brazil. “Towards the end of the year we will have at least one longer-term contract communicated,” Chief Financial Officer Svein Knudsen said in an interview in Oslo today. “We’re promising improved backlog and we’re sticking to that promise.” EMGS, based in Trondheim, plans to almost double its order backlog to at least $100 million by the end of the first half, Knudsen said in a presentation today. Shares in the company have fallen more than 30 percent since the start of the year amid concerns that the company’s backlog is shrinking. That’s even after EMGS signed a three-year agreement with Royal Dutch Shell Plc for the provision of 3D electromagnetic services globally. EMGS, which uses electromagnetic technology to search for oil and gas under the seabed, signed a deal with Spectrum ASA (SPU) to access its multi-client 2D survey library in order to plan a multi-client 3D survey in the Foz de Amazonas basin off Brazil, it said in a statement today. The survey will cover blocks that are included in the 11th licensing round, it said. EMGS expects to agree on similar collaborations elsewhere in the world with potential partners including Petroleum Geo- Services ASA (PGS) and TGS Nopec Geophysical ASA, Norway ’s two-largest seismic companies, Knudsen said. EMGS uses a method of mapping the seabed that competes with the seismic technology employed by companies such as Schlumberger Ltd., TGS and PGS. Shares in EMGS gained as much as 3.3 percent and closed unchanged at 9 kroner in Oslo.


Chelsea Market Owner Purchases L.A. Complex Housing IMAX

The firm that owns New York’s One Times Square, where the ball drops each New Year’s Eve, bought a California office-and-studio complex that houses Dick Clark Productions and big-screen movie-theater operator IMAX Corp. (IMX)  Jamestown , which also owns Chelsea Market in New York, bought the 484,840-square-foot (45,043-square-meter) Lantana campus in Santa Monica from Lionstone Group, Atlanta-based Jamestown said today in a statement. The purchase price was about $310 million, according to a person with knowledge of the transaction. “The innovation economy is driving demand for work spaces that offer a vibrant mix of amenities, metropolitan ambience and collaborative, flexible environments,” Jamestown Chief Operating Officer Michael Phillips said in the statement. Lantana is the company’s first purchase in Southern California. The property’s tenants include film-sound company Todd-AO and workout-video producer Beachbody LLC. The complex is 94 percent leased, with “exceptional in-place cash flow,” Jamestown said. The company also owns properties in San Luis Obispo, California , and the San Francisco Bay area.


Afren Sees 29% Growth in Oil Reserves After Expanding in Nigeria

Afren Plc (AFR) , a U.K.-based oil and gas company focused on West Africa and Iraq , will expand reserves by 29 percent after increasing its stake in a Nigerian explorer. Proven and probable reserves will climb to 270 million barrels of oil equivalent from 210 million barrels, the London- based company said today in a statement. Afren agreed to pay about $37 million for 10.4 percent of First Hydrocarbon Nigeria Co., raising its stake to about 55 percent. Oil companies are adding reserves to gain from a four-year increase in prices. Nigeria, Africa’s biggest oil-producing nation, accounted for 92 percent of Afren’s sales in 2011, data compiled by Bloomberg show. The country provides cash to invest in other regions including East Africa and the Middle East. “We are currently drilling very exciting wells in Kurdistan, which are going very well,” Chief Executive Officer Osman Shahenshah said today in an interview. In Africa, the company is adding a “whole suite” of wells in Kenya, Tanzania and Madagascar, he said. Afren advanced as much as 3.2 percent to 152.3 pence in London trading, the highest in a month. The stock was at 152 pence as of 9:59 a.m. local time, extending its gain this year to 16 percent. The company also reported a 66 percent jump in 2012 net income to $202.5 million. Sales rose to a record $1.5 billion, according to a separate statement today. Its exploration program is targeting 650 million barrels of oil and gas resources this year, and output is forecast at 40,000 to 47,000 barrels a day. Afren began production at the Barda Rash field in Iraqi Kurdistan last year. Volumes from the deposit may enable the company to exceed its output guidance in 2013, Galib Virani, an associate director at Afren, said in January. The company’s other producing assets are in Nigeria and Ivory Coast. Elsewhere in Africa, it completed a seismic study in the Seychelles this quarter, the first three-dimensional survey in the archipelago, where the company may hold 2.8 billion barrels in prospective resources, it said.


Broadwind Climbs on $87 Million Order for Turbine Towers

Broadwind Energy Inc. (BWEN) , a Cicero, Illinois-based maker of steel towers, rose to the highest in more than a year after winning an order worth $87 million from a U.S. wind-turbine manufacturer. Broadwind rose 10 percent to $5.50 at 8:52 a.m. in premarket trading in New York. The shares have more than doubled this year. The company will manufacture the towers at its factories in Abilene, Texas , and Manitowoc, Wisconsin , and plans to deliver them next year, according to a statement today. The buyer wasn’t identified. Broadwind has received orders for $200 million worth of towers this year, the company said. “Broadwind’s tower business continues to benefit from the rebound of the U.S. wind energy industry as a result of the Production Tax Credit extension and the continuing decline in the cost of electricity from wind energy,” Chief Executive Officer Peter C. Duprey said in the statement.


Google Poised to End U.S. Antitrust Probe With Agreement

The U.S. is set to close its antitrust probe of Google Inc. (GOOG) with an agreement letter from the company that addresses the use of other websites’ content and the ability of advertisers to transfer data to other platforms, a person familiar with the matter said. The Federal Trade Commission announced it will hold a press conference today about the Google investigation at 1 p.m. The Google letter, which the agency will describe as enforceable by law, will offer other websites the option to say they don’t want their content to be displayed in Google search results, said the person, who asked not to be named because details of the FTC’s decision aren’t public. The letter will also include a non-discrimination provision on Google’s advertising practices, allowing advertisers to compare data from their campaigns across platforms, the person said. The decision to close the probe of Google’s search practices without enforcement action is a blow to competitors including Microsoft Corp. (MSFT) , Yelp Inc. (YELP) , and Expedia Inc. (EXPE) An alliance of such e-commerce and Web-search companies pressed the agency to bring a lawsuit, claiming Google’s dominance of Internet search, combined with favoring its own services in answers to queries, violates antitrust laws. Unprecedented Step Matt Reilly, a former FTC litigator who represents Fairsearch.org, the alliance of Google competitors, said it was unprecedented for the agency to resolve a case with a voluntary agreement. “Any time we were negotiating and the parties would say ‘Can we do this without a consent decree?’ the answer was always ‘No,’” Reilly said. “Now, everybody is going to want the deal Google got. There is a lot of disappointment.” Google is expected to settle allegations it misused patents to thwart competitors in smartphone technology, three people familiar with the matter have said.


Fusion-io Surges on Sales Forecast and Acquisition of NexGen

Fusion-io Inc. (FIO) , a maker of data- storage computers, climbed the most in eight months after forecasting sales that topped analysts’ estimates and announcing the acquisition of NexGen Storage Inc. Revenue in the fourth quarter, which ends in June, will be about $110 million, the Salt Lake City-based company said yesterday in a statement. That’s higher than the average analyst estimate of $107.5 million, according to data compiled by Bloomberg. Fusion-io, whose biggest customers include Apple Inc. (AAPL) and Facebook Inc., is aiming to sell to small and medium-sized businesses with more affordable flash-memory-based services. The company said it purchased NexGen for about $119 million in cash and stock, and is already promoting NexGen’s hybrid storage product on its website. Fusion-io rose 20 percent to $20.03 at 9:46 a.m. in New York, and earlier touched $20.20 for the biggest intraday gain since August. The stock had dropped 27 percent this year through yesterday, compared with an 11 percent gain for the Standard & Poor’s 500 Index. While revenue in the fiscal third quarter fell 7 percent to $87.7 million, it topped the $80.4 million average analyst estimate. Fusion-io said its net loss widened to $20 million, or 21 cents a share, from $4.68 million, or 5 cents, a year earlier. Fusion-io, which employs Apple co-founder Steve Wozniak as chief scientist, sells corporate storage computers based on flash memory , which is faster and more expensive than widely used magnetic disk drives. NexGen, based in Louisville, Colorado , introduced its product in November 2011, a week after saying it raised $10 million from Next World Capital, Grotech Ventures and Access Venture Partners. “This would take years to build out as a stand-alone company,” NexGen Chief Executive Officer John Spiers wrote yesterday in a blog post. “Overnight, those concerns are eliminated.”


Nissan Opens Silicon Valley Center for Self-Driving Car Research

Nissan Motor Co. opened a research center in California’s Silicon Valley that the Japanese carmaker plans to make its hub for research on self-driving vehicles and Internet-connected auto technology. The facility will be staffed by more than 60 engineers and technicians within three years, Carla Bailo , Nissan’s senior vice president for North American research and development, said in a phone interview. Work on so-called autonomous vehicle systems will move from an R&D center in Japan to Sunnyvale, she said, declining to provide investment details for the project. “We’re going to focus on this technology really in the heart of where it lies today,” Bailo said in the Feb. 15 interview. “We are going to be shifting work that’s being done in our Nissan Advanced Technical Center in Atsugi, and moving that work over to the heart of the industry.” Automakers and companies including Google Inc. plan new electronics and programming for vehicles with a goal of reducing collisions, fuel use and improving traffic flow. Carlos Ghosn , Nissan’s chief executive officer, told reporters last month in Detroit that regulatory, legal and technological hurdles mean self-driving cars won’t be feasible until about 2020. Google has tested a self-driving Prius in California for years, and the U.S. Defense Department has sponsored autonomous- vehicle research for more than a decade. Nissan’s Sunnyvale center will work with automotive labs at Stanford University, the University of California’s Berkeley and Davis campuses, and Silicon Valley companies, Bailo said. The Yokohama-based carmaker plans to apply technology developed at the new R&D center within a decade, she said. Maarten Sierhuis , a former NASA scientist specializing in artificial intelligence research, will lead the Sunnyvale facility, Nissan said today. Toyota Motor Corp. opened its own Silicon Valley outpost in Mountain View in 2012 to work with technology companies there on in-car data and entertainment systems. The company said in January that it’s developing autonomous safety systems to create a virtual “co-pilot” in vehicles to help drivers avoid accidents rather than self-driving cars and trucks.


EMGS Confident of Boosting Backlog After Signing Brazil Deal

Electromagnetic GeoServices ASA (EMGS) , a Norwegian oil and gas surveyor, said it’s confident of increasing its order backlog after signing an agreement that expands its coverage of the waters off Brazil. “Towards the end of the year we will have at least one longer-term contract communicated,” Chief Financial Officer Svein Knudsen said in an interview in Oslo today. “We’re promising improved backlog and we’re sticking to that promise.” EMGS, based in Trondheim, plans to almost double its order backlog to at least $100 million by the end of the first half, Knudsen said in a presentation today. Shares in the company have fallen more than 30 percent since the start of the year amid concerns that the company’s backlog is shrinking. That’s even after EMGS signed a three-year agreement with Royal Dutch Shell Plc for the provision of 3D electromagnetic services globally. EMGS, which uses electromagnetic technology to search for oil and gas under the seabed, signed a deal with Spectrum ASA (SPU) to access its multi-client 2D survey library in order to plan a multi-client 3D survey in the Foz de Amazonas basin off Brazil, it said in a statement today. The survey will cover blocks that are included in the 11th licensing round, it said. EMGS expects to agree on similar collaborations elsewhere in the world with potential partners including Petroleum Geo- Services ASA (PGS) and TGS Nopec Geophysical ASA, Norway ’s two-largest seismic companies, Knudsen said. EMGS uses a method of mapping the seabed that competes with the seismic technology employed by companies such as Schlumberger Ltd., TGS and PGS. Shares in EMGS gained as much as 3.3 percent and closed unchanged at 9 kroner in Oslo.


Ceragon Jumps Most in 1 Month on Order: Israel Overnight

Ceragon Networks Ltd. (CRNT) , a maker of systems for mobile-phone and Internet providers, rose the most in a month in New York after reporting orders of $3.5 million that tempered investors’ concern about a revenue drop. The shares climbed 2.3 percent to $3.11 yesterday, the most since June 7, to trade 0.3 percent above the stock listed in Tel Aviv. The Bloomberg Israel-US Equity Index gained a second day, led by Caesarstone Sdot Yam Ltd., a maker of quartz countertops. ClickSoftware (CKSW) Technologies Ltd., whose second-biggest investor is George Soros ’s Soros Fund Management LLC, extended a three-day plunge to 15 percent. Ceragon, which got 16 percent of sales from the Asia-Pacific region last year, said yesterday that a “large” mobile operator in the region placed the orders. Ceragon is still down 20 percent since April 8, when Chief Executive Officer Ira Palti cut revenue forecasts. The company has reported losses for nine consecutive quarters and is projected to post the biggest decline in annual sales in more than a decade, according to the average estimate of analysts surveyed by Bloomberg. The new order “shows some momentum,” Joseph Wolf , an analyst at Barclays Plc, who rates the stock the equivalent of hold, said in a telephone interview from Tel Aviv yesterday. “But it’s not an inflection point in the weak demand environment that’s an overhang on Ceragon’s growth rate. It’s a good company with a very tough end market.” The Bloomberg Israel-US gauge rose 0.7 percent to 92.15, the highest price since June 18. The benchmark TA-25 Index (TA-25) advanced for the first time in three days, adding 0.7 percent to 1,214.69 at 10:24 a.m. in Tel Aviv. Ceragon’s Tel Aviv shares this morning slipped 0.8 percent to 11.19 shekels, or $3.09, in volume of 4 percent of the three-month daily average. Caesarstone (CSTE) jumped 6 percent to $29.10, the highest price since May 30. The shares have rallied 82 percent in 2013. ClickSoftware, an Israeli software developer, declined 4.4 percent to $7.11, the lowest price since April 23. Earlier this week, the Petach Tikva, Israel-based company reported profit and revenue that trailed analysts’ projections.


GT Advanced to Idle Plant, Expects Solar Recovery in 2014

GT Advanced Technologies Inc. (GTAT) , a U.S. supplier of manufacturing equipment for solar and energy- efficient lighting products, plans to sell a factory in St. Louis. The facility is developing high-efficiency solar technology that may not be needed until demand for photovoltaic panels picks up next year. The company will fire 35 people at the polysilicon plant, according to a statement today. Nashua, New Hampshire-based GT Advanced also said it will scale back its acquisition efforts this year. A global glut of solar production capacity drove down panel prices 61 percent in the past two years and slowed sales of GT Advanced’s systems. The company is transferring development of a high-efficiency polysilicon technology, HiCz, to a Merrimack, New Hampshire , plant from St. Louis. The systems will be ready this year and Chief Executive Officer Tom Gutierrez expects orders in 2014 when he anticipates solar demand to recover. “Given current market conditions in the PV industry, orders for the new generation systems may not occur until 2014 when the industry begins to recover,” the company said in the statement. Idling the plant will save about $15 million annually. The company will incur charges, which weren’t quantified, for the fourth quarter and in the current year for the restructuring. 2013 Forecast GT Advanced expects 2013 earnings of 25 cents to 45 cents a share, it said Dec. 18. That forecast includes the anticipated savings from the St. Louis plant, according to today’s statement. GT Advanced also expects to take a charge of $80 million to $90 million for the fourth quarter to write down the majority of its inventory of its existing line of polysilicon production systems, known as the DSS family. David Gray , chief strategy and new business officer, will leave the company Jan. 25 and his position is being eliminated as Gutierrez assumes responsibility for GT Advanced’s reduced acquisition efforts The new structure will “streamline our operations and align our business with current market and business conditions,” Gutierrez said in the statement. GT Advanced fell 0.9 percent to $3.41 at the close in New York.


BMC Software Drops After Profit Forecast Misses: Houston Mover

BMC Software Inc. (BMC) , a maker of programs that help companies update servers and personal computers, fell after forecasting fiscal 2013 profit that missed analysts’ estimates, a sign it’s struggling to clinch big contracts. The shares declined 6.2 percent to $41.71 at the close in New York , the biggest decline since October 2011. The Houston- based company said yesterday that profit excluding some items will be $3.35 to $3.45 a share for fiscal 2013, which ends in March. That’s less than BMC’s prior forecast of $3.49 to $3.59 and trails analysts’ average $3.56 projection, according to data compiled by Bloomberg. Activist shareholder Elliott Associates LP pressed BMC to consider a sale last year, unsettling customers and making it harder for the company to clinch deals, BMC Chief Executive Officer Bob Beauchamp told analysts on a conference call yesterday. The company had two large contract renewals pushed to the March quarter from the December period, crimping results. “BMC underperformed across all metrics for the quarter,” and deals in the mainframe and server-software businesses didn’t get closed, Abhey Lamba, an analyst at Mizuho Securities USA Inc., wrote in a research report. “Despite having completed its accelerated share repurchase program, the company lowered its EPS target for the year, which is clearly disappointing.” ‘Growth Areas’ The company said it needs to be “more consistent and disciplined” in the way it secures big contracts, according to the statement. Management is “scrutinizing the entire company to improve our operational discipline,” Beauchamp said. BMC added board members and initiated a $1 billion share buyback to assuage Elliott’s complaints that it wasn’t doing enough to maximize shareholder value. Competitors are “aggressively” capitalizing on the pressure by Elliott, prompting many customers to seek personal visits from BMC management before completing purchases, Beauchamp said. Beauchamp said during a conference call with analysts that he’s undertaking an operational review aimed at directing spending toward better performing products to increase earnings, cash flow and operating margin amid low revenue growth. The company is trying to improve profitability even as revenue this year is projected to grow in the low single digits. Web-delivered software, cloud-computing management products and tools for managing mobile devices are selling well and will get a greater share of investment compared with underperforming products, he said. “We’re going to find a way to move funds to these growth areas,” Beauchamp said. “It’s just moving faster.” BMC kicked off the review after the end of the quarter on Dec. 31, according to Mark Stouse, a spokesman.


Samsung Boosting Venture-Capital Funding to $1.1 Billion

Samsung Electronics Co. (005930) , the world’s largest maker of mobile phones, said it was boosting venture- capital funding by $100 million to help create new businesses for its unit making memory chips and electronics parts. The company unveiled the Catalyst Fund for early stage startups, as well as a strategy and innovation center in Menlo Park , California , at an event near the site on Sand Hill Road. The company and affiliates also run the $1 billion Samsung Ventures America Fund. “We must reach out to global hot spots and global talent,” Young Sohn, president and chief strategy officer of device solutions for Samsung, said at the event. “This is the epicenter of disruptive forces.” Like other companies such as Intel Corp. (INTC) , Google Inc. and Microsoft Corp., Samsung is trying to use its cash to encourage the development of technology that will benefit its business in the future. Traditional venture-capital investors aren’t investing in basic science and technology startups, creating a gap that Samsung intends to help fill, Sohn said. Samsung’s venture arm, now headquartered in Seoul, has more than $1 billion in assets under management and did 20 deals worth $160 million last year, Sohn said. Investments by the new funds will be in the areas of Internet-connected devices, mobile health, computer-human interface, cloud-data centers and mobile privacy, he said. Shares of the Suwon, South Korea-based company fell 0.1 percent to 1,435,000 won in South Korean trading today.


Google Says Mosley Suit Over Sex-Party Search Defies Free Speech

Google Inc. (GOOG) said a lawsuit by Max Mosley, Formula One’s former president, asking it to block any further search results referring to a “Nazi-themed” sex party would create unprecedented censorship on the Internet. Lawyers for Mosley asked a French court today to force Google, the owner of the world’s most-used Internet search engine, to build software filters that would detect and delete certain content, Daphne Keller, an associate general counsel at Google, said in a post on its website. “We have removed hundreds of pages for Mr. Mosley, and stand ready to remove others he identifies,” Keller said. “But the law does not support Mr. Mosley’s demand for the construction of an unprecedented new Internet censorship tool.” Mosley, 73, won a 60,000 pound ($94,000) breach-of-privacy award in a U.K. court in 2008 from News Corp.’s now-defunct News of the World newspaper for publishing the story on a Nazi-themed “orgy,” along with a video, without contacting him. A judge ruled there was no Nazi theme and the story wasn’t in the public interest. Mosley won a similar ruling in France in 2011 when a judge ordered News Corp. to pay as much as 32,000 euros in fines and fees over the story. Mosley in 2011 told a U.K. inquiry investigating the phone-hacking scandal at News Corp. that he filed suits against Google in Germany and France over the search results. ‘People’s Dignity’ France and Germany are “pilot cases” and depending on how successful Mosley is, and if he doesn’t reach an agreement with Google out of court, there will be more lawsuits, said Tanja Irion, a lawyer representing Mosley at law firm Irion in Hamburg, Germany. The suits could be filed in other EU countries and in Mountain View, California , where Google is based, she said. “The case is very important, not only for Mr. Mosley, also generally for the protection of people’s dignity and privacy on the Internet,” Irion said. The German case will be heard by the Hamburg court on September 20, the court’s press office said today. Rulings in both cases can be appealed. The case raises similar issues with a Belgian suit that ended up in the EU’s top court, testing the scope of the powers of national courts. The EU Court of Justice in Luxembourg in a precedent-setting case ruled in 2011 that a national court cannot order an Internet-service provider to install a filtering system to prevent illegal downloading of files.


HootSuite Gets $165 Million to Expand Social-Media Tools

HootSuite Media Inc. , a developer of software used to manage social-networking activity, has raised $165 million from investors in one of the largest venture capital deals in North America this year. Insight Venture Partners led the second financing round, which also included Accel Partners and OMERS Ventures, Vancouver-based HootSuite said in a statement today. Terms of the investment weren’t disclosed. HootSuite plans to use the funds to add more than 100 employees in the next six months and expand to new markets including Asia and Latin America , Chief Executive Officer Ryan Holmes said in an interview. The investment will also help HootSuite build its business without seeking funds via an initial public offering, he said. “I personally would love to take this company to an acquisition or an IPO,” Holmes said. “That being said, my goal right now is building an amazing business. I think this raise takes some of that pressure off.” The company is also in talks to make at least two acquisitions, Holmes said, declining to name the targets. HootSuite has almost doubled in size, to 300 employees, in the past year as it adds tools to help businesses manage their presence on Facebook Inc. (FB) , Twitter Inc. and other social sites. The financing is the third-biggest venture deal for a North American technology company this year, behind the $200 million investment in mobile-device management company AirWatch LLC and online bulletin board Pinterest Inc., according to data compiled by Bloomberg.


Nissan Opens Silicon Valley Center for Self-Driving Car Research

Nissan Motor Co. opened a research center in California’s Silicon Valley that the Japanese carmaker plans to make its hub for research on self-driving vehicles and Internet-connected auto technology. The facility will be staffed by more than 60 engineers and technicians within three years, Carla Bailo , Nissan’s senior vice president for North American research and development, said in a phone interview. Work on so-called autonomous vehicle systems will move from an R&D center in Japan to Sunnyvale, she said, declining to provide investment details for the project. “We’re going to focus on this technology really in the heart of where it lies today,” Bailo said in the Feb. 15 interview. “We are going to be shifting work that’s being done in our Nissan Advanced Technical Center in Atsugi, and moving that work over to the heart of the industry.” Automakers and companies including Google Inc. plan new electronics and programming for vehicles with a goal of reducing collisions, fuel use and improving traffic flow. Carlos Ghosn , Nissan’s chief executive officer, told reporters last month in Detroit that regulatory, legal and technological hurdles mean self-driving cars won’t be feasible until about 2020. Google has tested a self-driving Prius in California for years, and the U.S. Defense Department has sponsored autonomous- vehicle research for more than a decade. Nissan’s Sunnyvale center will work with automotive labs at Stanford University, the University of California’s Berkeley and Davis campuses, and Silicon Valley companies, Bailo said. The Yokohama-based carmaker plans to apply technology developed at the new R&D center within a decade, she said. Maarten Sierhuis , a former NASA scientist specializing in artificial intelligence research, will lead the Sunnyvale facility, Nissan said today. Toyota Motor Corp. opened its own Silicon Valley outpost in Mountain View in 2012 to work with technology companies there on in-car data and entertainment systems. The company said in January that it’s developing autonomous safety systems to create a virtual “co-pilot” in vehicles to help drivers avoid accidents rather than self-driving cars and trucks.


HootSuite Gets $165 Million to Expand Social-Media Tools

HootSuite Media Inc. , a developer of software used to manage social-networking activity, has raised $165 million from investors in one of the largest venture capital deals in North America this year. Insight Venture Partners led the second financing round, which also included Accel Partners and OMERS Ventures, Vancouver-based HootSuite said in a statement today. Terms of the investment weren’t disclosed. HootSuite plans to use the funds to add more than 100 employees in the next six months and expand to new markets including Asia and Latin America , Chief Executive Officer Ryan Holmes said in an interview. The investment will also help HootSuite build its business without seeking funds via an initial public offering, he said. “I personally would love to take this company to an acquisition or an IPO,” Holmes said. “That being said, my goal right now is building an amazing business. I think this raise takes some of that pressure off.” The company is also in talks to make at least two acquisitions, Holmes said, declining to name the targets. HootSuite has almost doubled in size, to 300 employees, in the past year as it adds tools to help businesses manage their presence on Facebook Inc. (FB) , Twitter Inc. and other social sites. The financing is the third-biggest venture deal for a North American technology company this year, behind the $200 million investment in mobile-device management company AirWatch LLC and online bulletin board Pinterest Inc., according to data compiled by Bloomberg.


Nicolas Cage Hired by China’s BAIC Motor to Promote Senova Cars

Actor Nicolas Cage has signed on as brand ambassador for Chinese automaker BAIC Motor Corp., joining Brad Pitt and Barcelona soccer star Lionel Messi in promoting automobiles in the world’s largest vehicle market. Cage will appear in a television commercial to be aired in May, according to Beijing-based BAIC Motor. The state-owned automaker said it chose the 49-year-old Academy Award winner as its brand spokesman because he’s well-known to Chinese audiences, especially target buyers in their 40s. “Cage will be our ambassador for the Senova D series that will hit the market soon,” Alex Qu, a spokesman for the company, said by phone today. Qu declined to provide more details on the events that Cage will attend in China. BAIC, whose parent traces its roots to making sidecar motorcycles used in the Korean War , is stepping up publicity as it prepares to roll out its Senova brand in the second quarter. The automaker is also targeting sales to senior government officials, with Beijing municipal Party Secretary Guo Jinlong and Mayor Wang Anshun expressing interesting in replacing their Audi sedans, BAIC’s overseas business head Dong Haiyang said. Cage, currently the voice of a caveman in the animated film “The Croods,” won an Oscar for his performance in “Leaving Las Vegas ” and has also starred in movies such as “Drive Angry.” General Motors Co. (GM) signed up Pitt in January to promote its China-made Cadillac XTS, while Chery Automobile Co. hired Barcelona soccer player Messi in 2010 to pitch their Riich brand cars. Zhejiang Geely Holding Group Co.’s Volvo Cars entered an endorsement deal with Houston Rockets player Jeremy Lin last year. BAIC Motor’s parent has said it is seeking an initial share sale for the car unit in Hong Kong and mainland China. To contact Bloomberg News staff for this story: Tian Ying in Beijing at  ytian@bloomberg.net 


Ansell to Purchase BarrierSafe Glove Maker for $615 Million

Ansell Ltd. (ANN) , the maker of Hyflex industrial gloves, agreed to buy BarrierSafe Solutions International for about $615 million, bolstering sales of hand protection products in North America. The acquisition from Odyssey Investment Partners and others is subject to U.S. antitrust clearance and is expected to be completed in the first quarter of 2014, Melbourne-based Ansell said in an e-mailed statement today. The purchase will be funded by a $300 million debt facility and the sale of new shares through a private placement of A$338 million ($308 million) and a share purchase plan of as much as A$100 million. Acquisitions will be a key part of Ansell’s growth strategy, the company said in July 2010. BarrierSafe is the leading North American provider of single-use gloves, Ansell said today. Founded in 1987, it has annual sales of about $290 million and will start adding to Ansell’s earnings next year, excluding transaction costs and one-time integration expenses. “The acquisition of BarrierSafe is a rare opportunity to strengthen the position of Ansell in the core North American single-use glove market and will further enhance our position as the global leader in hand protection solutions,” said Magnus Nicolin, Ansell’s chief executive officer and managing director, in the statement. Standard & Poor’s revised its rating outlook on Ansell to negative from stable following the announcement, meaning the ratings company sees a one-in-three chance that Ansell’s BBB-long-term credit rating may be lowered over the next two years “because of greater financial leverage and potential integration risks following the acquisition,” it said today. Ansell was advised by UBS AG, and Drinker Biddle & Reath LLP and Baker & McKenzie provided legal advice. The announcement was made before the start of trading on the Australian stock exchange. Ansell fell 0.3 percent to A$19.46 yesterday. The shares , which are halted from trading until tomorrow, have jumped 27 percent in 2013, almost double the 15 percent increase of the benchmark S&P/ASX 200 Index.


Aviv REIT Plans to Raise Up to $264 Million in Stock Sale

Aviv REIT Inc. (AVI) , a real estate investment trust that owns skilled-nursing facilities in 29 U.S. states, filed to raise as much as $264 million in an initial public offering. The company plans to sell 13.2 million shares at $18 to $20 each, according to a filing today with the U.S. Securities and Exchange Commission. Aviv, based in Chicago, said it plans to use proceeds from the offering to repay debt and for general corporate purposes, including real estate acquisitions. Aviv owns 258 properties, including $559 million of real estate acquired in the past five years. The skilled-nursing property market is fragmented and “we believe there is a significant consolidation opportunity,” the REIT said in the filing. Aviv’s tenants are operators of skilled-nursing facilities, which provide care for people recovering from illness or surgery as well as long-term residents who need assistance for daily living. The operators are responsible for paying such property expenses as maintenance and repairs. David Smith, a spokesman for Aviv REIT, didn’t immediately return a telephone call seeking comment. The company will have a market capitalization of almost $900 million after the offering, based on the midpoint of the planned pricing range and 47.3 million shares outstanding. Morgan Stanley, Bank of America Corp. and Goldman Sachs Group Inc. are leading the share sale.


ANA Considers Ending AirAsia Tieup on Japan Discount Carrier

ANA Holdings Inc. (9202) , which owns a majority stake in a Japanese discount carrier set up in partnership with AirAsia Bhd. (AIRA) , said it is considering dissolving the tie-up as it aims to improve sales at the unit. Ending the partnership in the venture, AirAsia Japan Co. , is one option and no final decision has been made, Megumi Tezuka, a spokeswoman for Tokyo-based ANA, said by phone today. AirAsia Group chief Executive Officer Tony Fernandes said by e-mail that he would comment “in due course.” ANA would continue operating the business as a low-cost carrier from Narita airport even if the partnership with AirAsia is dissolved, Tezuka said. AirAsia Japan has struggled to fill as many seats as low-cost carriers Peach Aviation Ltd., partly owned by ANA, and Jetstar Japan Co. , which also began flights in Japan last year. “ANA should be able to turn around the carrier,” said Ryota Himeno , an analyst at Barclays Securities Japan Ltd. “They’ve already shown they can be successful with Peach. AirAsia may have discovered the difficulties of flying from Narita,” rather than Japan’s downtown Haneda airport, he said. The Nikkei newspaper reported earlier today that ANA will acquire AirAsia’s stake in AirAsia Japan, without saying where it got the information. 33 Percent Stake ANA, which owns 67 percent of AirAsia Japan, may buy out AirAsia’s stake in the venture, Tezuka said. Narita, Japan-based AirAsia Japan filled 67.6 percent of seats on its domestic flights during Japan’s Golden Week holiday period from April 26 to May 6, the company said in a statement last month. In comparison Peach, which flies from Kansai airport in Osaka , Japan, filled 91.3 percent of its domestic seats, according to the company. Jetstar Japan, which only flies within the country and is also based in Narita, filled 78.8 percent of seats, according to the company. ANA has said that it aims to fill 80 percent or more of its seats on low-cost carrier operations. AirAsia Japan, which began operations in August, has five domestic services and also flies to Seoul and Busan in South Korea. The carrier will add flights to Taiwan next month and is considering another overseas flight before the end of March as it more than doubles its fleet to nine planes from four, Yoshinori Odagiri, chief executive officer of the carrier, said last week. ANA rose 3.6 percent to 204 yen at the close of trading in Tokyo and is up 13 percent this year. AirAsia, based in Sepang Selangor, Malaysia, rose 0.6 percent to close at 3.35 ringgit in Kuala Lumpur.


Oxford Nanopore’s Gene Sequencer Facing Technical Delays

Oxford Nanopore Technologies Ltd., the company in a race with Illumina Inc. (ILMN) and Life Technologies Corp. (LIFE) to develop smaller, cheaper and quicker gene-sequencing devices, is facing technical setbacks. Oxford Nanopore, based in Oxford, England , has had to redesign components of a custom sensor because it wasn’t performing satisfactorily, which has lengthened development and testing periods, according to a post yesterday on the blog Pathogens: Genes and Genomes. The blog is found on the University of Birmingham’s website and written by Nick Loman, a doctor who works as a bioinformatics specialist at the English university. “That put us back about five months, but it was the right thing to do,” Loman quoted Oxford Nanopore Chief Technology Officer Clive Brown as saying at the Advances in Genome Biology and Technology conference last month. The closely held company said more than a year ago that it expected to begin selling its products by the end of 2012. Yesterday, it didn’t deny Loman’s account of his conversation with Brown when contacted by Bloomberg News. “We should have made it clear that was off the record,” Chief Executive Officer Gordon Sanghera said of Loman’s report. “We aren’t giving any further comment on it.” The company has also had to address chemistry in its devices degrading and has been striving to reduce the error rate to 1 percent from 4 percent, according to Loman’s account of his conversation with Brown, which took place during the Feb. 20-23 AGBT meeting in Marco Island, Florida. Technical Risks IP Group Plc (IPO) owns about 20 percent of Oxford Nanopore. Competitor Illumina owns about 15 percent, and the company’s other shareholders include hedge-fund manager Lansdowne Partners LP and Invesco Perpetual, the U.K. group of mutual funds. Numis Securities’ London-based analyst Charles Weston reduced his estimate of Oxford Nanopore’s value by 25 percent to $1.5 billion from $2 billion in a Feb. 20 note to clients, citing technical-development risks. Loman wrote that Brown told him the device wouldn’t be “a year, two years late” and that the company would release data from its early-access program for its products this year. Oxford Nanopore was spun out of Oxford University in 2005 and its technology was initially based on research by Hagan Bayley, a professor of chemical biology at the university. Its instruments use a novel technique known as strand sequencing, in which an entire string of DNA is guided by an enzyme and passes intact through a tiny hole in a cell membrane one-billionth of a meter wide.


Fusion-io Surges on Sales Forecast and Acquisition of NexGen

Fusion-io Inc. (FIO) , a maker of data- storage computers, climbed the most in eight months after forecasting sales that topped analysts’ estimates and announcing the acquisition of NexGen Storage Inc. Revenue in the fourth quarter, which ends in June, will be about $110 million, the Salt Lake City-based company said yesterday in a statement. That’s higher than the average analyst estimate of $107.5 million, according to data compiled by Bloomberg. Fusion-io, whose biggest customers include Apple Inc. (AAPL) and Facebook Inc., is aiming to sell to small and medium-sized businesses with more affordable flash-memory-based services. The company said it purchased NexGen for about $119 million in cash and stock, and is already promoting NexGen’s hybrid storage product on its website. Fusion-io rose 20 percent to $20.03 at 9:46 a.m. in New York, and earlier touched $20.20 for the biggest intraday gain since August. The stock had dropped 27 percent this year through yesterday, compared with an 11 percent gain for the Standard & Poor’s 500 Index. While revenue in the fiscal third quarter fell 7 percent to $87.7 million, it topped the $80.4 million average analyst estimate. Fusion-io said its net loss widened to $20 million, or 21 cents a share, from $4.68 million, or 5 cents, a year earlier. Fusion-io, which employs Apple co-founder Steve Wozniak as chief scientist, sells corporate storage computers based on flash memory , which is faster and more expensive than widely used magnetic disk drives. NexGen, based in Louisville, Colorado , introduced its product in November 2011, a week after saying it raised $10 million from Next World Capital, Grotech Ventures and Access Venture Partners. “This would take years to build out as a stand-alone company,” NexGen Chief Executive Officer John Spiers wrote yesterday in a blog post. “Overnight, those concerns are eliminated.”


Energy Storage Company Ioxus Gets $15 Million in Equity Offering

Ioxus Inc., a closely held maker of energy-storage devices, received $15 million in an equity offering with six unnamed investors, according to a filing. The company is seeking to raise $20 million, according to a document filed with the U.S. Securities and Exchange Commission today. Chief Financial Officer Henry Barber didn’t immediately return a phone message seeking comment. Ioxus, based in Oneonta, New York, is backed by Braemar Energy Ventures, Northwater Capital Management Inc., Aster Capital and Energy Technology Ventures, a joint venture between General Electric Co., ConocoPhillips and NRG Energy Inc. Aster is sponsored by Alstom SA, Schneider Electric SA and Solvay SA, according to its website. The company makes ultracapacitors that rapidly absorb and release high-voltage power, according to its website. The devices can store power generated by solar and wind farms or be paired with batteries in electric vehicles to capture energy from braking.


Afren Sees 29% Growth in Oil Reserves After Expanding in Nigeria

Afren Plc (AFR) , a U.K.-based oil and gas company focused on West Africa and Iraq , will expand reserves by 29 percent after increasing its stake in a Nigerian explorer. Proven and probable reserves will climb to 270 million barrels of oil equivalent from 210 million barrels, the London- based company said today in a statement. Afren agreed to pay about $37 million for 10.4 percent of First Hydrocarbon Nigeria Co., raising its stake to about 55 percent. Oil companies are adding reserves to gain from a four-year increase in prices. Nigeria, Africa’s biggest oil-producing nation, accounted for 92 percent of Afren’s sales in 2011, data compiled by Bloomberg show. The country provides cash to invest in other regions including East Africa and the Middle East. “We are currently drilling very exciting wells in Kurdistan, which are going very well,” Chief Executive Officer Osman Shahenshah said today in an interview. In Africa, the company is adding a “whole suite” of wells in Kenya, Tanzania and Madagascar, he said. Afren advanced as much as 3.2 percent to 152.3 pence in London trading, the highest in a month. The stock was at 152 pence as of 9:59 a.m. local time, extending its gain this year to 16 percent. The company also reported a 66 percent jump in 2012 net income to $202.5 million. Sales rose to a record $1.5 billion, according to a separate statement today. Its exploration program is targeting 650 million barrels of oil and gas resources this year, and output is forecast at 40,000 to 47,000 barrels a day. Afren began production at the Barda Rash field in Iraqi Kurdistan last year. Volumes from the deposit may enable the company to exceed its output guidance in 2013, Galib Virani, an associate director at Afren, said in January. The company’s other producing assets are in Nigeria and Ivory Coast. Elsewhere in Africa, it completed a seismic study in the Seychelles this quarter, the first three-dimensional survey in the archipelago, where the company may hold 2.8 billion barrels in prospective resources, it said.


Weil on Finance: Europe Wakes Up, Taibbi Roars

Happy Monday, View fans. Here is some of what I’ve been reading this morning. In case you forgot, many of Europe’s banks are still a mess Banca Monte dei Paschi di Siena, the world’s oldest bank, said it suspended interest payments on three bond issues. Bloomberg News said the move came “after European authorities demanded bondholders contribute to the restructuring of the bailed out Italian lender.” Can’t help but wonder: Now that the German elections are done, what other European banking scandals and sovereign-debt blowups might surface? There’s no way Europe’s banks have cured themselves. And we’ve been hearing the same refrain all year from European Union officials that nothing much could get done until Angela Merkel was re-elected. The Guardian had an article over the weekend on this same subject: “This year summer has been extended, and Europe's leaders seem to be hanging around even longer before doing anything meaningful. They have been waiting, of course, for the German elections today before springing into action with long wish lists for the reinvention of Europe.” Maybe our grandkids will succeed in closing Fannie and Freddie Nick Timiraos of the Wall Street Journal has some good, pithy lines in this piece on the outlook for Fannie Mae and Freddie Mac. “The firms are proving to be as difficult to shut down as the U.S.-operated Guantanamo Bay prison in Cuba,” he writes. “The real debate boils down to this: Should all Americans continue to have relatively easy access to the pre-payable, 30-year, fixed-rate mortgage?” The answer probably is no, but good luck getting most Americans or members of Congress to agree. How to lose $46 million on a single house And you thought the losses in the U.S. housing crash were bad? The Irish Times had an article over the weekend about a property developer, David Arnold, who lost 34 million euros ($46 million) buying a single residence in the Dublin suburb of Foxrock in July 2006 with a mortgage from the Bank of Ireland. He once planned to demolish the home and turn the grounds into a residential-and-retail complex. But that’s not happening. So now the property is back on the market. Matt Taibbi is back on the vampire-squid beat It seems a glowing Forbes column about Goldman Sachs really ticked him off. Great rant. Funny stuff. What a messy desk says about you It isn't all bad. It could mean you’re creative. But you might die sooner, according to those crazy scientists and their “findings.” The New York Times Magazine had a fun write-up over the weekend. (Jonathan Weil is a Bloomberg View columnist. Follow him on Twitter.)


Whether Public or Private, Information Technology Is Hard To Do Right

A timely reminder that the Department of Health and Human Services is not the only place having information technology woes: This at the NSA -- which is supposed to be so good at what it does that I am half expecting a cheery note from my personal NSA agent reminding me that I should have had whole wheat toast this morning instead of the cheap white bread. Yet the NSA is not even being undone by something difficult, like melding multiple databases into one coherent website. It's having trouble keeping electrical boxes from melting down, which is one of those problems I thought the utilities had solved decades ago. There’s a lesson in this for both sides of the political spectrum. We live our lives immersed in wondrous technology -- especially those of us who spend our workdays on the Internet. Over time, we’ve come to think that anyone can do this sort of thing. But IT is hard. Conservatives who argue that this shows the government can never do IT right should remember that lots of companies can -- and do -- get IT wrong. We think the private sector is so good at it because we see only the winners who got it right. The many projects that went horribly wrong have slipped out of view, and memory. But liberals who have been proclaiming that the health exchange glitches will be fixed eventually because after all, Amazon does this, should remember that the end of every glitchy project is not a product that actually works. Horrifyingly bad launches, into which category I’d say the exchanges now fall, often end when the product is jerked out of production. More than occasionally, the company that made the product goes away. Next time you go onto Amazon, or Netflix or Travelocity, remember these glitches and remind yourself what a hard job these companies are doing amazingly well. You live in an age of technological miracles. You should occasionally let yourself feel the awe that this ought to inspire.


Weil's View on Finance, Afternoon Edition

Good afternoon, dear readers. Here is some of what I've been reading this afternoon while watching the markets go down. Hamptons excess back to being excessive The wealthy enclaves of Long Island, New York, called the Hamptons have fully rebounded -- and then some. Jim Rutenberg of the New York Times got an interview with Joe Farrell, the area's largest real-estate developer, and a tour of his $43 million, 17,000-square-foot home. The going rate to rent it for two weeks: $500,000. Europe's banks preparing for new regulator The European Central Bank is about to become the lead regulator for the euro area's biggest lenders. One of its first assignments will be an asset-quality review focusing on banks' loans. Patrick Jenkins of the Financial Times writes: "Properly done, the exercise could quickly establish the ECB as a tough regulator and help restore investor faith in Europe’s lenders. A succession of botched stress tests over recent years and laggardly recapitalization of troubled lenders have deterred investors from backing the sector." The real test will be if the ECB has the guts to try telling Deutsche Bank what to do. Angela Merkel blames her predecessor (who is from a different political party, of course) for Europe's debt crisis It's election season in Germany, and the country's leader is in full campaign mode. "For example, Greece shouldn't have been allowed into the euro" in 2001, Merkel told supporters at a rally in Rendsberg. “Chancellor [Gerhard] Schroeder accepted Greece in and weakened the Stability Pact, and both decisions were fundamentally wrong, and one of the starting points for our current troubles.” Jack Lew's letter to Congress on the debt ceiling Here we go again. The U.S. will hit the debt ceiling in mid-October, the Treasury secretary said. "Operating the government with no borrowing authority, and with only the cash on hand on a given day, would place the United States in an unacceptable position," Lew said. That's a given. But by now we're so accustomed to this ritual that the markets are assuming there will be a last-second deal, as usual. Haiku of the week (in case you didn't know there were business journalists out there who write things like this) This comes from Seattle Times columnist Jon Talton, who just returned from a two-week vacation: Two weeks off the grid I was differently informed Now, girding myself (Jonathan Weil is a Bloomberg View columnist. Follow him on Twitter.)


Moyes Will Revamp United Scouting, Mirror Says: Soccer Roundup

The following is a roundup of soccer stories from U.K. newspapers, with clickable Internet links. Scouting Changes Manchester United manager David Moyes plans to revamp the club’s scouting system after the Red Devils failed to secure several players during the transfer window, the Daily Mirror reported. Moyes has been conducting a review and plans to bring in his own talent spotters, the newspaper added. The manager wants staff that can identify suitable players and establish what the chances are of signing them, the Mirror said. Carroll Injury West Ham United striker Andy Carroll may not play again this year after injuring his foot during his return to full training, the Daily Telegraph reported. Carroll, who has yet to play for West Ham this season because of a heel injury, had the unrelated setback yesterday and could be sidelined for four months, the newspaper added. Eto’o Quits Cameroon Chelsea striker Samuel Eto’o told his Cameroon teammates that he’s retired from international soccer, the Daily Mirror reported. Eto’o, a four-time African Footballer of the Year, has been in dispute with Cameroon coach Volker Finke over the line-up to play Libya in a World Cup qualifier two days ago and refused to train last week, the newspaper added. Newcastle Target Newcastle United target Bafetimbi Gomis’s proposed move to the Premier League club from Lyon collapsed over his wages, although the French striker said he could still reach an agreement and move in January, the Daily Mail reported. Editor: Alex Duff


Mars Has Too Little Methane to Support Life, Study Finds

Methane, which results from the decay of organic matter, wasn’t detected on Mars, suggesting the planet probably doesn’t support living microbes, according to new readings from the Curiosity rover. The craft has identified no methane on its sensors, and the greatest amount of the gas that may exist in the atmosphere is about 1.3 parts per billion, about six times lower than previous estimates made by earth-bound telescopes and orbiting satellites, according to a paper released today in the journal Science. Earth has about 1,800 parts per billion of methane in the atmosphere, said Chris Webster, a study author and the director of the microdevices lab at NASA ’s Jet Propulsion Laboratory in Pasadena , California. The gas is mostly the result of the decay of organic matter, and a signal of biology. Higher readings of methane on Mars taken from Earth and orbiting satellites raised hopes that microbes might lie beneath the Martian surface, Webster said. “There aren’t significant amounts of methane, so that paints a whole different picture,” Webster said. “We can say that there isn’t significant microbial activity.” That doesn’t rule out the possibility that some microbes don’t produce methane, Webster said. The reading also doesn’t mean that life forms have never existed on Mars, he said. “Curiosity established that it was a habitable world about a billion or less years ago, with water flowing and energy --the right ingredients for simple life forms,” Webster said. Earlier Observations Some previous readings found plumes of methane , which appear to have dispersed, with as much as 45 parts per billion, Webster said. Another report, from an orbiter, said that it detected 10 parts per billion, plus or minus 5. Some scientists have criticizing the methods used to make those readings. “That’s in part because they’re very difficult observations,” Webster said. The sensor on Curiosity is more sensitive and doesn’t have to be read through Earth’s dense atmosphere, he said. Though Curiosity is measuring in only one place, the atmosphere on Mars mixes and turns over every few months, Webster said. So if there were a plume somewhere, it would mix to form the background atmosphere being analyzed by Curiosity.


Nicolas Cage Hired by China’s BAIC Motor to Promote Senova Cars

Actor Nicolas Cage has signed on as brand ambassador for Chinese automaker BAIC Motor Corp., joining Brad Pitt and Barcelona soccer star Lionel Messi in promoting automobiles in the world’s largest vehicle market. Cage will appear in a television commercial to be aired in May, according to Beijing-based BAIC Motor. The state-owned automaker said it chose the 49-year-old Academy Award winner as its brand spokesman because he’s well-known to Chinese audiences, especially target buyers in their 40s. “Cage will be our ambassador for the Senova D series that will hit the market soon,” Alex Qu, a spokesman for the company, said by phone today. Qu declined to provide more details on the events that Cage will attend in China. BAIC, whose parent traces its roots to making sidecar motorcycles used in the Korean War , is stepping up publicity as it prepares to roll out its Senova brand in the second quarter. The automaker is also targeting sales to senior government officials, with Beijing municipal Party Secretary Guo Jinlong and Mayor Wang Anshun expressing interesting in replacing their Audi sedans, BAIC’s overseas business head Dong Haiyang said. Cage, currently the voice of a caveman in the animated film “The Croods,” won an Oscar for his performance in “Leaving Las Vegas ” and has also starred in movies such as “Drive Angry.” General Motors Co. (GM) signed up Pitt in January to promote its China-made Cadillac XTS, while Chery Automobile Co. hired Barcelona soccer player Messi in 2010 to pitch their Riich brand cars. Zhejiang Geely Holding Group Co.’s Volvo Cars entered an endorsement deal with Houston Rockets player Jeremy Lin last year. BAIC Motor’s parent has said it is seeking an initial share sale for the car unit in Hong Kong and mainland China. To contact Bloomberg News staff for this story: Tian Ying in Beijing at  ytian@bloomberg.net 


Whitehaven Coal Targeted by Green Group in Hoax Funding E-mail

Whitehaven Coal Ltd. (WHC) , developing the A$766 million ($803 million) Maules Creek mine, was targeted by a hoax e-mail from a green group campaigning against the project that sent its shares down as much as 8.8 percent. Frontline Action on Coal , an environmental group that opposes the mine in Australia ’s New South Wales state, sent a fake press release that Australia & New Zealand Banking Group Ltd. (ANZ) had terminated a A$1.2 billion loan to Whitehaven on environmental grounds, Jonathan Moylan, a spokesman for the group, said by phone. Whitehaven recovered to trade 0.3 percent lower at A$3.51 at 2:03 p.m. in Sydney after it and ANZ revealed the e-mail was a hoax. The company’s shares plunge d after the Australian Financial Review and the Australian Associated Press reported the fake statement as news. The hoax release, which mirrored the format of ANZ’s official statements, listed a phone number purportedly for Toby Kent, the head of corporate sustainability at ANZ. The number, identical to one on the green group’s website, was answered by a man who identified himself as Moylan when Bloomberg News phoned for comment. Green groups have stepped up campaigns in courts and by occupying port facilities in Australia in an effort to reduce shipments from the world’s largest exporter of coal. Whitehaven last month completed a A$1.2 billion debt facility, underwritten by ANZ, Commonwealth Bank of Australia (CBA) and National Australia Bank Ltd. (NAB) The loan replaced existing facilities and the company plans to spend the funds on Maules Creek and general corporate purposes, it said in a statement. The company in October received approval from the NSW government for Maules Creek, located in the Gunnedah Basin with first coal sales planned in 2014.


Chelsea Market Owner Purchases L.A. Complex Housing IMAX

The firm that owns New York’s One Times Square, where the ball drops each New Year’s Eve, bought a California office-and-studio complex that houses Dick Clark Productions and big-screen movie-theater operator IMAX Corp. (IMX)  Jamestown , which also owns Chelsea Market in New York, bought the 484,840-square-foot (45,043-square-meter) Lantana campus in Santa Monica from Lionstone Group, Atlanta-based Jamestown said today in a statement. The purchase price was about $310 million, according to a person with knowledge of the transaction. “The innovation economy is driving demand for work spaces that offer a vibrant mix of amenities, metropolitan ambience and collaborative, flexible environments,” Jamestown Chief Operating Officer Michael Phillips said in the statement. Lantana is the company’s first purchase in Southern California. The property’s tenants include film-sound company Todd-AO and workout-video producer Beachbody LLC. The complex is 94 percent leased, with “exceptional in-place cash flow,” Jamestown said. The company also owns properties in San Luis Obispo, California , and the San Francisco Bay area.


Brain Made Transparent May Lead to Clearer Understanding

Scientists have developed a technique to make brains transparent, enabling them to see vast networks of neurons and structures for a big picture view of the organ that’s mostly studied in slices. In a report published today in the journal Nature, researchers at Stanford University described the method that replaced fats in the brain of mice with a gel that made the organ transparent. The scientists were able to make images of the brain’s structures and see down to the cells and molecules, according to the paper. The approach also was used on a human brain, the report said. Earlier this month, President Barack Obama announced the BRAIN Initiative, which will spend $100 million beginning in 2014 to map the complex interactions between neurons. The new technique may aid the research, allowing long-range circuits to be more easily mapped, said Michelle Freund, of the National Institute of Mental Health. “The BRAIN Initiative recently announced by the White House is designed to facilitate innovative strategies to better understand the brain,” Freund said in a video released by the institute. “In order to do that, we need new technologies, and clarity is a great example of the type of technologies that will enable future brain studies.” Previous methods required scientists to slice brains into small sections to look at anatomy. This slicing deformed the organs and made it difficult for researchers to see large-scale information running across the slices. Fat Replacement The researchers at Stanford near Palo Alto, California , replaced the fats that support the brain’s cells with a clear material that held all the delicate systems in place. First, the brains were soaked in a chemical bath and then heated, creating a clear gel in place of the fats. That allowed the scientists more access to the whole brains. The team tested the technique, dubbed Clarity, on the brain of a mouse that had been genetically tweaked to produce a fluorescent protein. Under an electron microscope, the scientists were able to see even the spaces between the neurons. What’s more, the researchers tested Clarity on the brain of an autistic person that had been preserved in formaldehyde for six years. There too, scientists saw individual nerve fibers. Since many brains have been preserved in formaldehyde for years, those organs stored in libraries may also be studied.


Harvest Natural Surges on Gabon Offshore Oil Discovery

Harvest Natural Resources Inc. (HNR) , the oil explorer with operations in Venezuela, Indonesia and Oman, climbed the most in more than six months after reporting its second crude discovery off the coast of Gabon. Harvest advanced 11 percent to $10.25 at the close in New York , the most since June 22, and its highest since Nov. 8, 2011. Harvest found oil in the Dussafu Tortue Marin-1 well, the Houston-based company said in a statement today. The well, in 380 feet (116 meters) of water, was dug to a depth of more than 2 miles. Further appraisal is needed to determine the scope of the discovery, including fluid sampling and a sidetrack well, according to the statement. Harvest operates Dussafu and holds a 66.66 percent stake. Panoro Energy ASA (PEN) , owner of 33.33 percent of Dussafu, rose 20 percent in Oslo to close at 3.35 kroner, the biggest one-day gain since it was spun off by Norse Energy Corp ASA (NEC) in June, 2010. Harvest reported its first discovery off Gabon in June 2011.


Northern Trust Profit Rises 1.7% as Market Boosts Assets

Northern Trust Corp. (NTRS) , the third- biggest independent U.S. custody bank, said first-quarter profit rose 1.7 percent as stock-market gains boosted the value of assets the company oversees. Net income increased to $164 million, or 67 cents per diluted share, from $161.2 million, or 66 cents a share, a year earlier, the Chicago-based company said today in a statement. Results missed the 72-cent average estimate of 16 analysts surveyed by Bloomberg. “We have a better macro backdrop with higher market levels this quarter, and we saw good organic growth across all three” of the largest publicly listed custody banks, Brian Bedell, a New York-based analyst at ISI Group Inc., said in an interview before results were released. The custody banks, under pressure to improve profitability hurt by record-low interest rates, are benefiting as global equity markets climb for the second straight year. Northern Trust, led by Chairman and Chief Executive Officer Frederick H. Waddell, followed larger rivals Bank of New York Mellon Corp. (BK) and State Street Corp. (STT) with job cuts in 2012 that, combined with technology initiatives, are designed to increase annual pretax income by $250 million by the end of this year. The Standard & Poor’s 500 Index (SPX) of U.S. stocks returned 14 percent in the 12 months ended March 31, and global stocks, as measured by the MSCI ACWI Index, advanced 11 percent, with reinvested dividends. Rate Impact Low rates hurt custody banks by reducing the return they make on lending and on their own investments. The U.S. Federal Reserve has held its benchmark interest rate at zero to 0.25 percent since December 2008 in an attempt to stimulate borrowing and economic growth. Custody banks keep records, track performance and lend securities for institutional investors, including mutual funds, pension funds and hedge funds. Northern Trust also manages investments for individuals and institutions. Results were announced before the start of regular U.S. trading. Northern Trust gained 7.3 percent this year through yesterday, compared with a 15 percent advance by the Standard & Poor’s 20-company index of asset managers and custody banks. BNY Mellon is scheduled to released earnings tomorrow, followed by State Street on April 19. (Northern Trust is scheduled to hold a conference call for investors at 10 a.m. New York time. The call can be accessed at http://www.northerntrust/financialreleases .)


Nissan Opens Silicon Valley Center for Self-Driving Car Research

Nissan Motor Co. opened a research center in California’s Silicon Valley that the Japanese carmaker plans to make its hub for research on self-driving vehicles and Internet-connected auto technology. The facility will be staffed by more than 60 engineers and technicians within three years, Carla Bailo , Nissan’s senior vice president for North American research and development, said in a phone interview. Work on so-called autonomous vehicle systems will move from an R&D center in Japan to Sunnyvale, she said, declining to provide investment details for the project. “We’re going to focus on this technology really in the heart of where it lies today,” Bailo said in the Feb. 15 interview. “We are going to be shifting work that’s being done in our Nissan Advanced Technical Center in Atsugi, and moving that work over to the heart of the industry.” Automakers and companies including Google Inc. plan new electronics and programming for vehicles with a goal of reducing collisions, fuel use and improving traffic flow. Carlos Ghosn , Nissan’s chief executive officer, told reporters last month in Detroit that regulatory, legal and technological hurdles mean self-driving cars won’t be feasible until about 2020. Google has tested a self-driving Prius in California for years, and the U.S. Defense Department has sponsored autonomous- vehicle research for more than a decade. Nissan’s Sunnyvale center will work with automotive labs at Stanford University, the University of California’s Berkeley and Davis campuses, and Silicon Valley companies, Bailo said. The Yokohama-based carmaker plans to apply technology developed at the new R&D center within a decade, she said. Maarten Sierhuis , a former NASA scientist specializing in artificial intelligence research, will lead the Sunnyvale facility, Nissan said today. Toyota Motor Corp. opened its own Silicon Valley outpost in Mountain View in 2012 to work with technology companies there on in-car data and entertainment systems. The company said in January that it’s developing autonomous safety systems to create a virtual “co-pilot” in vehicles to help drivers avoid accidents rather than self-driving cars and trucks.


India’s Rajan Says RBI Steps Don’t Signal Monetary Policy Shift

India’s steps this month to stabilize its currency should not be seen as indicating a forthcoming major shift in the central bank’s monetary policy stance, the Finance Ministry’s chief economic adviser said. “These measures should be seen not as a precursor to a substantial change in monetary stance, which, thus far, has been on the easing side,” Raghuram Rajan , the top adviser in the finance ministry, said in an interview to Bloomberg India TV today. The steps taken so far should be sufficient, he said. The Reserve Bank of India raised two interest rates and drained liquidity in the past two weeks to steady the rupee, joining nations including Brazil and Indonesia in tightening policy to check currency volatility. The steps were in contrast to the lowering of the benchmark repurchase rate three times this year, as growth weakened to the slowest pace in a decade. “By stabilizing the rupee, we will also enhance the possibility of stronger growth,” said Rajan. “Because the inflationary consequence will be contained and, hopefully over time, that will give RBI room to be more accommodative.” The central bank’s next policy review is due July 30.


Agthia Jumps Most on Record as Quarterly Profit Climbs 38%

Agthia Group PJSC (AGTHIA) jumped the most on record after the Abu Dhabi-based company reported a 38 percent increase in first-quarter profit. The shares soared 11 percent, the most since they were listed in July 2005, to 2.9 dirhams at the close in the emirate. About 1.8 million shares were traded, more than seven times the three-month daily average, according to data compiled by Bloomberg. The benchmark ADX General Index rose 1 percent. Net income rose to 37 million dirhams ($10.1 million), the company that creates, invests, trades and operates companies that are involved in the food and beverage business said in a statement yesterday. “We remain optimistic and expect another successful growth year,” the company said, commenting on the outlook for 2013. Today’s jump pushed Agthia’s gains this year to 33 percent, compared with a 25 percent rise for the benchmark index. The stock’s 14-day relative strength index rose to 73 today. A reading above 70 indicates to some analysts that a security or index is poised to decline. Three analysts recommend investors buy the stock and four advise holding the shares, data compiled by Bloomberg show.


Draghi Says ECB ‘Sharpened’ Communication by Pledging Low Rates

European Central Bank President Mario Draghi said the Governing Council enhanced its communication last week by giving an outlook on interest rates at a time when the euro-region economy remains weak. “The Governing Council sharpened its communication by announcing that it expects the key ECB interest rates to remain at present or lower levels for an extended period of time,” Draghi told the European Parliament’s Economic and Monetary Affairs Committee in Brussels today. “This expectation was based on the overall subdued outlook for inflation extending into the medium term, given the broad-based weakness in the real economy and subdued monetary dynamics.” The ECB and the Bank of England both indicated on July 4 that they will keep benchmark interest rates low for longer than investors were betting, breaking new ground to protect their economies from a U.S.-led surge in bond yields. Draghi’s outlook represented a change in policy for an institution whose officials had until then consistently said that it “never precommits” on monetary policy. “The euro area still faces considerable challenges” and “the economy is still weak,” Draghi said today. Still, recent confidence indicators “have shown some further improvement” and economic activity “should stabilize and recover over the course of the year, although at a subdued pace.” ‘Stable’ Conditions Draghi also said that the euro region faces an improved situation compared with a year earlier. “Clearly financial conditions in the euro area today are more stable and resilient than they were last summer,” he said. “This is partly due to our determined monetary policy actions. Governments and parliaments have also played a key role in the relative return of confidence and stability by undertaking courageous reforms.” In his opening statement to lawmakers, Draghi updated them on progress toward the creation of a program led by the European Investment Bank to stimulate lending to small and medium-sized companies, and on the ECB’s work to establish a banking supervisor. “Reinstating beyond doubt the soundness of the banking system, reintegrating the banking system and delivering a supervisory framework that will prevent a repeat of the past risk buildup” are among the ECB’s objectives, he said. “All core elements should fall into place swiftly to reap the full benefits of a banking union”


ARM Sees Shift to Cheaper Smartphones Boosting Chip Royalty

ARM Holdings Plc (ARM) , the U.K. chip designer whose products power Apple Inc. (AAPL) ’s iPhones, predicts a shift in demand toward lower-priced smartphones will help boost royalty revenue. “There is a shift advantage for ARM as mid-range and entry smartphones drive smartphone growth. We’ve got that dynamic going into next year,” Chief Financial Officer Tim Score said at a conference in Barcelona yesterday. “Those phones generate much more royalty for ARM because they use more silicon.” ARM has had five consecutive years of revenue growth as its designs for chips that control graphics and processing were used in increasingly popular smartphones and tablets. Royalty payments took the spotlight last quarter as they missed analysts’ estimates. ARM shares have fallen almost 9 percent since the earnings report on Oct. 22. “As ARM technology becomes more sophisticated, the royalty percentage goes up,” Score said. “On average, it’s going to very gradually go up.” Global smartphone shipments surged 40 percent last quarter, slowing from 53 percent growth for the previous three-month period, according to researcher IDC. The average selling prices of the devices fell by 13 percent to $317 from a year earlier, because of demand for more affordable handsets. In spite of concerns about slowing growth, especially in China, Score said ARM can continue expanding its customer base and outperforming the semiconductor market in the next five years, driven by cheaper handsets, enterprise networking and micro-controllers. ARM shares fell less than 1 percent to 953.50 pence at 9:16 a.m. in London. New Chief Chief Executive Officer Simon Segars, who succeeded Warren East in July, is confronting a shift in the industry’s growth to low-end smartphones and new entrant Intel Corp. (INTC) , which has begun designing its own chips for mobile phones. ARM, based in Cambridge, England, is also seeking to boost its market share in servers as companies from Facebook Inc. (FB) to Google Inc. (GOOG) look for lower-power machines to drive their data centers. It’s “possible to identify such a market and build a chip specifically for that market, whether it is to handle only storage or only video loading, for example. We can make the chip much more efficient,” Score said at the annual Technology, Media and Telecoms Conference organized by Morgan Stanley. “Everyone knows Intel has a very strong ecosystem around servers, we’re not kidding ourselves,” Score said. “For that reason we see it as further out.”


Mobius Searches for China Stocks to Buy as PBOC Spurs Confidence

Mark Mobius , who oversees $53 billion in emerging markets , said he has confidence in China ’s central bank and is keeping his overweight position in the nation’s equities following a five-day tumble. “We should be confident about what the government is doing and they are cleaning up the system,” Mobius, the executive chairman of Templeton Emerging Markets Group, said in a phone interview today from Monaco. “We are looking to add to Chinese exposure if the price is right. If the price comes down substantially, we would.” The Shanghai Composite Index has tumbled 9.3 percent in the past five days, sending valuations to the lowest level on record, as interbank borrowing costs reached all-time highs. The gauge slipped 0.2 percent today, rebounding from an earlier 5.8 percent plunge , amid speculation the government will take steps to support financial markets. Liquidity risks are controllable and seasonal forces affecting interest rates will fade, a People’s Bank of China official said today. China’s cabinet, led by Premier Li Keqiang , said last week that finance companies must do more to support economic transformation and reduce risks, after administrative measures to crack down on property prices and local government investments were bypassed by so-called shadow banking activities. A gauge of financial shares in the CSI 300 Index (SHSZ300) slumped 12 percent in the past five days amid concern higher interbank borrowing costs will hurt some lenders’ earnings. The seven-day repurchase rate, a gauge of liquidity in the interbank market , rose 68 basis points today to 8 percent, according to a fixing by the National Interbank Funding Center. That is more than double this year’s average of 3.78 percent. No Problems The central bank will closely monitor the money-market rate going forward and keep it at reasonable levels, Ling Tao, deputy director of the Shanghai branch of the PBOC, said at a briefing in Shanghai today. China’s major banks “are not going to have a problem,” Mobius said. The Shanghai Composite index (SHCOMP) is valued at 8 times estimated 12-month earnings, the lowest level on a weekly basis since Bloomberg began compiling the data in 2006. That compares with a multiple of 9.2 for the MSCI Emerging Markets Index. Mobius said in a June 2012 interview that he was adding to Chinese equities after valuations plunged. The Shanghai index fell as much as 15 percent through Dec. 3, before rallying 24 percent to a high in February. “China is very, very key part of the whole EM space and we will continue to be invested in China,” the fund manager said. “Not only in our China fund but also our global fund, Asia fund and in the private equity area.”


Moody’s Raises Outlook of U.K. Banking System on Impairments

Moody’s Investors Service raised its outlook of the U.K. banking system to stable, citing improving profitability and lower impairments after lenders stepped up efforts to clean up balance sheets and raise capital ratios. Moody’s increased the outlook from negative, according to an e-mailed statement today. The ratings firm said that it maintains a negative outlook on the long-term debt and deposit ratings of large U.K. banks, “reflecting its view that the U.K. authorities will continue to take steps to reduce the level of systemic support over the medium term.” British banks have sold assets and eliminated jobs to bolster earnings and meet tougher capital rules aimed to make them more resilient in future financial crises. Barclays Plc (BARC) , Lloyds Banking Group Plc (LLOY) and Royal Bank of Scotland Group Plc have all said that sales of businesses and retained profits mean they won’t need to raise money from equity investors. “We believe that U.K. banks are sufficiently well capitalized to absorb expected losses from both our central and adverse stress scenarios,” the ratings company said. “Moody’s expects profitability to recover from its very low levels, reflecting the improvement in asset quality and already high levels of provisions for conduct-related costs.” Capital Shortfall Moody’s said the upgrade also reflects an “increasingly stable economic outlook.” While U.K. lenders continue to face “the prospect of low medium-term economic growth,” the firm said it doesn’t expect “a deterioration in the operating environment.” Banks will be “well capitalized for the risks they face” once they meet the Prudential Regulation Authority’s buffer requirements, according to the report. The PRA, the Bank of England’s new banking supervisor, last month outlined 13.4 billion pounds ($20 billion) that lenders must raise by the end of 2013 to withstand possible losses on loans, fines and other risks. It had previously found a capital shortfall of 27.1 billion pounds. “In the long term, Moody’s expects U.K. systemic risk will be reduced by higher capital requirements , including significant loss-absorbing and counter-cyclical capital buffers,” it said. “The stable outlook for the system is compatible with the stable outlook on the standalone credit assessment of most U.K. banks.”


Ceragon Jumps Most in 1 Month on Order: Israel Overnight

Ceragon Networks Ltd. (CRNT) , a maker of systems for mobile-phone and Internet providers, rose the most in a month in New York after reporting orders of $3.5 million that tempered investors’ concern about a revenue drop. The shares climbed 2.3 percent to $3.11 yesterday, the most since June 7, to trade 0.3 percent above the stock listed in Tel Aviv. The Bloomberg Israel-US Equity Index gained a second day, led by Caesarstone Sdot Yam Ltd., a maker of quartz countertops. ClickSoftware (CKSW) Technologies Ltd., whose second-biggest investor is George Soros ’s Soros Fund Management LLC, extended a three-day plunge to 15 percent. Ceragon, which got 16 percent of sales from the Asia-Pacific region last year, said yesterday that a “large” mobile operator in the region placed the orders. Ceragon is still down 20 percent since April 8, when Chief Executive Officer Ira Palti cut revenue forecasts. The company has reported losses for nine consecutive quarters and is projected to post the biggest decline in annual sales in more than a decade, according to the average estimate of analysts surveyed by Bloomberg. The new order “shows some momentum,” Joseph Wolf , an analyst at Barclays Plc, who rates the stock the equivalent of hold, said in a telephone interview from Tel Aviv yesterday. “But it’s not an inflection point in the weak demand environment that’s an overhang on Ceragon’s growth rate. It’s a good company with a very tough end market.” The Bloomberg Israel-US gauge rose 0.7 percent to 92.15, the highest price since June 18. The benchmark TA-25 Index (TA-25) advanced for the first time in three days, adding 0.7 percent to 1,214.69 at 10:24 a.m. in Tel Aviv. Ceragon’s Tel Aviv shares this morning slipped 0.8 percent to 11.19 shekels, or $3.09, in volume of 4 percent of the three-month daily average. Caesarstone (CSTE) jumped 6 percent to $29.10, the highest price since May 30. The shares have rallied 82 percent in 2013. ClickSoftware, an Israeli software developer, declined 4.4 percent to $7.11, the lowest price since April 23. Earlier this week, the Petach Tikva, Israel-based company reported profit and revenue that trailed analysts’ projections.


Stratec Drops as Abbott Laboratories Leaves Partnership

Stratec Biomedical AG (SBS) fell to almost a two-year low after analysts at Kepler Cheuvreux said the medical-analysis equipment maker may miss growth opportunities after Abbott Laboratories (ABT) ended a partnership. Stratec dropped as much as 7.4 percent to 26.40 euros, the lowest intraday price since September 2011, and was trading down 6 percent at 1:42 p.m. in Frankfurt, valuing the German company at 314.6 million euros ($410 million). The daily trading volume was more than five times the three-month average. The end of the development program with U.S. drugmaker Abbott is “a meaningful setback of a kind we so far considered highly unlikely,” Fabian Wenner and Maja Pataki, Zurich-based analysts at Kepler Cheuvreux, said today in a research note. “We see higher instrument demand, and Abbott was supposed to be a key driver for 2014 and 2015.” Stratec and Abbott’s diagnostics unit reached an agreement in December 2010 to cooperate on developing an analyzer system. Birkenfeld, Germany-based Stratec said on July 12 that Abbott decided against proceeding because of strategy changes. The German company cut its forecasts for 2013 to a range of 127 million euros to 138 million euros and earnings before interest and taxes at 14 percent to 15.5 percent of sales because of Abbott’s withdrawal. The Kepler Cheuvreux analysts said Stratec’s earlier projections were for revenue of 145 million euros to 150 million euros, with an Ebit margin range of 15.5 percent to 16 percent. They downgraded Stratec’s recommendation to hold from buy and reduced their stock-price estimate to 28.50 euros from 41 euros.


Weil's View on Finance, Afternoon Edition

Good afternoon, dear readers. Here is some of what I've been reading this afternoon while watching the markets go down. Hamptons excess back to being excessive The wealthy enclaves of Long Island, New York, called the Hamptons have fully rebounded -- and then some. Jim Rutenberg of the New York Times got an interview with Joe Farrell, the area's largest real-estate developer, and a tour of his $43 million, 17,000-square-foot home. The going rate to rent it for two weeks: $500,000. Europe's banks preparing for new regulator The European Central Bank is about to become the lead regulator for the euro area's biggest lenders. One of its first assignments will be an asset-quality review focusing on banks' loans. Patrick Jenkins of the Financial Times writes: "Properly done, the exercise could quickly establish the ECB as a tough regulator and help restore investor faith in Europe’s lenders. A succession of botched stress tests over recent years and laggardly recapitalization of troubled lenders have deterred investors from backing the sector." The real test will be if the ECB has the guts to try telling Deutsche Bank what to do. Angela Merkel blames her predecessor (who is from a different political party, of course) for Europe's debt crisis It's election season in Germany, and the country's leader is in full campaign mode. "For example, Greece shouldn't have been allowed into the euro" in 2001, Merkel told supporters at a rally in Rendsberg. “Chancellor [Gerhard] Schroeder accepted Greece in and weakened the Stability Pact, and both decisions were fundamentally wrong, and one of the starting points for our current troubles.” Jack Lew's letter to Congress on the debt ceiling Here we go again. The U.S. will hit the debt ceiling in mid-October, the Treasury secretary said. "Operating the government with no borrowing authority, and with only the cash on hand on a given day, would place the United States in an unacceptable position," Lew said. That's a given. But by now we're so accustomed to this ritual that the markets are assuming there will be a last-second deal, as usual. Haiku of the week (in case you didn't know there were business journalists out there who write things like this) This comes from Seattle Times columnist Jon Talton, who just returned from a two-week vacation: Two weeks off the grid I was differently informed Now, girding myself (Jonathan Weil is a Bloomberg View columnist. Follow him on Twitter.)


HootSuite Gets $165 Million to Expand Social-Media Tools

HootSuite Media Inc. , a developer of software used to manage social-networking activity, has raised $165 million from investors in one of the largest venture capital deals in North America this year. Insight Venture Partners led the second financing round, which also included Accel Partners and OMERS Ventures, Vancouver-based HootSuite said in a statement today. Terms of the investment weren’t disclosed. HootSuite plans to use the funds to add more than 100 employees in the next six months and expand to new markets including Asia and Latin America , Chief Executive Officer Ryan Holmes said in an interview. The investment will also help HootSuite build its business without seeking funds via an initial public offering, he said. “I personally would love to take this company to an acquisition or an IPO,” Holmes said. “That being said, my goal right now is building an amazing business. I think this raise takes some of that pressure off.” The company is also in talks to make at least two acquisitions, Holmes said, declining to name the targets. HootSuite has almost doubled in size, to 300 employees, in the past year as it adds tools to help businesses manage their presence on Facebook Inc. (FB) , Twitter Inc. and other social sites. The financing is the third-biggest venture deal for a North American technology company this year, behind the $200 million investment in mobile-device management company AirWatch LLC and online bulletin board Pinterest Inc., according to data compiled by Bloomberg.


Chelsea Market Owner Purchases L.A. Complex Housing IMAX

The firm that owns New York’s One Times Square, where the ball drops each New Year’s Eve, bought a California office-and-studio complex that houses Dick Clark Productions and big-screen movie-theater operator IMAX Corp. (IMX)  Jamestown , which also owns Chelsea Market in New York, bought the 484,840-square-foot (45,043-square-meter) Lantana campus in Santa Monica from Lionstone Group, Atlanta-based Jamestown said today in a statement. The purchase price was about $310 million, according to a person with knowledge of the transaction. “The innovation economy is driving demand for work spaces that offer a vibrant mix of amenities, metropolitan ambience and collaborative, flexible environments,” Jamestown Chief Operating Officer Michael Phillips said in the statement. Lantana is the company’s first purchase in Southern California. The property’s tenants include film-sound company Todd-AO and workout-video producer Beachbody LLC. The complex is 94 percent leased, with “exceptional in-place cash flow,” Jamestown said. The company also owns properties in San Luis Obispo, California , and the San Francisco Bay area.


Zynga Rises on Speculation Yahoo Might Consider Buyout

Zynga Inc., the biggest maker of online social games, climbed to the highest price since July on speculation that Yahoo! Inc. might consider buying the company. The shares rose 10 percent to $3.91 at the close in New York. The stock has advanced 67 percent this year, compared with a 9.4 percent gain for the Russell 1000 Index. Yahoo may consider buying Zynga as Chief Executive Officer Marissa Mayer seeks acquisitions to bolster the U.S. Web portal’s mobile and social capabilities, Blake Harper, an analyst at Wunderlich Securities Inc., wrote in a research report today. Since Mayer arrived, Yahoo has ceded share in its core business of display advertising amid mounting competition from Google Inc. and Facebook Inc. With a market value of about $3 billion, Zynga might be an attractive target if Mayer opts for deals valued at more than $1 billion, Harper said. Yahoo might also look at purchasing food- review sites Yelp Inc. (YELP) and OpenTable Inc. (OPEN) , or Millennial Media Inc. (MM) , which offers mobile advertising, Harper said. Representatives from Sunnyvale, California-based Yahoo and Zynga, based in San Francisco, declined to comment. Yelp, OpenTable and Millennial Media also declined to comment.


Stripper Site OK’d for Dancers’ Notice of Lawsuit Dollars

When companies resolve class-action suits, they typically tell people about the settlement by postcard or in publications such as the Wall Street Journal, USA Today or the New York Times. Stripper Web and Xtreme magazine weren’t the usual vehicles for delivering the news -- until now. A federal judge in New York approved the parties’ plan to use those publications to notify 1,245 current and former exotic dancers of an $8 million settlement in a wage-and-hour class action against Manhattan ’s Penthouse Executive Club. The agreement would resolve a group of suits, first filed in 2009, over claims the Penthouse Club shortchanged dancers by classifying them as independent contractors rather than employees. The club also owes dancers minimum wage and overtime pay, the dancers claimed. U.S. District Judge Kimba Wood, who preliminarily approved the deal April 29, signed off on a plan by the parties May 9 to use Stripper Web, an online forum that includes reviews of strip clubs, and Xtreme, a “free guide to local adult entertainment.” Wood’s one-page order was made public May 10. The Stripper Web and Xtreme notices will supplement settlement alerts mailed to the last known address of each dancer in the class, according to court papers. Stripper Web didn’t respond to an e-mail seeking comment on the plan. A phone call to Xtreme wasn’t returned. Direct Mail Campaigns to contact class members can include direct mail and publication in magazines including People and National Geographic, Todd Hilsee , whose Hilsee Group LLC offers consulting services on class notification, said in a phone interview. Class members are also targeted on social media and websites. A successful notice campaign can reach from 70 percent to 95 percent of class members, he said. “There’s no ‘usual’ anymore,” Hilsee said. Hilsee said he hadn’t heard of Stripper Web or Xtreme. “Major publications, where there are large audiences, are going to reach a large swath of the target audience,” Hilsee said. Wood will hold a hearing on the fairness of the settlement on Sept. 18. The case is In re Penthouse Executive Club Compensation Litigation, 10-cv-01145, U.S. District Court, Southern District of New York (Manhattan).


Pfizer to Split Off Rest of Zoetis Animal Health Business

Pfizer Inc. (PFE) , the world’s largest drugmaker, is offering investors a share exchange to reduce its 80.2 percent stake in Zoetis Inc. (ZTS) , the animal-health business that the company spun off almost four months ago. Pfizer shareholders will have the option of swapping their shares for Zoetis stock in a tax-free transaction, New York-based Pfizer said in a statement today. If the exchange is fully subscribed, Pfizer will no longer have a controlling interest, leaving Madison, New Jersey-based Zoetis fully independent. The exchange would end the latest stage of Pfizer Chief Executive Officer Ian Read’s plans to slim down the company and concentrate on producing new drugs. Pfizer agreed in April 2012 to sell its baby food unit to Nestle SA (NESN) , the world’s biggest food maker, for $11.9 billion. Zoetis shares have risen 29 percent since trading started in February after an initial public offering. “The proceeds from the IPO plus the Pfizer share reduction from this exchange will more than offset the lost income from Zoetis,” Mark Schoenebaum , an analyst with International Strategy & Investment Group LLC in New York said in a note to clients. The timing was sooner than expected, and a positive for Pfizer because it should add to earnings next year, he said. The exchange also advances Read’s effort of cutting Pfizer’s shares outstanding and boosting earnings per share through moves including stock buybacks. Better Position “Given the strong demand in the IPO and a favorable market environment, we concluded that now is the appropriate time to distribute our remaining stake in Zoetis,” Read said in today’s statement. The transaction “better positions Pfizer to focus on our core business as an innovative biopharmaceutical company.” Pfizer shares rose 1.8 percent to $29.30 at the close in New York. Zoetis climbed 1.5 percent to $33.55. The Pfizer shares will be exchanged at a 7 percent discount subject to an upper limit of 0.9898 shares of Zoetis stock per Pfizer share. If the limit isn’t in effect, $100 in Pfizer shares will be worth $107.52 of Zoetis, Pfizer said. The exchange offer is scheduled to end on June 19 when the exact ratio will be announced, Pfizer said in the statement. JPMorgan Chase & Co., Bank of America Corp., Goldman Sachs Group Inc. and Morgan Stanley will manage the deal. Skadden, Arps, Slate, Meagher & Flom LLP acted as legal adviser.


BMC Software Drops After Profit Forecast Misses: Houston Mover

BMC Software Inc. (BMC) , a maker of programs that help companies update servers and personal computers, fell after forecasting fiscal 2013 profit that missed analysts’ estimates, a sign it’s struggling to clinch big contracts. The shares declined 6.2 percent to $41.71 at the close in New York , the biggest decline since October 2011. The Houston- based company said yesterday that profit excluding some items will be $3.35 to $3.45 a share for fiscal 2013, which ends in March. That’s less than BMC’s prior forecast of $3.49 to $3.59 and trails analysts’ average $3.56 projection, according to data compiled by Bloomberg. Activist shareholder Elliott Associates LP pressed BMC to consider a sale last year, unsettling customers and making it harder for the company to clinch deals, BMC Chief Executive Officer Bob Beauchamp told analysts on a conference call yesterday. The company had two large contract renewals pushed to the March quarter from the December period, crimping results. “BMC underperformed across all metrics for the quarter,” and deals in the mainframe and server-software businesses didn’t get closed, Abhey Lamba, an analyst at Mizuho Securities USA Inc., wrote in a research report. “Despite having completed its accelerated share repurchase program, the company lowered its EPS target for the year, which is clearly disappointing.” ‘Growth Areas’ The company said it needs to be “more consistent and disciplined” in the way it secures big contracts, according to the statement. Management is “scrutinizing the entire company to improve our operational discipline,” Beauchamp said. BMC added board members and initiated a $1 billion share buyback to assuage Elliott’s complaints that it wasn’t doing enough to maximize shareholder value. Competitors are “aggressively” capitalizing on the pressure by Elliott, prompting many customers to seek personal visits from BMC management before completing purchases, Beauchamp said. Beauchamp said during a conference call with analysts that he’s undertaking an operational review aimed at directing spending toward better performing products to increase earnings, cash flow and operating margin amid low revenue growth. The company is trying to improve profitability even as revenue this year is projected to grow in the low single digits. Web-delivered software, cloud-computing management products and tools for managing mobile devices are selling well and will get a greater share of investment compared with underperforming products, he said. “We’re going to find a way to move funds to these growth areas,” Beauchamp said. “It’s just moving faster.” BMC kicked off the review after the end of the quarter on Dec. 31, according to Mark Stouse, a spokesman.


Weil on Finance: Europe Wakes Up, Taibbi Roars

Happy Monday, View fans. Here is some of what I’ve been reading this morning. In case you forgot, many of Europe’s banks are still a mess Banca Monte dei Paschi di Siena, the world’s oldest bank, said it suspended interest payments on three bond issues. Bloomberg News said the move came “after European authorities demanded bondholders contribute to the restructuring of the bailed out Italian lender.” Can’t help but wonder: Now that the German elections are done, what other European banking scandals and sovereign-debt blowups might surface? There’s no way Europe’s banks have cured themselves. And we’ve been hearing the same refrain all year from European Union officials that nothing much could get done until Angela Merkel was re-elected. The Guardian had an article over the weekend on this same subject: “This year summer has been extended, and Europe's leaders seem to be hanging around even longer before doing anything meaningful. They have been waiting, of course, for the German elections today before springing into action with long wish lists for the reinvention of Europe.” Maybe our grandkids will succeed in closing Fannie and Freddie Nick Timiraos of the Wall Street Journal has some good, pithy lines in this piece on the outlook for Fannie Mae and Freddie Mac. “The firms are proving to be as difficult to shut down as the U.S.-operated Guantanamo Bay prison in Cuba,” he writes. “The real debate boils down to this: Should all Americans continue to have relatively easy access to the pre-payable, 30-year, fixed-rate mortgage?” The answer probably is no, but good luck getting most Americans or members of Congress to agree. How to lose $46 million on a single house And you thought the losses in the U.S. housing crash were bad? The Irish Times had an article over the weekend about a property developer, David Arnold, who lost 34 million euros ($46 million) buying a single residence in the Dublin suburb of Foxrock in July 2006 with a mortgage from the Bank of Ireland. He once planned to demolish the home and turn the grounds into a residential-and-retail complex. But that’s not happening. So now the property is back on the market. Matt Taibbi is back on the vampire-squid beat It seems a glowing Forbes column about Goldman Sachs really ticked him off. Great rant. Funny stuff. What a messy desk says about you It isn't all bad. It could mean you’re creative. But you might die sooner, according to those crazy scientists and their “findings.” The New York Times Magazine had a fun write-up over the weekend. (Jonathan Weil is a Bloomberg View columnist. Follow him on Twitter.)


Former Consumer Bureau Deputy Date Starts Consulting Firm

Raj Date, the former deputy director of the U.S. Consumer Financial Protection Bureau, has formed a consulting firm to focus on the industry his agency regulated. Fenway Summer LLC, Date’s new Washington-based firm, will advise clients on business strategy, work on mergers and acquisitions, and help new firms that “have little more than an idea to start with,” Date said in an interview. In his new role he won’t lobby for clients or advise them on government relations, he said. “I am not especially interested in technical compliance work and government focus,” he said. “I think it’s important work, but it’s not where my particular background or expertise lies.” Date, 42, was the first deputy director of the bureau, which was created by the 2010 Dodd-Frank Act that overhauled U.S. financial regulation. He helped Democratic Senator Elizabeth Warren of Massachusetts , then an adviser to President Barack Obama , set up the agency 2010 and 2011. He briefly led the bureau before Obama named Richard Cordray director in January 2012. Date is also an external adviser to McKinsey & Co., one of his former employers, according to a person briefed on his work. He declined to speak about that engagement. McKinsey spokeswoman Yolande Daeninck did not respond to a request for comment. A former consumer finance executive at Capital One Financial Corp. (COF) and Deutsche Bank AG, Date said that changes to bank business models and the emergence of new technologies offer chances to recast some fundamental products, such as the checking account. New businesses “could make the products vastly cheaper to deliver and offer something to medium-income households,” Date said. The name of the new firm is a Massachusetts-oriented allusion to a public-policy group Date founded to participate in the debate over Dodd-Frank, the Cambridge Winter Center for Financial Institutions Policy.


Kony Solutions Weighs Public Offer as Early as This Year

Kony Solutions Inc. , a provider of mobile-application platforms, has talked to banks including Goldman Sachs Group Inc. (GS) and JPMorgan Chase & Co. (JPM) about selling shares in a public offering as early as this year. The company, which has its headquarters in Orlando , Florida , as well as in London , plans to select bankers this year after introducing a service that will provide easier access to its app platforms, Chief Executive Officer Raj Koneru said in an interview yesterday. The share sale will take place this year or in 2014, he said. “We are talking with a lot of bankers,” Koneru said. The company hasn’t decided how much stock to sell or the amount of funds to raise, he said. Kony, which has yet to make a profit, expects to record $80 million to $100 million in revenue this year, about double its 2012 sales, Koneru said. An initial public offering could raise at least $120 million for Kony, valuing the company at more than $700 million, Tom Taulli , an analyst who covers IPOs, said in an interview. “Enterprise is really hot right now,” Taulli said. “Right place at the right time -- that always helps with an IPO. I suspect they’ll do well.” Spending on app-development tools for multiple mobile operating systems will grow at an average annual rate of 39 percent to reach $8.2 billion in 2016, according to Smith’s Point Analytics LLC. Founded in 2007, Kony sells annual subscriptions for its app platform-service, which allows developers to customize products to their own specifications. The company has raised about $35 million from Georgian Partners and Insight Venture Partners and may use proceeds from an IPO to improve its software and for research and development, Koneru said.


Fusion-io Surges on Sales Forecast and Acquisition of NexGen

Fusion-io Inc. (FIO) , a maker of data- storage computers, climbed the most in eight months after forecasting sales that topped analysts’ estimates and announcing the acquisition of NexGen Storage Inc. Revenue in the fourth quarter, which ends in June, will be about $110 million, the Salt Lake City-based company said yesterday in a statement. That’s higher than the average analyst estimate of $107.5 million, according to data compiled by Bloomberg. Fusion-io, whose biggest customers include Apple Inc. (AAPL) and Facebook Inc., is aiming to sell to small and medium-sized businesses with more affordable flash-memory-based services. The company said it purchased NexGen for about $119 million in cash and stock, and is already promoting NexGen’s hybrid storage product on its website. Fusion-io rose 20 percent to $20.03 at 9:46 a.m. in New York, and earlier touched $20.20 for the biggest intraday gain since August. The stock had dropped 27 percent this year through yesterday, compared with an 11 percent gain for the Standard & Poor’s 500 Index. While revenue in the fiscal third quarter fell 7 percent to $87.7 million, it topped the $80.4 million average analyst estimate. Fusion-io said its net loss widened to $20 million, or 21 cents a share, from $4.68 million, or 5 cents, a year earlier. Fusion-io, which employs Apple co-founder Steve Wozniak as chief scientist, sells corporate storage computers based on flash memory , which is faster and more expensive than widely used magnetic disk drives. NexGen, based in Louisville, Colorado , introduced its product in November 2011, a week after saying it raised $10 million from Next World Capital, Grotech Ventures and Access Venture Partners. “This would take years to build out as a stand-alone company,” NexGen Chief Executive Officer John Spiers wrote yesterday in a blog post. “Overnight, those concerns are eliminated.”


Kony Solutions Weighs Public Offer as Early as This Year

Kony Solutions Inc. , a provider of mobile-application platforms, has talked to banks including Goldman Sachs Group Inc. (GS) and JPMorgan Chase & Co. (JPM) about selling shares in a public offering as early as this year. The company, which has its headquarters in Orlando , Florida , as well as in London , plans to select bankers this year after introducing a service that will provide easier access to its app platforms, Chief Executive Officer Raj Koneru said in an interview yesterday. The share sale will take place this year or in 2014, he said. “We are talking with a lot of bankers,” Koneru said. The company hasn’t decided how much stock to sell or the amount of funds to raise, he said. Kony, which has yet to make a profit, expects to record $80 million to $100 million in revenue this year, about double its 2012 sales, Koneru said. An initial public offering could raise at least $120 million for Kony, valuing the company at more than $700 million, Tom Taulli , an analyst who covers IPOs, said in an interview. “Enterprise is really hot right now,” Taulli said. “Right place at the right time -- that always helps with an IPO. I suspect they’ll do well.” Spending on app-development tools for multiple mobile operating systems will grow at an average annual rate of 39 percent to reach $8.2 billion in 2016, according to Smith’s Point Analytics LLC. Founded in 2007, Kony sells annual subscriptions for its app platform-service, which allows developers to customize products to their own specifications. The company has raised about $35 million from Georgian Partners and Insight Venture Partners and may use proceeds from an IPO to improve its software and for research and development, Koneru said.


LinkedIn Sales Forecast Lags Estimates Amid Shift to Mobile

May 2 (Bloomberg) -- LinkedIn Corp. (LNKD) , the biggest online professional-networking service, forecast second-quarter sales that trailed analysts’ estimates, raising concern that efforts to boost mobile advertising will be slow to kick in. Revenue in the current period will be $342 million to $347 million, the Mountain View , California-based company said today in a statement. That missed the $359.7 million average analyst projection, according to data compiled by Bloomberg. LinkedIn, founded a decade ago to connect recruiters and job seekers, is adding features and content to appeal to people who use the site from smartphones. The company revamped its mobile app and agreed last month to buy Pulse, a service that lets users access news on handheld devices. Still, mobile-ad revenue may not be rising on pace with the transition to wireless computing, said Aaron Kessler, an analyst at Raymond James & Associates Inc. “It could be the desktop-to-mobile shift, and they don’t have as much advertising on mobile,” said Kessler, who has the equivalent of a hold rating on the stock and is based in San Francisco. LinkedIn’s ad business came in weaker than projected, Kessler said. LinkedIn shares dropped as much as 12 percent in extended trading. They had gained 3.5 percent to $201.67 at the close in New York. The stock has more than quadrupled since the company’s initial public offering almost two years ago. Estimate Shortfall The results come a day after Facebook Inc., the biggest social-networking site, reported revenue that beat analysts’ predictions, with its mobile business making up about 30 percent of ad sales. LinkedIn and Facebook, like many popular Web applications, are testing ways to make money from mobile ads. LinkedIn’s full-year sales forecast also trailed estimates. The company said 2013 revenue will be $1.43 billion to $1.46 billion, compared with the average analyst estimate of $1.5 billion, according to data compiled by Bloomberg. LinkedIn didn’t break out mobile revenue in its release. Total sales in the first quarter climbed 72 percent to $324.7 million from $188.5 million a year earlier, LinkedIn said in the statement. Analysts on average had estimated $318.2 million. Net income more than quadrupled to $22.6 million, or 20 cents a share, from $5 million, or 4 cents, a year ago. LinkedIn’s biggest revenue generator is still its talent-solutions product, aimed at helping recruiters fill jobs. Sales in that unit jumped 80 percent to $184.3 million. Ad revenue increased 56 percent to $74.8 million, while premium-subscription sales rose 73 percent to $65.6 million. Membership increased 7.9 percent from the fourth quarter to 218 million. That number has about doubled since the IPO.


NewStar Financial Said to Work With Credit Suisse on Sale

NewStar Financial Inc. (NEWS) , the business lender backed by private-equity firms Corsair Capital LLC and Capital Z Partners Management LLC, is exploring a sale, according to people with knowledge of the matter. NewStar is working with Credit Suisse Group AG (CSGN) to solicit offers, said one of the people, who asked not to be identified because the process is confidential. Boston-based NewStar rose 13 percent to $17.76 today, its highest since February 2007, giving the company a market value of about $863 million. Specialty lenders like NewStar, which makes loans to health-care, manufacturing and energy companies, typically charge higher interest rates, making them appealing targets for U.S. banks looking to increase profits. CapitalSource Inc. (CSE) , which also lends to businesses, agreed in July to sell to PacWest Bancorp (PACW) for $2.3 billion. “This company is a prime takeover target,” said Sameer Gokhale, an analyst who covers NewStar for Janney Montgomery Scott LLC in Philadelphia. NewStar will appeal to regional banks interested in expanding in commercial lending, said Gokhale, who has a “neutral” rating on the stock and a price target of $17. Logical buyers include Jacksonville, Florida-based EverBank Financial Corp. (EVER) , Miami Lakes, Florida-based BankUnited Inc. (BKU) , and Wayzata, Minnesota-based TCF Financial Corp. (TCB) , he said. Early Backers Robert Brown , a spokesman for NewStar, declined to comment, as did Credit Suisse spokesman Jack Grone and Robert Siegfried at Corsair. Spokesmen at Capital Z and the three regional banks didn’t immediately reply to requests for comment. NewStar, with offices in nine U.S. cities, makes loans and leases of $5 million to $20 million to companies with annual sales of $25 million to $500 million, according to its annual report. The company had assets of $2.28 billion at the end of June, data compiled by Bloomberg show. Along with other backers, Corsair and Capital Z, which each own close to 20 percent of NewStar, invested $210 million in equity and $450 million in debt to help start the company in 2004, according to a press release at the time. PacWest offered about 1.47 times CapitalSource’s book value, in the cash-and-stock takeover that was announced in July. At that multiple, NewStar would be worth about $18.21 a share, based on its book value of $12.39 a share at the end of June, data compiled by Bloomberg show.


HootSuite Gets $165 Million to Expand Social-Media Tools

HootSuite Media Inc. , a developer of software used to manage social-networking activity, has raised $165 million from investors in one of the largest venture capital deals in North America this year. Insight Venture Partners led the second financing round, which also included Accel Partners and OMERS Ventures, Vancouver-based HootSuite said in a statement today. Terms of the investment weren’t disclosed. HootSuite plans to use the funds to add more than 100 employees in the next six months and expand to new markets including Asia and Latin America , Chief Executive Officer Ryan Holmes said in an interview. The investment will also help HootSuite build its business without seeking funds via an initial public offering, he said. “I personally would love to take this company to an acquisition or an IPO,” Holmes said. “That being said, my goal right now is building an amazing business. I think this raise takes some of that pressure off.” The company is also in talks to make at least two acquisitions, Holmes said, declining to name the targets. HootSuite has almost doubled in size, to 300 employees, in the past year as it adds tools to help businesses manage their presence on Facebook Inc. (FB) , Twitter Inc. and other social sites. The financing is the third-biggest venture deal for a North American technology company this year, behind the $200 million investment in mobile-device management company AirWatch LLC and online bulletin board Pinterest Inc., according to data compiled by Bloomberg.


Valeo Stock Rises as Revenue Gain Sustains 2013 Forecast

Valeo SA (FR) , France ’s second-biggest auto-parts maker, rose the most in more than five weeks after reporting third-quarter sales growth in China and North America and signaling that the European car market is stabilizing. Valeo jumped as much as 3.3 percent, the biggest intraday increase since Sept. 10, and was trading up 2.6 percent at 69.50 euros at 10:15 a.m. in Paris. Volume was 51 percent of the three-month daily average after a little more than an hour of trading. The stock, which has been at the highest price since January 2000 for the past week, has gained 85 percent this year, valuing the manufacturer at 5.53 billion euros ($7.56 billion). The French manufacturer reiterated a full-year target yesterday of a “slight increase” in operating profit as a proportion of revenue, “assuming stabilized market conditions in Europe .” Industrywide car sales in the region are set to shrink for a sixth consecutive year following an 18-month recession in the euro region that ended in the second quarter. Third-quarter sales at Paris-based Valeo increased 2.2 percent from a year earlier to 2.91 billion euros, the manufacturer said yesterday in a statement. Excluding acquisitions, disposals and currency effects, sales to carmakers rose 25 percent in North America , 13 percent in Asia and 10 percent in both Europe and South America , exceeding competitors’ growth in all markets, Valeo said. ‘Fastest Growing’ “This leaves Valeo the fastest-growing European auto supplier and highlights its credentials as an attractive growth stock,” Philip Watkins, a London-based analyst at Citigroup Inc., said today in a report to clients. Valeo, whose products span windshield wipers, headlights and stop-start ignition systems, is focusing on technology promoting vehicle safety, comfort and pollution reduction to increase profitability. Chief Executive Officer Jacques Aschenbroich pledged in March 2011 to boost annual revenue to 14 billion euros by 2015, propelled by sales of fuel-saving components. The third-quarter figures “reflect the gradual entry into production of the high order intake recorded by the group over the last three years and the strength of Valeo’s growth model,” with gains in both original-equipment and spare-parts sales demonstrating “balanced growth,” Aschenbroich said in the statement yesterday. Valeo is sticking to predictions that Europe’s car market will shrink 2 percent to 3 percent this year, with worldwide auto sales rising 2 percent, Aschenbroich told journalists and analysts on a conference call yesterday.


Facebook Appoints Mike Schroepfer to CTO Position

Facebook Inc. (FB) appointed Mike Schroepfer as chief technology officer, promoting an executive steeped in the software and hardware that runs the world’s largest social network. Schroepfer, who joined Facebook in 2008, has been involved in many of Facebook’s recent upgrades as vice president of engineering. He replaces Bret Taylor, who stepped down last year and left the company following its initial public offering. The promotion helps fill Facebook’s executive ranks as it seeks to ramp up sales and user growth amid pressure from investors to generate more money from its more than 1 billion users. The company’s shares have declined 30 percent since the IPO in May. “Mike Schroepfer’s new designation as Facebook’s CTO reflects the unique and important role he plays across the company,” Tucker Bounds, a spokesman for Menlo Park, California-based Facebook , wrote in an e-mailed statement. Prior to joining Facebook, Schroepfer was vice president of Mozilla Corp., the company behind the Firefox Web browser. He was also at Sun Microsystems, now part of Oracle Corp. (ORCL) 


NBA Unveils Advanced Statistical Player Analysis on Its Website

The National Basketball Association added advanced statistical categories to its website , joining a movement toward increased use of sports data to help teams, fans and media members analyze player performance. The website now includes every box score in NBA history, shooting charts that help visualize offensive and defensive performances and statistics that break down player attributes such as efficiency and how clutch they are, the league said in a statement. “Sports fans have a passion for statistics because it brings them closer to the game, their favorite teams and players,” said Steve Peck , senior vice president of global strategic initiatives at SAP AG , which partnered with the NBA to power the website. The NBA’s entire official statistics history now will be offered free after previously being made available only to teams and league officials. Content found on the site can be shared on social media platforms Twitter, Facebook and Google+. Using advanced data to analyze player performance has become common in Major League Baseball since Bill James used statistics to help make the Boston Red Sox World Series champions in 2004. The movement was popularized by Michael Lewis ’s book “Moneyball,” which chronicled Oakland Athletics General Manager Billy Beane ’s use of statistics to build his roster. Lewis is a Bloomberg News columnist.


Nissan Opens Silicon Valley Center for Self-Driving Car Research

Nissan Motor Co. opened a research center in California’s Silicon Valley that the Japanese carmaker plans to make its hub for research on self-driving vehicles and Internet-connected auto technology. The facility will be staffed by more than 60 engineers and technicians within three years, Carla Bailo , Nissan’s senior vice president for North American research and development, said in a phone interview. Work on so-called autonomous vehicle systems will move from an R&D center in Japan to Sunnyvale, she said, declining to provide investment details for the project. “We’re going to focus on this technology really in the heart of where it lies today,” Bailo said in the Feb. 15 interview. “We are going to be shifting work that’s being done in our Nissan Advanced Technical Center in Atsugi, and moving that work over to the heart of the industry.” Automakers and companies including Google Inc. plan new electronics and programming for vehicles with a goal of reducing collisions, fuel use and improving traffic flow. Carlos Ghosn , Nissan’s chief executive officer, told reporters last month in Detroit that regulatory, legal and technological hurdles mean self-driving cars won’t be feasible until about 2020. Google has tested a self-driving Prius in California for years, and the U.S. Defense Department has sponsored autonomous- vehicle research for more than a decade. Nissan’s Sunnyvale center will work with automotive labs at Stanford University, the University of California’s Berkeley and Davis campuses, and Silicon Valley companies, Bailo said. The Yokohama-based carmaker plans to apply technology developed at the new R&D center within a decade, she said. Maarten Sierhuis , a former NASA scientist specializing in artificial intelligence research, will lead the Sunnyvale facility, Nissan said today. Toyota Motor Corp. opened its own Silicon Valley outpost in Mountain View in 2012 to work with technology companies there on in-car data and entertainment systems. The company said in January that it’s developing autonomous safety systems to create a virtual “co-pilot” in vehicles to help drivers avoid accidents rather than self-driving cars and trucks.


BMC Software Drops After Profit Forecast Misses: Houston Mover

BMC Software Inc. (BMC) , a maker of programs that help companies update servers and personal computers, fell after forecasting fiscal 2013 profit that missed analysts’ estimates, a sign it’s struggling to clinch big contracts. The shares declined 6.2 percent to $41.71 at the close in New York , the biggest decline since October 2011. The Houston- based company said yesterday that profit excluding some items will be $3.35 to $3.45 a share for fiscal 2013, which ends in March. That’s less than BMC’s prior forecast of $3.49 to $3.59 and trails analysts’ average $3.56 projection, according to data compiled by Bloomberg. Activist shareholder Elliott Associates LP pressed BMC to consider a sale last year, unsettling customers and making it harder for the company to clinch deals, BMC Chief Executive Officer Bob Beauchamp told analysts on a conference call yesterday. The company had two large contract renewals pushed to the March quarter from the December period, crimping results. “BMC underperformed across all metrics for the quarter,” and deals in the mainframe and server-software businesses didn’t get closed, Abhey Lamba, an analyst at Mizuho Securities USA Inc., wrote in a research report. “Despite having completed its accelerated share repurchase program, the company lowered its EPS target for the year, which is clearly disappointing.” ‘Growth Areas’ The company said it needs to be “more consistent and disciplined” in the way it secures big contracts, according to the statement. Management is “scrutinizing the entire company to improve our operational discipline,” Beauchamp said. BMC added board members and initiated a $1 billion share buyback to assuage Elliott’s complaints that it wasn’t doing enough to maximize shareholder value. Competitors are “aggressively” capitalizing on the pressure by Elliott, prompting many customers to seek personal visits from BMC management before completing purchases, Beauchamp said. Beauchamp said during a conference call with analysts that he’s undertaking an operational review aimed at directing spending toward better performing products to increase earnings, cash flow and operating margin amid low revenue growth. The company is trying to improve profitability even as revenue this year is projected to grow in the low single digits. Web-delivered software, cloud-computing management products and tools for managing mobile devices are selling well and will get a greater share of investment compared with underperforming products, he said. “We’re going to find a way to move funds to these growth areas,” Beauchamp said. “It’s just moving faster.” BMC kicked off the review after the end of the quarter on Dec. 31, according to Mark Stouse, a spokesman.


Homeserve Advances on Private Equity Attractions: London Mover

Homeserve Plc (HSV) , the supplier of home emergency services to almost five million households in Europe and the U.S., was the best performer in the FTSE All-Share Index (ASX) after Liberum Capital said the company may attract private equity buyers. The shares rose as much as 14 percent, the most since the Walsall, England-based company published annual results on May 21. The stock was 10 percent higher at 271 pence at 12:18 p.m. in London , heading for the highest closing price in four months. The volume of shares traded was 45 percent more than the three-month daily average. Full-year results gave some reassurance that Homeserve will deliver “stability” and perhaps growth,’’ Joe Brent, an analyst at Liberum in London, wrote in a note to clients today. The company is also “approaching an end game” with the U.K. regulator over telephone misselling, wrote Brent, who reiterated his hold rating on the stock. “The combination of these two things makes Homeserve more attractive to a private equity buyer, who would value the relative resilience of the U.K. and the growth of overseas,” Brent wrote. Private Equity The Financial Times today reported there had been “theories” about potential private equity interest, without saying where it got the information. Homeserve, under investigation by the Financial Services Authority for more than a year over inappropriate telephone selling practices, said last month it had set aside 6 million pounds ($9 million) for a possible fine and related costs. That would be less than analysts had expected, according to a note at the time by Panmure Gordon. The shares have halved since their peak two years ago, falling 56 percent in 12 days after saying in October 2011 it was suspending telephone sales. The company now has more than half its business outside the U.K. and expects to have more customers in the U.S. than in Britain by 2015, Chief Executive Officer Richard Harpin said in a telephone interview on May 21.


HootSuite Gets $165 Million to Expand Social-Media Tools

HootSuite Media Inc. , a developer of software used to manage social-networking activity, has raised $165 million from investors in one of the largest venture capital deals in North America this year. Insight Venture Partners led the second financing round, which also included Accel Partners and OMERS Ventures, Vancouver-based HootSuite said in a statement today. Terms of the investment weren’t disclosed. HootSuite plans to use the funds to add more than 100 employees in the next six months and expand to new markets including Asia and Latin America , Chief Executive Officer Ryan Holmes said in an interview. The investment will also help HootSuite build its business without seeking funds via an initial public offering, he said. “I personally would love to take this company to an acquisition or an IPO,” Holmes said. “That being said, my goal right now is building an amazing business. I think this raise takes some of that pressure off.” The company is also in talks to make at least two acquisitions, Holmes said, declining to name the targets. HootSuite has almost doubled in size, to 300 employees, in the past year as it adds tools to help businesses manage their presence on Facebook Inc. (FB) , Twitter Inc. and other social sites. The financing is the third-biggest venture deal for a North American technology company this year, behind the $200 million investment in mobile-device management company AirWatch LLC and online bulletin board Pinterest Inc., according to data compiled by Bloomberg.


NewStar Financial Said to Work With Credit Suisse on Sale

NewStar Financial Inc. (NEWS) , the business lender backed by private-equity firms Corsair Capital LLC and Capital Z Partners Management LLC, is exploring a sale, according to people with knowledge of the matter. NewStar is working with Credit Suisse Group AG (CSGN) to solicit offers, said one of the people, who asked not to be identified because the process is confidential. Boston-based NewStar rose 13 percent to $17.76 today, its highest since February 2007, giving the company a market value of about $863 million. Specialty lenders like NewStar, which makes loans to health-care, manufacturing and energy companies, typically charge higher interest rates, making them appealing targets for U.S. banks looking to increase profits. CapitalSource Inc. (CSE) , which also lends to businesses, agreed in July to sell to PacWest Bancorp (PACW) for $2.3 billion. “This company is a prime takeover target,” said Sameer Gokhale, an analyst who covers NewStar for Janney Montgomery Scott LLC in Philadelphia. NewStar will appeal to regional banks interested in expanding in commercial lending, said Gokhale, who has a “neutral” rating on the stock and a price target of $17. Logical buyers include Jacksonville, Florida-based EverBank Financial Corp. (EVER) , Miami Lakes, Florida-based BankUnited Inc. (BKU) , and Wayzata, Minnesota-based TCF Financial Corp. (TCB) , he said. Early Backers Robert Brown , a spokesman for NewStar, declined to comment, as did Credit Suisse spokesman Jack Grone and Robert Siegfried at Corsair. Spokesmen at Capital Z and the three regional banks didn’t immediately reply to requests for comment. NewStar, with offices in nine U.S. cities, makes loans and leases of $5 million to $20 million to companies with annual sales of $25 million to $500 million, according to its annual report. The company had assets of $2.28 billion at the end of June, data compiled by Bloomberg show. Along with other backers, Corsair and Capital Z, which each own close to 20 percent of NewStar, invested $210 million in equity and $450 million in debt to help start the company in 2004, according to a press release at the time. PacWest offered about 1.47 times CapitalSource’s book value, in the cash-and-stock takeover that was announced in July. At that multiple, NewStar would be worth about $18.21 a share, based on its book value of $12.39 a share at the end of June, data compiled by Bloomberg show.


Ansell to Purchase BarrierSafe Glove Maker for $615 Million

Ansell Ltd. (ANN) , the maker of Hyflex industrial gloves, agreed to buy BarrierSafe Solutions International for about $615 million, bolstering sales of hand protection products in North America. The acquisition from Odyssey Investment Partners and others is subject to U.S. antitrust clearance and is expected to be completed in the first quarter of 2014, Melbourne-based Ansell said in an e-mailed statement today. The purchase will be funded by a $300 million debt facility and the sale of new shares through a private placement of A$338 million ($308 million) and a share purchase plan of as much as A$100 million. Acquisitions will be a key part of Ansell’s growth strategy, the company said in July 2010. BarrierSafe is the leading North American provider of single-use gloves, Ansell said today. Founded in 1987, it has annual sales of about $290 million and will start adding to Ansell’s earnings next year, excluding transaction costs and one-time integration expenses. “The acquisition of BarrierSafe is a rare opportunity to strengthen the position of Ansell in the core North American single-use glove market and will further enhance our position as the global leader in hand protection solutions,” said Magnus Nicolin, Ansell’s chief executive officer and managing director, in the statement. Standard & Poor’s revised its rating outlook on Ansell to negative from stable following the announcement, meaning the ratings company sees a one-in-three chance that Ansell’s BBB-long-term credit rating may be lowered over the next two years “because of greater financial leverage and potential integration risks following the acquisition,” it said today. Ansell was advised by UBS AG, and Drinker Biddle & Reath LLP and Baker & McKenzie provided legal advice. The announcement was made before the start of trading on the Australian stock exchange. Ansell fell 0.3 percent to A$19.46 yesterday. The shares , which are halted from trading until tomorrow, have jumped 27 percent in 2013, almost double the 15 percent increase of the benchmark S&P/ASX 200 Index.


BMC Software Drops After Profit Forecast Misses: Houston Mover

BMC Software Inc. (BMC) , a maker of programs that help companies update servers and personal computers, fell after forecasting fiscal 2013 profit that missed analysts’ estimates, a sign it’s struggling to clinch big contracts. The shares declined 6.2 percent to $41.71 at the close in New York , the biggest decline since October 2011. The Houston- based company said yesterday that profit excluding some items will be $3.35 to $3.45 a share for fiscal 2013, which ends in March. That’s less than BMC’s prior forecast of $3.49 to $3.59 and trails analysts’ average $3.56 projection, according to data compiled by Bloomberg. Activist shareholder Elliott Associates LP pressed BMC to consider a sale last year, unsettling customers and making it harder for the company to clinch deals, BMC Chief Executive Officer Bob Beauchamp told analysts on a conference call yesterday. The company had two large contract renewals pushed to the March quarter from the December period, crimping results. “BMC underperformed across all metrics for the quarter,” and deals in the mainframe and server-software businesses didn’t get closed, Abhey Lamba, an analyst at Mizuho Securities USA Inc., wrote in a research report. “Despite having completed its accelerated share repurchase program, the company lowered its EPS target for the year, which is clearly disappointing.” ‘Growth Areas’ The company said it needs to be “more consistent and disciplined” in the way it secures big contracts, according to the statement. Management is “scrutinizing the entire company to improve our operational discipline,” Beauchamp said. BMC added board members and initiated a $1 billion share buyback to assuage Elliott’s complaints that it wasn’t doing enough to maximize shareholder value. Competitors are “aggressively” capitalizing on the pressure by Elliott, prompting many customers to seek personal visits from BMC management before completing purchases, Beauchamp said. Beauchamp said during a conference call with analysts that he’s undertaking an operational review aimed at directing spending toward better performing products to increase earnings, cash flow and operating margin amid low revenue growth. The company is trying to improve profitability even as revenue this year is projected to grow in the low single digits. Web-delivered software, cloud-computing management products and tools for managing mobile devices are selling well and will get a greater share of investment compared with underperforming products, he said. “We’re going to find a way to move funds to these growth areas,” Beauchamp said. “It’s just moving faster.” BMC kicked off the review after the end of the quarter on Dec. 31, according to Mark Stouse, a spokesman.


EMGS Confident of Boosting Backlog After Signing Brazil Deal

Electromagnetic GeoServices ASA (EMGS) , a Norwegian oil and gas surveyor, said it’s confident of increasing its order backlog after signing an agreement that expands its coverage of the waters off Brazil. “Towards the end of the year we will have at least one longer-term contract communicated,” Chief Financial Officer Svein Knudsen said in an interview in Oslo today. “We’re promising improved backlog and we’re sticking to that promise.” EMGS, based in Trondheim, plans to almost double its order backlog to at least $100 million by the end of the first half, Knudsen said in a presentation today. Shares in the company have fallen more than 30 percent since the start of the year amid concerns that the company’s backlog is shrinking. That’s even after EMGS signed a three-year agreement with Royal Dutch Shell Plc for the provision of 3D electromagnetic services globally. EMGS, which uses electromagnetic technology to search for oil and gas under the seabed, signed a deal with Spectrum ASA (SPU) to access its multi-client 2D survey library in order to plan a multi-client 3D survey in the Foz de Amazonas basin off Brazil, it said in a statement today. The survey will cover blocks that are included in the 11th licensing round, it said. EMGS expects to agree on similar collaborations elsewhere in the world with potential partners including Petroleum Geo- Services ASA (PGS) and TGS Nopec Geophysical ASA, Norway ’s two-largest seismic companies, Knudsen said. EMGS uses a method of mapping the seabed that competes with the seismic technology employed by companies such as Schlumberger Ltd., TGS and PGS. Shares in EMGS gained as much as 3.3 percent and closed unchanged at 9 kroner in Oslo.


Ceragon Jumps Most in 1 Month on Order: Israel Overnight

Ceragon Networks Ltd. (CRNT) , a maker of systems for mobile-phone and Internet providers, rose the most in a month in New York after reporting orders of $3.5 million that tempered investors’ concern about a revenue drop. The shares climbed 2.3 percent to $3.11 yesterday, the most since June 7, to trade 0.3 percent above the stock listed in Tel Aviv. The Bloomberg Israel-US Equity Index gained a second day, led by Caesarstone Sdot Yam Ltd., a maker of quartz countertops. ClickSoftware (CKSW) Technologies Ltd., whose second-biggest investor is George Soros ’s Soros Fund Management LLC, extended a three-day plunge to 15 percent. Ceragon, which got 16 percent of sales from the Asia-Pacific region last year, said yesterday that a “large” mobile operator in the region placed the orders. Ceragon is still down 20 percent since April 8, when Chief Executive Officer Ira Palti cut revenue forecasts. The company has reported losses for nine consecutive quarters and is projected to post the biggest decline in annual sales in more than a decade, according to the average estimate of analysts surveyed by Bloomberg. The new order “shows some momentum,” Joseph Wolf , an analyst at Barclays Plc, who rates the stock the equivalent of hold, said in a telephone interview from Tel Aviv yesterday. “But it’s not an inflection point in the weak demand environment that’s an overhang on Ceragon’s growth rate. It’s a good company with a very tough end market.” The Bloomberg Israel-US gauge rose 0.7 percent to 92.15, the highest price since June 18. The benchmark TA-25 Index (TA-25) advanced for the first time in three days, adding 0.7 percent to 1,214.69 at 10:24 a.m. in Tel Aviv. Ceragon’s Tel Aviv shares this morning slipped 0.8 percent to 11.19 shekels, or $3.09, in volume of 4 percent of the three-month daily average. Caesarstone (CSTE) jumped 6 percent to $29.10, the highest price since May 30. The shares have rallied 82 percent in 2013. ClickSoftware, an Israeli software developer, declined 4.4 percent to $7.11, the lowest price since April 23. Earlier this week, the Petach Tikva, Israel-based company reported profit and revenue that trailed analysts’ projections.


Kony Solutions Weighs Public Offer as Early as This Year

Kony Solutions Inc. , a provider of mobile-application platforms, has talked to banks including Goldman Sachs Group Inc. (GS) and JPMorgan Chase & Co. (JPM) about selling shares in a public offering as early as this year. The company, which has its headquarters in Orlando , Florida , as well as in London , plans to select bankers this year after introducing a service that will provide easier access to its app platforms, Chief Executive Officer Raj Koneru said in an interview yesterday. The share sale will take place this year or in 2014, he said. “We are talking with a lot of bankers,” Koneru said. The company hasn’t decided how much stock to sell or the amount of funds to raise, he said. Kony, which has yet to make a profit, expects to record $80 million to $100 million in revenue this year, about double its 2012 sales, Koneru said. An initial public offering could raise at least $120 million for Kony, valuing the company at more than $700 million, Tom Taulli , an analyst who covers IPOs, said in an interview. “Enterprise is really hot right now,” Taulli said. “Right place at the right time -- that always helps with an IPO. I suspect they’ll do well.” Spending on app-development tools for multiple mobile operating systems will grow at an average annual rate of 39 percent to reach $8.2 billion in 2016, according to Smith’s Point Analytics LLC. Founded in 2007, Kony sells annual subscriptions for its app platform-service, which allows developers to customize products to their own specifications. The company has raised about $35 million from Georgian Partners and Insight Venture Partners and may use proceeds from an IPO to improve its software and for research and development, Koneru said.


Whether Public or Private, Information Technology Is Hard To Do Right

A timely reminder that the Department of Health and Human Services is not the only place having information technology woes: This at the NSA -- which is supposed to be so good at what it does that I am half expecting a cheery note from my personal NSA agent reminding me that I should have had whole wheat toast this morning instead of the cheap white bread. Yet the NSA is not even being undone by something difficult, like melding multiple databases into one coherent website. It's having trouble keeping electrical boxes from melting down, which is one of those problems I thought the utilities had solved decades ago. There’s a lesson in this for both sides of the political spectrum. We live our lives immersed in wondrous technology -- especially those of us who spend our workdays on the Internet. Over time, we’ve come to think that anyone can do this sort of thing. But IT is hard. Conservatives who argue that this shows the government can never do IT right should remember that lots of companies can -- and do -- get IT wrong. We think the private sector is so good at it because we see only the winners who got it right. The many projects that went horribly wrong have slipped out of view, and memory. But liberals who have been proclaiming that the health exchange glitches will be fixed eventually because after all, Amazon does this, should remember that the end of every glitchy project is not a product that actually works. Horrifyingly bad launches, into which category I’d say the exchanges now fall, often end when the product is jerked out of production. More than occasionally, the company that made the product goes away. Next time you go onto Amazon, or Netflix or Travelocity, remember these glitches and remind yourself what a hard job these companies are doing amazingly well. You live in an age of technological miracles. You should occasionally let yourself feel the awe that this ought to inspire.


Nissan Opens Silicon Valley Center for Self-Driving Car Research

Nissan Motor Co. opened a research center in California’s Silicon Valley that the Japanese carmaker plans to make its hub for research on self-driving vehicles and Internet-connected auto technology. The facility will be staffed by more than 60 engineers and technicians within three years, Carla Bailo , Nissan’s senior vice president for North American research and development, said in a phone interview. Work on so-called autonomous vehicle systems will move from an R&D center in Japan to Sunnyvale, she said, declining to provide investment details for the project. “We’re going to focus on this technology really in the heart of where it lies today,” Bailo said in the Feb. 15 interview. “We are going to be shifting work that’s being done in our Nissan Advanced Technical Center in Atsugi, and moving that work over to the heart of the industry.” Automakers and companies including Google Inc. plan new electronics and programming for vehicles with a goal of reducing collisions, fuel use and improving traffic flow. Carlos Ghosn , Nissan’s chief executive officer, told reporters last month in Detroit that regulatory, legal and technological hurdles mean self-driving cars won’t be feasible until about 2020. Google has tested a self-driving Prius in California for years, and the U.S. Defense Department has sponsored autonomous- vehicle research for more than a decade. Nissan’s Sunnyvale center will work with automotive labs at Stanford University, the University of California’s Berkeley and Davis campuses, and Silicon Valley companies, Bailo said. The Yokohama-based carmaker plans to apply technology developed at the new R&D center within a decade, she said. Maarten Sierhuis , a former NASA scientist specializing in artificial intelligence research, will lead the Sunnyvale facility, Nissan said today. Toyota Motor Corp. opened its own Silicon Valley outpost in Mountain View in 2012 to work with technology companies there on in-car data and entertainment systems. The company said in January that it’s developing autonomous safety systems to create a virtual “co-pilot” in vehicles to help drivers avoid accidents rather than self-driving cars and trucks.


Ansell to Purchase BarrierSafe Glove Maker for $615 Million

Ansell Ltd. (ANN) , the maker of Hyflex industrial gloves, agreed to buy BarrierSafe Solutions International for about $615 million, bolstering sales of hand protection products in North America. The acquisition from Odyssey Investment Partners and others is subject to U.S. antitrust clearance and is expected to be completed in the first quarter of 2014, Melbourne-based Ansell said in an e-mailed statement today. The purchase will be funded by a $300 million debt facility and the sale of new shares through a private placement of A$338 million ($308 million) and a share purchase plan of as much as A$100 million. Acquisitions will be a key part of Ansell’s growth strategy, the company said in July 2010. BarrierSafe is the leading North American provider of single-use gloves, Ansell said today. Founded in 1987, it has annual sales of about $290 million and will start adding to Ansell’s earnings next year, excluding transaction costs and one-time integration expenses. “The acquisition of BarrierSafe is a rare opportunity to strengthen the position of Ansell in the core North American single-use glove market and will further enhance our position as the global leader in hand protection solutions,” said Magnus Nicolin, Ansell’s chief executive officer and managing director, in the statement. Standard & Poor’s revised its rating outlook on Ansell to negative from stable following the announcement, meaning the ratings company sees a one-in-three chance that Ansell’s BBB-long-term credit rating may be lowered over the next two years “because of greater financial leverage and potential integration risks following the acquisition,” it said today. Ansell was advised by UBS AG, and Drinker Biddle & Reath LLP and Baker & McKenzie provided legal advice. The announcement was made before the start of trading on the Australian stock exchange. Ansell fell 0.3 percent to A$19.46 yesterday. The shares , which are halted from trading until tomorrow, have jumped 27 percent in 2013, almost double the 15 percent increase of the benchmark S&P/ASX 200 Index.


Apple Is Said to Debut New IPads at Oct. 22 Event

Apple Inc. (AAPL) will update its lineup of iPads at an Oct. 22 event, a person with knowledge of the plans said, as the company works to remain ahead of rivals in the increasingly crowded tablet market. The Cupertino, California-based company will unveil a new version of the original 9.7-inch iPad as well as the smaller iPad mini, said the person, who asked not to be identified because the plans haven’t been made official. Apple also will show its new Mac personal-computer operating system, called Mavericks, and Mac Pro PC, the person said. The event, scheduled for a month after Apple debuted two new iPhone models, will feature changes to the bigger iPad that include a thinner design, people familiar with the plans said in August. A new iPad mini will sport the high-definition screen now on the iPhone and bigger iPads, the people said. The event was reported earlier by AllThingsD. Trudy Muller , a spokeswoman for Apple, declined to comment. In the absence of a new iPad, the tablet market shrank in the second quarter, suggesting pent-up demand for the new models. Global tablet unit sales fell 9.7 percent to 45.1 million, with iPad shipments falling to 14.6 million units from 19.5 million in the first quarter, according to researcher IDC. While Apple remains the market leader for tablets, which meld the functions of touch-screen smartphones and laptops, its share has been threatened as competitors such as Amazon.com Inc., Samsung Electronics Co. (005930) and Microsoft Corp. introduce other models. Apple shares fell 1.4 percent to $480.94 at yesterday’s close in New York. The stock has fallen 9.6 percent this year.


Arabtec, Samsung Engineering Form Joint Venture

Arabtec Holding PJSC (ARTC) , the Persian Gulf’s largest publicly traded construction company, formed a venture with Samsung Engineering Co. (028050) to further its expansion in the oil and gas industry. Arabtec-Samsung Engineering will be 60 percent owned by Arabtec and 40 percent owned by Seoul-based Samsung Engineering. Management will be appointed shortly, according to a statement by the two companies today. The joint venture will “undertake large-scale projects in oil & gas, power and infrastructure in the Middle East and North Africa,” the companies said. Arabtec-Samsung will bid for contracts ranging from $3 billion to $10 billion in value. Samsung Engineering currently has a $10 billion contract backlog in the regions. Arabtec, which helped build the world’s tallest tower in Dubai, in February replaced its management and said it plans to more than double its market value. The Dubai-based company also said it’s planning to raise 6.4 billion dirhams ($1.7 billion) through a bond and share sale. The proceeds will be used to bolster the company’s oil-and-gas installation business and expand its affordable housing capacity, Arabtec said in March. Yesterday, Arabtec signed a 10-year agreement to lease the 59-floor office tower in Abu Dhabi where Arabtec Construction and Arabtec-Samsung Engineering will be based, the company’s Chief Executive Officer Hasan Ismaik said.


Pearson’s Penguin Settles States’ E-Book Antitrust Claims

Pearson Plc’s (PSON) Penguin unit said it reached a $75 million settlement of antitrust claims by 33 states and consumers over electronic-book pricing. Penguin, which was sued last year for allegedly participating in a conspiracy with other publishers and Apple Inc. (AAPL) to fix the price of e-books, announced the settlement in a statement today. The company resolved related claims with the U.S. Justice Department in December. The states claimed damages on behalf of people who were forced to pay too much for e-books because of the alleged conspiracy. The Justice Department and the states claimed Apple and five publishers, including Penguin, fixed prices in an attempt to undermine Amazon.com Inc. (AMZN) ’s dominance of the market. The Penguin settlement must be approved by a federal judge in New York before it can take effect. Apple is scheduled to begin a trial June 3 on civil antitrust claims by the Justice Department and states. The case is U.S. v. Apple, 11-cv-02293, U.S. District Court, Southern District of New York ( Manhattan ).


Real Loser in Apple E-Book Verdict Is Barnes & Noble

The Apple e-book verdict is in: guilty. A federal judge ruled today that the world's biggest technology company violated antitrust laws by scheming with publishers to fix the prices of electronic books. Apple Inc., which plans to appeal, will probably face damages. The real loser in the ongoing e-book battle, however, continues to be the only company that exists solely to provide consumers with books: Barnes & Noble Inc. Barnes & Noble, which came to Apple's defense in the trial, had employed the same so-called agency model Apple used to compete with market-leader Amazon.com Inc. Apple let publishers set prices for the e-books it sold, then collected 30 percent of the revenue. Under the deal, publishers couldn't sell more cheaply to Amazon, which undercut Amazon's discount strategy. The agency model enabled Apple and Barnes & Noble to gain market share, but it collapsed last year when, in a settlement with the Justice Department, five major publishers agreed to drop the practice. Given that the changes in the way e-books are sold have already taken place, it's unlikely that, even with today's verdict, Apple will lose much of its share of the e-book market. In any case, e-books are only an ancillary source of revenue for the company, and a fine would merely be a slap on the wrist. Barnes & Noble has more at stake. The company lost $475 million on its Nook tablet business in the last fiscal year. It has discontinued its color tablets, and Chief Executive Officer William Lynch, who made the Nook a key part of the company's strategy, resigned Monday. Still, there's hope for Barnes & Noble. Even as the Nook business deteriorated, its retail unit became more profitable last fiscal year, generating $374.2 million in earnings, a 16 percent increase from the year before. As Matt Townsend writes in Bloomberg News, printed books still have value. The rise of e-books, which make up just 20 percent of the book market, is slowing. Publishers use Barnes & Noble to sell their older titles, since the e-book market is primarily a place for new releases. There's also the academic market: Barnes & Noble manages some 700 university bookstores, Lydia DePillis writes in the Washington Post. And where else would writers go on book tours? The Apple verdict is another wakeup call in a string of many for Barnes & Noble, which needs to turn away from the e-book battle and toward the challenges of being a bookstore. (Kirsten Salyer is the social media editor for Bloomberg View. Follow her on Twitter.)


Chelsea Market Owner Purchases L.A. Complex Housing IMAX

The firm that owns New York’s One Times Square, where the ball drops each New Year’s Eve, bought a California office-and-studio complex that houses Dick Clark Productions and big-screen movie-theater operator IMAX Corp. (IMX)  Jamestown , which also owns Chelsea Market in New York, bought the 484,840-square-foot (45,043-square-meter) Lantana campus in Santa Monica from Lionstone Group, Atlanta-based Jamestown said today in a statement. The purchase price was about $310 million, according to a person with knowledge of the transaction. “The innovation economy is driving demand for work spaces that offer a vibrant mix of amenities, metropolitan ambience and collaborative, flexible environments,” Jamestown Chief Operating Officer Michael Phillips said in the statement. Lantana is the company’s first purchase in Southern California. The property’s tenants include film-sound company Todd-AO and workout-video producer Beachbody LLC. The complex is 94 percent leased, with “exceptional in-place cash flow,” Jamestown said. The company also owns properties in San Luis Obispo, California , and the San Francisco Bay area.


NBA Unveils Advanced Statistical Player Analysis on Its Website

The National Basketball Association added advanced statistical categories to its website , joining a movement toward increased use of sports data to help teams, fans and media members analyze player performance. The website now includes every box score in NBA history, shooting charts that help visualize offensive and defensive performances and statistics that break down player attributes such as efficiency and how clutch they are, the league said in a statement. “Sports fans have a passion for statistics because it brings them closer to the game, their favorite teams and players,” said Steve Peck , senior vice president of global strategic initiatives at SAP AG , which partnered with the NBA to power the website. The NBA’s entire official statistics history now will be offered free after previously being made available only to teams and league officials. Content found on the site can be shared on social media platforms Twitter, Facebook and Google+. Using advanced data to analyze player performance has become common in Major League Baseball since Bill James used statistics to help make the Boston Red Sox World Series champions in 2004. The movement was popularized by Michael Lewis ’s book “Moneyball,” which chronicled Oakland Athletics General Manager Billy Beane ’s use of statistics to build his roster. Lewis is a Bloomberg News columnist.


CAF Seeks $3 Billion to Fund Latin American Projects This Year

Corporacion Andina de Fomento, the Latin American development lender known as CAF, plans to raise $3 billion this year to finance energy and infrastructure projects, according to Chief Financial Officer Hugo Sarmiento. About $2.5 billion will come from international bond sales and the remainder will come from loans, Sarmiento said today in an interview at Bloomberg’s offices in London. The Caracas-based lender plans to boost its credit portfolio by 10 percent this year from $16.5 billion at the end of 2012, Sarmiento said. Proceeds will be used to finance projects across the region, which may include part of Rio de Janeiro ’s Maracana soccer stadium, according to Sarmiento. CAF is also studying mass transit system projects in Sao Paulo and Rio in anticipation of the 2014 World Cup and 2016 Olympics, he said. “We’ve seen an increase in demand for Latin American assets” such as the bonds CAF will sell, he said. “When you look at the macroeconomic environment, which is very stable, and you compare that to the developed world, which is in crisis, it makes sense to park your funds there.” Stronger economic growth than in the developed world helped attract $131 billion of private financial flows to Latin America and the Caribbean in 2012, up from $85.8 billion in 2007, according to IMF data. Gross domestic product in eight Latin American countries rose by an estimated 2.66 percent on average in 2012, compared with an estimated contraction of 0.4 percent for the euro region and growth of 2.2 percent in the U.S., data compiled by Bloomberg show. CAF expects its next bond issue to be a U.S. dollar- denominated sale in the second quarter, Sarmiento said. The lender also intends to sell about 500 million yuan ($80 million) of bonds later in the year to tap growing demand from Asian investors, he said. The lender’s 4.375 percent dollar bonds due 2022 have returned 10 percent since they were issued in June. The yield was little changed today at 3.4 percent. CAF is rated Aa3 by Moody’s Investors Service, the third- highest investment grade, and AA- at Standard & Poor’s , the company’s fourth-highest ranking.


ANA Considers Ending AirAsia Tieup on Japan Discount Carrier

ANA Holdings Inc. (9202) , which owns a majority stake in a Japanese discount carrier set up in partnership with AirAsia Bhd. (AIRA) , said it is considering dissolving the tie-up as it aims to improve sales at the unit. Ending the partnership in the venture, AirAsia Japan Co. , is one option and no final decision has been made, Megumi Tezuka, a spokeswoman for Tokyo-based ANA, said by phone today. AirAsia Group chief Executive Officer Tony Fernandes said by e-mail that he would comment “in due course.” ANA would continue operating the business as a low-cost carrier from Narita airport even if the partnership with AirAsia is dissolved, Tezuka said. AirAsia Japan has struggled to fill as many seats as low-cost carriers Peach Aviation Ltd., partly owned by ANA, and Jetstar Japan Co. , which also began flights in Japan last year. “ANA should be able to turn around the carrier,” said Ryota Himeno , an analyst at Barclays Securities Japan Ltd. “They’ve already shown they can be successful with Peach. AirAsia may have discovered the difficulties of flying from Narita,” rather than Japan’s downtown Haneda airport, he said. The Nikkei newspaper reported earlier today that ANA will acquire AirAsia’s stake in AirAsia Japan, without saying where it got the information. 33 Percent Stake ANA, which owns 67 percent of AirAsia Japan, may buy out AirAsia’s stake in the venture, Tezuka said. Narita, Japan-based AirAsia Japan filled 67.6 percent of seats on its domestic flights during Japan’s Golden Week holiday period from April 26 to May 6, the company said in a statement last month. In comparison Peach, which flies from Kansai airport in Osaka , Japan, filled 91.3 percent of its domestic seats, according to the company. Jetstar Japan, which only flies within the country and is also based in Narita, filled 78.8 percent of seats, according to the company. ANA has said that it aims to fill 80 percent or more of its seats on low-cost carrier operations. AirAsia Japan, which began operations in August, has five domestic services and also flies to Seoul and Busan in South Korea. The carrier will add flights to Taiwan next month and is considering another overseas flight before the end of March as it more than doubles its fleet to nine planes from four, Yoshinori Odagiri, chief executive officer of the carrier, said last week. ANA rose 3.6 percent to 204 yen at the close of trading in Tokyo and is up 13 percent this year. AirAsia, based in Sepang Selangor, Malaysia, rose 0.6 percent to close at 3.35 ringgit in Kuala Lumpur.


Aussie 10-Year Bond Yield at 17-Month High Before U.S. Jobs Data

Australian bonds fell, pushing the benchmark 10-year yield to its highest in 17 months, before a U.S. Labor Department report that may show jobs growth accelerated last month in the world’s largest economy. The Aussie and New Zealand dollars maintained declines from yesterday on prospects the report will fuel expectations that the Federal Reserve will reduce its bond purchases this month, slowing a program that has devalued the greenback. Both currencies are set to advance this week on data showing China ’s manufacturing expanded and the euro-area economy returned to growth. The Aussie’s share of global currency turnover rose one percentage point to about 8.5 percent, the central bank said today citing a survey by the Bank for International Settlements. “The U.S. growth story is very important as is the better economic data coming out of Europe , because if you get a recovery in global demand, that benefits Australian exports,” said Philip Brown, a fixed-income strategist in Melbourne at Commonwealth Bank of Australia , the nation’s largest bank. “There is scope for the bond market to keep selling off, though it should slow a bit.” Australia ’s 10-year bond yield rose as much as eight basis points to 4.151 percent, the most since April 2012, and was as 4.15 at 10:05 a.m. in Sydney. It has climbed 25 basis points since Aug. 30, set for the largest increase in three weeks. The three-year rate touched 2.986 percent, the highest since June 24. A basis point is 0.01 percentage point. Weekly Gains The Aussie traded at 91.20 U.S. cents, little changed from yesterday’s close of 91.22 and up 2.5 percent this week. It fell 0.1 percent to 91.26 yen, paring its weekly advance to 4.4 percent, the most since the period ended Dec. 2, 2011. New Zealand ’s kiwi rose 0.1 percent to 78.90 U.S. cents. It gained 2.1 percent since Aug. 30, ending a two-week slide. The currency was little changed at 78.99 yen and traded 4.1 percent higher from the end of last week. The U.S. Labor Department is forecast to say today that payrolls rose by 180,000 in August, from 162,000 the previous month, while the jobless rate held at 7.4 percent, according to the median estimate in a Bloomberg News poll. A Bloomberg survey of economists last month showed that 65 percent expected that the Fed will start unwinding its $85 billion a month bond-buying program at this month’s meeting.


HootSuite Gets $165 Million to Expand Social-Media Tools

HootSuite Media Inc. , a developer of software used to manage social-networking activity, has raised $165 million from investors in one of the largest venture capital deals in North America this year. Insight Venture Partners led the second financing round, which also included Accel Partners and OMERS Ventures, Vancouver-based HootSuite said in a statement today. Terms of the investment weren’t disclosed. HootSuite plans to use the funds to add more than 100 employees in the next six months and expand to new markets including Asia and Latin America , Chief Executive Officer Ryan Holmes said in an interview. The investment will also help HootSuite build its business without seeking funds via an initial public offering, he said. “I personally would love to take this company to an acquisition or an IPO,” Holmes said. “That being said, my goal right now is building an amazing business. I think this raise takes some of that pressure off.” The company is also in talks to make at least two acquisitions, Holmes said, declining to name the targets. HootSuite has almost doubled in size, to 300 employees, in the past year as it adds tools to help businesses manage their presence on Facebook Inc. (FB) , Twitter Inc. and other social sites. The financing is the third-biggest venture deal for a North American technology company this year, behind the $200 million investment in mobile-device management company AirWatch LLC and online bulletin board Pinterest Inc., according to data compiled by Bloomberg.


Capstone to Buy Renewable Energy Developers to Boost Wind

Capstone Infrastructure Corp. (CSE) , a Canadian power producer, agreed to acquire Renewable Energy Developers Inc. for C$70 million ($66.6 million) to expand its wind operations. Renewable Energy shareholders will receive 0.26 Capstone common shares and 0.1 Canadian cent for each share owned, worth C$1.01 a share, Capstone said today in a statement. That’s a 10.8 percent premium over the 20-day weighted average of Renewable Energy’s shares. Both companies are based in Toronto. The deal gives Capstone 95 megawatts of wind farms in operation in Nova Scotia and Ontario, and access to 35 megawatts of wind projects under development. The deal is expected to close in September. Capstone also will have the option to acquire another 44 megawatts of wind power under development for C$60 million in equity funding. When complete, Capstone will have about 550 megawatts of operating and contracted power assets in Canada. The company also has solar, biomass and hydropower plants, and one natural gas cogeneration facility. Capstone shareholders will own about 80 percent of the combined company and Renewable Energy shareholders about 20 percent.


Wincanton Rises on Valero Deal to Transport Fuel: London Mover

Wincanton Plc (WIN) climbed 3 percent after the transporter of products ranging from groceries to cement extended a contract with the largest independent petroleum refiner, Valero Energy Corp. (VLO) , for five years. Wincanton will transport more than 2 billion liters (528 million gallons) of fuel a year to more than 1,000 U.K. locations under the deal, the company said in a statement today. Wincanton, which supplies logistics and transport services to customers including Tesco Plc (TSCO) , Britain’s biggest supermarket chain, and Marks & Spencer Group Plc, the U.K.’s largest clothing retailer, rose 3 pence to 103.25 pence in London. “We like Wincanton because there’s significant recovery potential in it,” Steve Woolf, an analyst at Numis Securities, said by phone. “Contract renewals and increasing scope of work with their customers is positive for the business longer term.” Woolf recommends buying the shares and has a target price of 120 pence for the stock. Under Chief Executive Officer Eric Born, Chippenham, England-based Wincanton, has won extended contracts in the past six months with clients including William Morrison Supermarkets Plc and cement company Cemex. Wincanton returned to profit in the fiscal year through March following two years of losses. The share price has more than doubled in 12 months, giving the company a market value of about 126 million pounds ($202 million). Of five analysts who follow the company and share their findings with Bloomberg, four recommend buying the stock and one suggests selling.


Whitehaven Coal Targeted by Green Group in Hoax Funding E-mail

Whitehaven Coal Ltd. (WHC) , developing the A$766 million ($803 million) Maules Creek mine, was targeted by a hoax e-mail from a green group campaigning against the project that sent its shares down as much as 8.8 percent. Frontline Action on Coal , an environmental group that opposes the mine in Australia ’s New South Wales state, sent a fake press release that Australia & New Zealand Banking Group Ltd. (ANZ) had terminated a A$1.2 billion loan to Whitehaven on environmental grounds, Jonathan Moylan, a spokesman for the group, said by phone. Whitehaven recovered to trade 0.3 percent lower at A$3.51 at 2:03 p.m. in Sydney after it and ANZ revealed the e-mail was a hoax. The company’s shares plunge d after the Australian Financial Review and the Australian Associated Press reported the fake statement as news. The hoax release, which mirrored the format of ANZ’s official statements, listed a phone number purportedly for Toby Kent, the head of corporate sustainability at ANZ. The number, identical to one on the green group’s website, was answered by a man who identified himself as Moylan when Bloomberg News phoned for comment. Green groups have stepped up campaigns in courts and by occupying port facilities in Australia in an effort to reduce shipments from the world’s largest exporter of coal. Whitehaven last month completed a A$1.2 billion debt facility, underwritten by ANZ, Commonwealth Bank of Australia (CBA) and National Australia Bank Ltd. (NAB) The loan replaced existing facilities and the company plans to spend the funds on Maules Creek and general corporate purposes, it said in a statement. The company in October received approval from the NSW government for Maules Creek, located in the Gunnedah Basin with first coal sales planned in 2014.


Quebecor’s Sun Media Closes 11 Newspapers to Cut Costs

Sun Media Corp., a unit of Canadian publisher and broadcaster Quebecor Inc. (QBR/B) , is closing 11 newspapers and cutting 360 jobs in a move the company expects will save C$55 million ($53 million) a year. The moves are needed to stay competitive as print revenue continues to fall and more of the industry moves to digital, Julie Tremblay, Sun Media’s chief operating officer, said in a statement today. “Digital is more than a strong trend,” said Tremblay. “Young people are getting their information almost exclusively from digital sources.” Sun Media is closing its free urban daily 24 Hours newspapers in Ottawa, Calgary and Edmonton. 24 Hours competes for readers with Torstar Corp. (TS/B) ’s Metro, which has papers in those three cities, according to Metro’s website. The cuts come eight months after Sun Media eliminated 500 jobs and closed two production facilities in Ontario. Quebecor fell 1 percent to $47.27 at 12:55 p.m. in Toronto. Sun Media pulled the urban dailies so it could focus on one paper in each urban market except for Montreal and Toronto, where large commuter populations justify two papers, said Sun Media. In addition to the three 24 Hours papers, eight smaller papers in Quebec , Ontario, Manitoba and Saskatchewan will close or have already shuttered publication, the company said. Sun Media says it’s the largest newspaper publisher in Canada , based on paid and free circulation.


Canadian Building Permits Rise Fourth Month on Vancouver Condos

Canadian building permits unexpectedly rose in April led by multiple-unit housing in Vancouver, a market that officials have said shows signs of overbuilding fueled by record consumer debt. The value of municipal permits rose 10.5 percent to C$6.96 billion ($6.73 billion), following a revised 6 percent rise in March, Statistics Canada said today in Ottawa. The gain was the fourth in a row, the longest streak of increases in a decade. Economists forecast a 3 percent decline in April, according to the median of 11 responses to a Bloomberg survey. Surging home construction has helped drive household debts to a record 165 percent of disposable income, a ratio the central bank says should stabilize as housing investment wanes. Bank of Canada Deputy Governor Timothy Lane said June 4 in Seoul there are signs of a “constructive evolution” in household finances after “imbalances” built up. Residential permits rose 21 percent to C$4.35 billion in April, Statistics Canada said today. Permits for housing projects such as apartments and condominiums jumped 51.9 percent to C$2.15 billion, and single-family permits rose 1.1 percent to C$2.21 billion. Vancouver made one of the largest contributions to the national increase among 34 cities, Statistics Canada said, with permits rising 50.7 percent to C$645 million led by multi-family dwellings. Calgary permits also rose 40.6 percent to C$773 million on commercial buildings. Condo Overbuilding Finance Minister Jim Flaherty tightened mortgage rules for a fourth time last year on concern that there has been an overbuilding of condos in cities such as Toronto and Vancouver. Permits for non-residential construction fell 3.6 percent to C$2.60 billion in April from March, as declines for industrial and institutional projects exceeded a gain in the commercial group. The value of permits was 5.4 percent higher in April than the same month a year earlier, with multiple-unit housing 40.4 percent higher on the year. Demand for homes has been supported by some of the lowest mortgage rates in decades, with the Bank of Canada reporting the average for a five-year fixed rate loan was 5.14 percent last week. The central bank has kept its overnight interest rate at 1 percent since September 2010 to encourage borrowing and spending, and has said business investment and exports will help lead the economic expansion.


Singapore’s Temasek Said to Buy 10% of Markit for $500 Million

Temasek Holdings Pte, Singapore ’s state-owned investment company, bought a stake in Markit Group Ltd. for about $500 million, according to a person with knowledge of the matter. Temasek’s investment was for a 10 percent stake in the company, the person said, declining to be identified as the terms of the deal were not public. Markit, which provides derivative and bond data and competes with Bloomberg News parent Bloomberg LP, said today that Temasek had made a “significant” equity investment in the company. “The strength of their position and profile in Asia , an area where we see significant potential and opportunity, will help fuel our growth in the region,” Lance Uggla, chief executive officer of Markit, said in an e-mailed statement. Stephen Forshaw , a spokesman for Temasek, confirmed the company’s investment. Reuters earlier reported the size of the investment.


Hypermarcas Advances as Earnings Beat Estimates: Sao Paulo Mover

Hypermarcas SA (HYPE3) , the Brazilian maker of more than 190 consumer products from diapers to generic drugs, rose to the highest in two months after second-quarter earnings exceeded analysts’ estimates. The shares rallied 3 percent to 17.08 reais at 11:45 a.m. in Sao Paulo, the highest intraday price since June 5. Brazil ’s benchmark Ibovespa index gained 0.3 percent. Hypermarcas’s earnings before interest, taxes, depreciation and amortization rose 12 percent to 258 million reais ($112 million) in the three months through June from a year earlier, according to a regulatory filing on August 2 after the market closed. That compares with an average estimate of 246.9 million reais among eight analysts surveyed by Bloomberg. The increase prompted the investment bank Itau BBA to raise its recommendation on the stock to the equivalent of buy. “The solid second-quarter results made us more confident in the investment story of Hypermarcas, one of the few companies in our space where consensus might need to revise numbers upwards,” Vitor Paschoal and Rachel Rodrigues, analysts at the bank, wrote in a note to clients today. Sao Paulo-based Hypermarcas gained 2.8 percent this year, while the Ibovespa slumped 20 percent.


Harvest Natural Surges on Gabon Offshore Oil Discovery

Harvest Natural Resources Inc. (HNR) , the oil explorer with operations in Venezuela, Indonesia and Oman, climbed the most in more than six months after reporting its second crude discovery off the coast of Gabon. Harvest advanced 11 percent to $10.25 at the close in New York , the most since June 22, and its highest since Nov. 8, 2011. Harvest found oil in the Dussafu Tortue Marin-1 well, the Houston-based company said in a statement today. The well, in 380 feet (116 meters) of water, was dug to a depth of more than 2 miles. Further appraisal is needed to determine the scope of the discovery, including fluid sampling and a sidetrack well, according to the statement. Harvest operates Dussafu and holds a 66.66 percent stake. Panoro Energy ASA (PEN) , owner of 33.33 percent of Dussafu, rose 20 percent in Oslo to close at 3.35 kroner, the biggest one-day gain since it was spun off by Norse Energy Corp ASA (NEC) in June, 2010. Harvest reported its first discovery off Gabon in June 2011.


Chelsea Market Owner Purchases L.A. Complex Housing IMAX

The firm that owns New York’s One Times Square, where the ball drops each New Year’s Eve, bought a California office-and-studio complex that houses Dick Clark Productions and big-screen movie-theater operator IMAX Corp. (IMX)  Jamestown , which also owns Chelsea Market in New York, bought the 484,840-square-foot (45,043-square-meter) Lantana campus in Santa Monica from Lionstone Group, Atlanta-based Jamestown said today in a statement. The purchase price was about $310 million, according to a person with knowledge of the transaction. “The innovation economy is driving demand for work spaces that offer a vibrant mix of amenities, metropolitan ambience and collaborative, flexible environments,” Jamestown Chief Operating Officer Michael Phillips said in the statement. Lantana is the company’s first purchase in Southern California. The property’s tenants include film-sound company Todd-AO and workout-video producer Beachbody LLC. The complex is 94 percent leased, with “exceptional in-place cash flow,” Jamestown said. The company also owns properties in San Luis Obispo, California , and the San Francisco Bay area.


We can now add the tool's output to the message chain with `ToolMessage`, so the model can answer based on the retrieved documents.

*Note: Here we use the base model instead of the RAG model to limit our agent to one retrieval call. A fully autonomous agent could decide to make subsequent calls to best answer the request*

In [338]:
from langchain_core.messages import ToolMessage

In [339]:
# Message list with the retrieved documents for the base model
messages = [
    # RAG System prompt
    SystemMessage(RAG_PROMPT),
    # Client request
    HumanMessage(request),
    # RAG model response
    rag_response,
]

# Create and add ToolMessage for each tool call
for tool_call in rag_response.tool_calls:
  # Get the document string for this specific tool call
  # (Assuming you store them in a list or dictionary)
  documents_str_for_tool_call = documents_str_list[
      rag_response.tool_calls.index(tool_call)
  ]

  messages.append(
      ToolMessage(
          documents_str_for_tool_call, tool_call_id=tool_call["id"]
      )
  )

# Invoke the base model with the messages
response = base_model.invoke(messages)

In [340]:
Markdown(response.content)

Based on the analysis of the Generative AI sector, here is a summary of key players, their challengers, and an investment recommendation table.

### Key Players and Their Challengers
1. **NVIDIA (NVDA)**: Leading GPU manufacturer for AI applications.
   - **Challengers**: AMD, Intel

2. **AMD (AMD)**: Competing in the GPU market, especially with its Radeon series.
   - **Challengers**: NVIDIA, Intel

3. **Intel (INTC)**: Major player in CPUs and emerging in AI with its Xe graphics.
   - **Challengers**: AMD, NVIDIA

4. **Amazon (AMZN)**: Dominates cloud computing with AWS, which is crucial for AI workloads.
   - **Challengers**: Microsoft Azure, Google Cloud

5. **Microsoft (MSFT)**: Strong presence in cloud services and AI integration.
   - **Challengers**: Amazon AWS, Google Cloud

6. **Google (GOOGL)**: Significant investments in AI and cloud computing.
   - **Challengers**: Amazon AWS, Microsoft Azure

7. **OpenAI**: Leading in generative AI models.
   - **Challengers**: Anthropic, Stability AI

8. **Adobe (ADBE)**: Integrating AI into creative software.
   - **Challengers**: Canva, other creative software companies.

### Investment Recommendation Table

| Company         | Ticker | Recommendation | Reason                                                                                          | Source & Date                                      |
|------------------|--------|----------------|-------------------------------------------------------------------------------------------------|---------------------------------------------------|
| NVIDIA           | NVDA   | Buy            | Strong demand for GPUs in AI applications; recent earnings beat expectations.                  | Bloomberg, October 2023                            |
| AMD              | AMD    | Hold           | Competitive but facing challenges from NVIDIA; recent product launches show promise.            | Bloomberg, October 2023                            |
| Intel            | INTC   | Hold           | Transitioning to AI but still lagging behind NVIDIA; recent management changes may help.       | Bloomberg, October 2023                            |
| Amazon           | AMZN   | Buy            | AWS continues to dominate cloud services; strong growth in AI-related services.                 | Bloomberg, October 2023                            |
| Microsoft        | MSFT   | Buy            | Significant investments in AI and cloud; strong growth in Azure services.                       | Bloomberg, October 2023                            |
| Google           | GOOGL  | Buy            | Major investments in AI and cloud computing; strong growth potential in AI applications.        | Bloomberg, October 2023                            |
| OpenAI           | N/A    | Buy            | Leading in generative AI; strong partnerships and funding support growth.                       | Bloomberg, October 2023                            |
| Adobe            | ADBE   | Buy            | Integrating AI into creative tools; strong market position and growth potential.               | Bloomberg, October 2023                            |

### Executive Summary
The Generative AI sector presents a compelling investment opportunity, driven by the increasing demand for advanced computing power and AI applications across various industries. Key players such as NVIDIA, Amazon, Microsoft, and Google are well-positioned to capitalize on this growth, with strong earnings reports and strategic investments in AI technologies. 

NVIDIA remains a top buy due to its dominance in the GPU market, while Amazon and Microsoft are also recommended for their robust cloud services that support AI workloads. OpenAI is a strong player in the generative AI space, and Adobe's integration of AI into its creative software positions it well for future growth. 

Investors should consider holding positions in AMD and Intel as they navigate competitive challenges, while keeping an eye on their strategic developments. Overall, the sector is poised for significant growth, making it an attractive area for investment.

## Practical Tips

- Watch the temperature setting: Lower values (like 0) are usually better for factual responses
- Pay attention to the number of retrieved documents (`RETRIEVAL_K`): More isn't always better
- The system prompt is crucial: It sets the context and behavior of your bot

## Conclusion

You just learned how to create a chatbot and augment it with a retrieval tool using **LangChain**, this concludes the first part of our workshop!

In the next section, we'll discover **LangGraph** and show how it allows to build sophisticated and flexible LLMs workflows.